In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.display import HTML

import pandas as pd
from decimal import Decimal
import datetime
from base64 import b64decode, b64encode
from django.db.models import Sum
from calendar import monthrange, month_name
import os
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.colors import HexColor
from reportlab.lib.pagesizes import letter, A4
# import temp_pdf_generate.py
from main import temp_pdf_generate


### Super Sale Group Counter Report

In [4]:
from_date = '2020-11-02'
to_date = '2020-11-09'
business_id = 2439
super_sale_group_counter_values = list(SuperSaleGroup.objects.filter(delivery_date__range=[from_date, to_date], business_id=business_id).values_list('delivery_date', 'salegrouptransactiontrace__counter_amount', 'time_created'))                     
super_sale_group_counter_olumn = ['delivery_date', 'counter_amount', 'order_time']
super_sale_group_counter_df = pd.DataFrame(super_sale_group_counter_values, columns=super_sale_group_counter_olumn)
super_sale_group_counter_df = super_sale_group_counter_df.groupby('delivery_date').agg({'counter_amount': 'sum','order_time': 'first'}).reset_index()
super_sale_group_counter_df

,delivery_date,counter_amount,order_time
0,2020-11-03,24104.00,2020-11-02 09:13:11.951726+00:00
1,2020-11-04,24495.00,2020-11-03 08:31:25.897941+00:00
2,2020-11-05,24119.00,2020-11-04 08:41:34.730740+00:00
3,2020-11-06,23756.50,2020-11-05 09:18:00.256772+00:00
4,2020-11-07,23804.00,2020-11-06 08:50:56.870111+00:00
5,2020-11-08,21226.50,2020-11-07 06:53:28.605229+00:00
6,2020-11-09,23074.00,2020-11-08 05:15:34.768245+00:00


### Super Sale Group Online Report

In [5]:
from_date = '2020-11-02'
to_date = '2020-11-09'
business_id = 2439
super_sale_group_online_values = list(SuperSaleGroup.objects.filter(delivery_date__range=[from_date, to_date], business_id=business_id).values_list('delivery_date', 'salegrouptransactiontrace__transacted_amount', 'time_created'))                     
super_sale_group_online_olumn = ['delivery_date', 'online_amount', 'order_time']
super_sale_group_online_df = pd.DataFrame(super_sale_group_online_values, columns=super_sale_group_online_olumn)
super_sale_group_online_df = super_sale_group_online_df.groupby('delivery_date').agg({'online_amount': 'sum','order_time': 'first'}).reset_index()
super_sale_group_online_df

,delivery_date,online_amount,order_time
0,2020-11-03,0.00,2020-11-02 09:13:11.951726+00:00
1,2020-11-04,0.00,2020-11-03 08:31:25.897941+00:00
2,2020-11-05,0.00,2020-11-04 08:41:34.730740+00:00
3,2020-11-06,0.00,2020-11-05 09:18:00.256772+00:00
4,2020-11-07,0.00,2020-11-06 08:50:56.870111+00:00
5,2020-11-08,0.00,2020-11-07 06:53:28.605229+00:00
6,2020-11-09,0.00,2020-11-08 05:15:34.768245+00:00


### Super Sale Group wallet Report


In [4]:
from_date = '2020-11-03'
to_date = '2020-11-09'
business_id = 2439
data_dict = {
    'opening_wallet_balance': None,
    'wallet_transaction': None,
    'closing_wallet_balance': None,
}

super_sale_group_wallet_values = list(SuperSaleGroup.objects.filter(delivery_date__range=[from_date, to_date], business__code=business_id, salegrouptransactiontrace__wallet_amount__gt=0).values_list('delivery_date', 'salegrouptransactiontrace__wallet_amount', 'time_created', 'salegrouptransactiontrace__wallet_transaction__wallet_balance_after_transaction_approval'))                     
super_sale_group_wallet_olumn = ['delivery_date', 'wallet_amount', 'order_time', 'wallet_balance']
super_sale_group_wallet_df = pd.DataFrame(super_sale_group_wallet_values, columns=super_sale_group_wallet_olumn)
data_dict['wallet_transaction'] = super_sale_group_wallet_df.to_dict('r')

# opening_balance_closing_balace
#opening_balance
if SaleGroupTransactionTrace.objects.filter(delivery_date=from_date, super_sale_group__business__code=business_id).exists():
    wallet_balance_before_obj = SaleGroupTransactionTrace.objects.filter(delivery_date=from_date, super_sale_group__business__code=business_id).order_by('-id')[0]

    if wallet_balance_before_obj.counter_transaction is not None:
        data_dict['opening_wallet_balance'] = wallet_balance_before_obj.counter_transaction.wallet_balance_before_this_transaction
    elif wallet_balance_before_obj.bank_transaction is not None:
        data_dict['opening_wallet_balance'] = wallet_balance_before_obj.bank_transaction.wallet_balance_before_this_transaction
    else:
        data_dict['opening_wallet_balance'] = wallet_balance_before_obj.wallet_transaction.wallet_balance_before_this_transaction

#closing_balance
if SaleGroupTransactionTrace.objects.filter(delivery_date=to_date, super_sale_group__business__code=business_id).exists():
    wallet_balance_after_obj = SaleGroupTransactionTrace.objects.filter(delivery_date=to_date, super_sale_group__business__code=business_id).order_by('-id')[0]

    if wallet_balance_after_obj.counter_transaction is not None:
        data_dict['closing_wallet_balance'] = wallet_balance_after_obj.counter_transaction.wallet_balance_after_transaction_approval
    elif wallet_balance_after_obj.bank_transaction is not None:
        data_dict['closing_wallet_balance'] = wallet_balance_after_obj.bank_transaction.wallet_balance_after_transaction_approval
    else:
        data_dict['closing_wallet_balance'] = wallet_balance_after_obj.wallet_transaction.wallet_balance_after_transaction_approval

data_dict

{'opening_wallet_balance': None,
 'wallet_transaction': [],
 'closing_wallet_balance': None}

###  Total Transaction

In [6]:
from_date = '2020-11-03'
to_date = '2020-11-09'
business_type_ids = [1,2,9]

super_sale_group_obj = SuperSaleGroup.objects.filter(delivery_date__range=[from_date, to_date], business__business_type_id__in=business_type_ids)
super_sale_group_counter_values = list(super_sale_group_obj.values_list('business__code', 'business__user_profile__user__first_name', 'salegrouptransactiontrace__counter_amount', 'salegrouptransactiontrace__transacted_amount'))                     
super_sale_group_counter_olumn = ['business_code', 'agent_name', 'counter_amount', 'online_amount']
super_sale_group_counter_df = pd.DataFrame(super_sale_group_counter_values, columns=super_sale_group_counter_olumn)
super_sale_group_counter_df = super_sale_group_counter_df.groupby(['business_code', 'agent_name']).agg({'counter_amount': 'sum', 'online_amount': 'sum'}).reset_index()
super_sale_group_counter_df
# display(HTML(super_sale_group_counter_df.to_html()))

,business_code,agent_name,counter_amount,online_amount
0,101,GOPALAKRISHNAN.V,170161.50,0.00
1,1011,MAHALAXMI.R,6832.00,0.00
2,1025,VELLINGIRI.K,17429.00,0.00
3,1026,MOHAMAD HANSAR,17766.00,0.00
4,104,IRUDAYARAJ,68110.00,0.00
...,...,...,...,...
671,968,KALAISELVI,26782.00,0.00
672,97,VELUMANI.R,35700.00,0.00
673,970,SAMYNATHAN.A,2585.00,0.00
674,98,S.BALASUBRAMANIAN,69906.50,0.00


In [23]:
# wallet details
from_date = '2020-11-02'
to_date = '2020-11-09'
business_type_ids = [1,2,9]
wallet_balance_obj = SaleGroupTransactionTrace.objects.filter(delivery_date__range=[from_date, to_date], super_sale_group__business__business_type_id__in=business_type_ids).order_by('-id')                                             

#before_transaction
wallet_before_transaction_list = list(wallet_balance_obj.values_list('super_sale_group__business__code', 'counter_transaction__wallet_balance_before_this_transaction', 'bank_transaction__wallet_balance_before_this_transaction', 'wallet_transaction__wallet_balance_before_this_transaction'))                    
wallet_before_transaction_column = ['business_code', 'counter_wallet_amount', 'bank_wallet_amount', 'wallet_transaction_amount']
wallet_before_transaction_df = pd.DataFrame(wallet_before_transaction_list, columns=wallet_before_transaction_column)
# wallet_transaction_df = counter_wallet_df.dropna()

#after_transaction
wallet_after_transaction_list = list(wallet_balance_obj.values_list('super_sale_group__business__code', 'counter_transaction__wallet_balance_after_transaction_approval', 'bank_transaction__wallet_balance_after_transaction_approval', 'wallet_transaction__wallet_balance_after_transaction_approval'))
wallet_after_transaction_column = ['business_code', 'counter_wallet_amount', 'bank_wallet_amount', 'wallet_transaction_amount']
wallet_after_transaction_df = pd.DataFrame(wallet_after_transaction_list, columns=wallet_after_transaction_column)
# wallet_after_transaction_df = wallet_df.dropna()

wallet_before_transaction_df = wallet_before_transaction_df.groupby('business_code').agg({'counter_wallet_amount': 'last', 'bank_wallet_amount': 'last', 'wallet_transaction_amount': 'last'}).reset_index()
wallet_after_transaction_df = wallet_after_transaction_df.groupby('business_code').agg({'counter_wallet_amount': 'first', 'bank_wallet_amount': 'first', 'wallet_transaction_amount': 'first'}).reset_index()

# display(HTML(wallet_transaction_df.to_html()))
wallet_after_transaction_df

,business_code,counter_wallet_amount,bank_wallet_amount,wallet_transaction_amount
0,101,None,None,None
1,1011,None,None,None
2,1025,0.00,None,None
3,1026,None,None,None
4,104,None,None,None
...,...,...,...,...
672,968,None,None,None
673,97,None,None,None
674,970,None,None,None
675,98,None,None,None


In [24]:
wallet_before_list = []
for index,value in wallet_before_transaction_df.iterrows():    
    if value.counter_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_before_amount': value.counter_wallet_amount}
        wallet_before_list.append(data_dict)
    elif value.bank_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_before_amount': value.bank_wallet_amount}
        wallet_before_list.append(data_dict)
    else:
        data_dict = {
            'business_code': value.business_code,
            'wallet_before_amount': value.wallet_transaction_amount}
        wallet_before_list.append(data_dict)
wallet_before_df = pd.DataFrame(wallet_before_list)
wallet_before_df

wallet_after_list = []
for index,value in wallet_after_transaction_df.iterrows():    
    if value.counter_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_after_amount': value.counter_wallet_amount}
        wallet_after_list.append(data_dict)
    elif value.bank_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_after_amount': value.bank_wallet_amount}
        wallet_after_list.append(data_dict)
    else:
        data_dict = {
            'business_code': value.business_code,
            'wallet_after_amount': value.wallet_transaction_amount}
        wallet_after_list.append(data_dict)
wallet_before_df = pd.DataFrame(wallet_before_list)
wallet_after_df = pd.DataFrame(wallet_after_list)
wallet_after_df

display(HTML(wallet_before_df.to_html()))


,business_code,wallet_before_amount
0,101,0.00
1,1011,0.00
2,1025,0.00
3,1026,0.00
4,104,0.00
5,105,0.00
6,1056,0.00
7,106,0.00
8,1064,0.00
9,107,0.00


In [25]:
wallet_df = pd.merge(wallet_before_df, wallet_after_df, left_on='business_code', right_on='business_code', how='outer')
wallet_df = wallet_df.fillna(0)
display(HTML(wallet_df.to_html()))

,business_code,wallet_before_amount,wallet_after_amount
0,101,0.00,0
1,1011,0.00,0
2,1025,0.00,0.00
3,1026,0.00,0
4,104,0.00,0
5,105,0.00,0
6,1056,0.00,0
7,106,0.00,0
8,1064,0.00,0
9,107,0.00,0


In [37]:
agent_sale_list = list(Sale.objects.filter(sale_group__business__business_type_id__in=business_type_ids, sale_group__date__range=[from_date, to_date]).order_by('product_id').values_list('sale_group__business__code', 'product__short_name', 'count', 'cost'))
agent_sale_column = ['business_id', 'product_name', 'product_count', 'product_cost']
agent_sale_df = pd.DataFrame(agent_sale_list, columns=agent_sale_column)
agent_sale_df = agent_sale_df.groupby(['business_id', 'product_name']).agg({'product_cost': 'sum'})

#convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
agent_product_cost_df = pd.pivot_table(agent_sale_df, index='business_id', columns='product_name', aggfunc=min, fill_value=0)

#convert_pivot_table_to_normal_df
agent_product_cost_df.columns = agent_product_cost_df.columns.droplevel(0) #remove amount
agent_product_cost_df.columns.name = None  #remove categories
agent_product_cost_df = agent_product_cost_df.reset_index() #index to columns
agent_product_cost_df

{'101': {'business_id': {0: '101',
   1: '101',
   2: '101',
   3: '101',
   4: '101',
   5: '101'},
  'product_name': {0: 'Curd 150',
   1: 'Curd 500',
   2: 'FCM 500',
   3: 'STD 250',
   4: 'STD 500',
   5: 'TM 500'},
  'product_cost': {0: Decimal('2160.00'),
   1: Decimal('294.00'),
   2: Decimal('23103.00'),
   3: Decimal('27528.00'),
   4: Decimal('138462.00'),
   5: Decimal('2451.00')}},
 '1011': {'business_id': {6: '1011', 7: '1011', 8: '1011'},
  'product_name': {6: 'FCM 500', 7: 'STD 250', 8: 'STD 500'},
  'product_cost': {6: Decimal('2856.00'),
   7: Decimal('3072.00'),
   8: Decimal('1880.00')}},
 '1025': {'business_id': {9: '1025',
   10: '1025',
   11: '1025',
   12: '1025',
   13: '1025'},
  'product_name': {9: 'Curd 150',
   10: 'Curd 500',
   11: 'FCM 500',
   12: 'STD 250',
   13: 'STD 500'},
  'product_cost': {9: Decimal('440.00'),
   10: Decimal('784.00'),
   11: Decimal('5355.00'),
   12: Decimal('1920.00'),
   13: Decimal('8930.00')}},
 '1026': {'business_id': {14

### Transactin Combain

In [8]:
from_date = '2020-11-13'
to_date = '2020-11-13'
business_type_ids = [1,2,9]

file_name = 'total_transaction_'+str(from_date)+'_to_'+str(to_date)+'.xlsx'

super_sale_group_obj = SuperSaleGroup.objects.filter(delivery_date__range=[from_date, to_date], business__business_type_id__in=business_type_ids)
super_sale_group_values = list(super_sale_group_obj.values_list('business__code', 'business__user_profile__user__first_name', 'salegrouptransactiontrace__counter_amount', 'salegrouptransactiontrace__transacted_amount'))                     
super_sale_group_olumn = ['business_code', 'agent_name', 'counter_amount', 'online_amount']
super_sale_group_df = pd.DataFrame(super_sale_group_values, columns=super_sale_group_olumn)
super_sale_group_df = super_sale_group_df.groupby(['business_code', 'agent_name']).agg({'counter_amount': 'sum', 'online_amount': 'sum'}).reset_index()


# wallet details

wallet_balance_obj = SaleGroupTransactionTrace.objects.filter(delivery_date__range=[from_date, to_date], super_sale_group__business__business_type_id__in=business_type_ids).order_by('-id')                                             

#before_transaction
wallet_before_transaction_list = list(wallet_balance_obj.values_list('super_sale_group__business__code', 'counter_transaction__wallet_balance_before_this_transaction', 'bank_transaction__wallet_balance_before_this_transaction', 'wallet_transaction__wallet_balance_before_this_transaction'))                    
wallet_before_transaction_column = ['business_code', 'counter_wallet_amount', 'bank_wallet_amount', 'wallet_transaction_amount']
wallet_before_transaction_df = pd.DataFrame(wallet_before_transaction_list, columns=wallet_before_transaction_column)
# wallet_transaction_df = counter_wallet_df.dropna()

#after_transaction
wallet_after_transaction_list = list(wallet_balance_obj.values_list('super_sale_group__business__code', 'counter_transaction__wallet_balance_after_transaction_approval', 'bank_transaction__wallet_balance_after_transaction_approval', 'wallet_transaction__wallet_balance_after_transaction_approval'))
wallet_after_transaction_column = ['business_code', 'counter_wallet_amount', 'bank_wallet_amount', 'wallet_transaction_amount']
wallet_after_transaction_df = pd.DataFrame(wallet_after_transaction_list, columns=wallet_after_transaction_column)
# wallet_after_transaction_df = wallet_df.dropna()

wallet_before_transaction_df = wallet_before_transaction_df.groupby('business_code').agg({'counter_wallet_amount': 'last', 'bank_wallet_amount': 'last', 'wallet_transaction_amount': 'last'}).reset_index()
wallet_after_transaction_df = wallet_after_transaction_df.groupby('business_code').agg({'counter_wallet_amount': 'first', 'bank_wallet_amount': 'first', 'wallet_transaction_amount': 'first'}).reset_index()

wallet_before_list = []
for index,value in wallet_before_transaction_df.iterrows():    
    if value.counter_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_before_amount': value.counter_wallet_amount}
        wallet_before_list.append(data_dict)
    elif value.bank_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_before_amount': value.bank_wallet_amount}
        wallet_before_list.append(data_dict)
    else:
        data_dict = {
            'business_code': value.business_code,
            'wallet_before_amount': value.wallet_transaction_amount}
        wallet_before_list.append(data_dict)
wallet_before_df = pd.DataFrame(wallet_before_list)
wallet_before_df

wallet_after_list = []
for index,value in wallet_after_transaction_df.iterrows():    
    if value.counter_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_after_amount': value.counter_wallet_amount}
        wallet_after_list.append(data_dict)
    elif value.bank_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_after_amount': value.bank_wallet_amount}
        wallet_after_list.append(data_dict)
    else:
        data_dict = {
            'business_code': value.business_code,
            'wallet_after_amount': value.wallet_transaction_amount}
        wallet_after_list.append(data_dict)
wallet_before_df = pd.DataFrame(wallet_before_list)
wallet_after_df = pd.DataFrame(wallet_after_list)

wallet_df = pd.merge(wallet_before_df, wallet_after_df, left_on='business_code', right_on='business_code', how='outer')
wallet_df = wallet_df.fillna(0)

# merging supersale_df, wallet_df 
wallet_super_sale_group_df = pd.merge(super_sale_group_df, wallet_df, left_on='business_code', right_on='business_code', how='left')


# product_wise_sale_df
agent_sale_list = list(Sale.objects.filter(sale_group__business__business_type_id__in=business_type_ids, sale_group__date__range=[from_date, to_date]).order_by('product_id').values_list('sale_group__business__code', 'product__short_name', 'count', 'cost'))
agent_sale_column = ['business_code', 'product_name', 'product_count', 'product_cost']
agent_sale_df = pd.DataFrame(agent_sale_list, columns=agent_sale_column)
agent_sale_df = agent_sale_df.groupby(['business_code', 'product_name']).agg({'product_cost': 'sum'})

#convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
agent_product_cost_df = pd.pivot_table(agent_sale_df, index='business_code', columns='product_name', aggfunc=min, fill_value=0)

#convert_pivot_table_to_normal_df
agent_product_cost_df.columns = agent_product_cost_df.columns.droplevel(0) #remove amount
agent_product_cost_df.columns.name = None  #remove categories
agent_product_cost_df = agent_product_cost_df.reset_index() #index to columns

# merging wallet_super_sale_group_df and agent_product_cost_df

final_df = pd.merge(wallet_super_sale_group_df, agent_product_cost_df, left_on='business_code', right_on='business_code', how='left')

final_df = final_df.reindex(columns=['business_code', 'agent_name', 'wallet_before_amount', 'counter_amount', 'online_amount', 'wallet_after_amount', 'Total Amount Paid To Aavin For This Month', 'BM 200', 'Curd 100', 'Curd 150', 'Curd 500', 'FCM 1000', 'FCM 500', 'STD 250', 'STD 500', 'TM 500', 'Total Cost Of The Products Orderd For This Month', 'Difference'])
final_df = final_df.fillna(0)

final_df['Total Amount Paid To Aavin For This Month'] = final_df['wallet_before_amount'].astype('float') + final_df['counter_amount'].astype('float') + final_df['online_amount'].astype('float') - final_df['wallet_after_amount'].astype('float')
final_df['Total Cost Of The Products Orderd For This Month'] = final_df['BM 200'].astype('float') + final_df['Curd 100'].astype('float') + final_df['Curd 150'].astype('float') + final_df['Curd 500'].astype('float') + final_df['FCM 1000'].astype('float') + final_df['FCM 500'].astype('float') + final_df['STD 250'].astype('float') + final_df['STD 500'].astype('float') + final_df['TM 500'].astype('float')

final_df['Difference'] =  final_df['Total Cost Of The Products Orderd For This Month'].astype('float') - final_df['Total Amount Paid To Aavin For This Month'].astype('float')
final_df=final_df.rename(columns={'business_code': 'Booth Code', 'agent_name': 'Agent Name', 'wallet_before_amount': 'Wallet At The Start Of Month', 'counter_amount': 'Paid At Counter For This Month', 'online_amount': 'Paid Via Online For This Month', 'wallet_after_amount': 'Wallet At The End Of This Month'})

# final_df.to_excel(file_name, index=False)
# df = pd.read_excel(file_name)
# df.to_excel(file_name, index=False)
# display(HTML(final_df.to_html()))
final_df

,Booth Code,Agent Name,Wallet At The Start Of Month,Paid At Counter For This Month,Paid Via Online For This Month,Wallet At The End Of This Month,Total Amount Paid To Aavin For This Month,BM 200,Curd 100,Curd 150,Curd 500,FCM 1000,FCM 500,STD 250,STD 500,TM 500,Total Cost Of The Products Orderd For This Month,Difference
0,2439,KANNA MOBILES,0.00,0.00,283.00,0.00,283.00,0,0,0,0,0.0,0.0,48.00,235.00,0.0,283.00,0.0
1,2658,Jenifer,0.00,524.54,0.00,0.00,524.54,14.54,20.00,0,0,0.0,0.0,0,0,0.0,34.54,-490.0
2,9026,KANNA MOBILES\t,1687.00,0.00,0.00,300.00,1387.00,0,0,0.00,1100.00,0.0,0.0,0,0,0.0,1100.00,-287.0


In [2]:
Business.objects.get(code=2439).id
import pandas as pd

In [35]:
list1 = list(wallet_df['business_code']) 
list2 = list(agent_product_cost_df['business_id'])
list(set(list2) - set(list1))

['2439']

In [24]:
from_date = '2020-11-01'
to_date = '2020-11-10'
business_type_ids = [1,2,9]

file_name = 'total_transaction_'+str(from_date)+'_to_'+str(to_date)+'.xlsx'

super_sale_group_obj = SuperSaleGroup.objects.filter(delivery_date__range=[from_date, to_date], business__business_type_id__in=business_type_ids) #, business__code=1821
super_sale_group_values = list(super_sale_group_obj.values_list('business__code', 'business__user_profile__user__first_name', 'salegrouptransactiontrace__counter_amount', 'salegrouptransactiontrace__transacted_amount'))                     
super_sale_group_olumn = ['business_code', 'agent_name', 'counter_amount', 'online_amount']
super_sale_group_df = pd.DataFrame(super_sale_group_values, columns=super_sale_group_olumn)
super_sale_group_df = super_sale_group_df.groupby(['business_code', 'agent_name']).agg({'counter_amount': 'sum', 'online_amount': 'sum'}).reset_index()

# wallet details

wallet_balance_before_obj = SaleGroupTransactionTrace.objects.filter(delivery_date=from_date, super_sale_group__business__business_type_id__in=business_type_ids).order_by('id') #, super_sale_group__business__code=1821                                            
wallet_balance_after_obj = SaleGroupTransactionTrace.objects.filter(delivery_date=to_date, super_sale_group__business__business_type_id__in=business_type_ids).order_by('id')                                             

#before_transaction
wallet_before_transaction_list = list(wallet_balance_before_obj.values_list('id', 'super_sale_group__business__code', 'counter_transaction__wallet_balance_before_this_transaction', 'bank_transaction__wallet_balance_before_this_transaction', 'wallet_transaction__wallet_balance_before_this_transaction'))                    
wallet_before_transaction_column = ['id', 'business_code', 'counter_wallet_amount', 'bank_wallet_amount', 'wallet_transaction_amount']
wallet_before_transaction_df = pd.DataFrame(wallet_before_transaction_list, columns=wallet_before_transaction_column)
# wallet_transaction_df = counter_wallet_df.dropna()
#after_transaction
wallet_after_transaction_list = list(wallet_balance_after_obj.values_list('super_sale_group__business__code', 'counter_transaction__wallet_balance_after_transaction_approval', 'bank_transaction__wallet_balance_after_transaction_approval', 'wallet_transaction__wallet_balance_after_transaction_approval'))
wallet_after_transaction_column = ['business_code', 'counter_wallet_amount', 'bank_wallet_amount', 'wallet_transaction_amount']
wallet_after_transaction_df = pd.DataFrame(wallet_after_transaction_list, columns=wallet_after_transaction_column)
# wallet_after_transaction_df = wallet_df.dropna()

wallet_before_transaction_df = wallet_before_transaction_df.groupby('business_code').agg({'counter_wallet_amount': 'first', 'bank_wallet_amount': 'first', 'wallet_transaction_amount': 'first'}).reset_index()
wallet_after_transaction_df = wallet_after_transaction_df.groupby('business_code').agg({'counter_wallet_amount': 'last', 'bank_wallet_amount': 'last', 'wallet_transaction_amount': 'last'}).reset_index()

wallet_before_list = []
for index,value in wallet_before_transaction_df.iterrows():    
    if value.counter_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_before_amount': value.counter_wallet_amount}
        wallet_before_list.append(data_dict)
    elif value.bank_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_before_amount': value.bank_wallet_amount}
        wallet_before_list.append(data_dict)
    else:
        data_dict = {
            'business_code': value.business_code,
            'wallet_before_amount': value.wallet_transaction_amount}
        wallet_before_list.append(data_dict)
wallet_before_df = pd.DataFrame(wallet_before_list)
wallet_before_df

wallet_after_list = []
for index,value in wallet_after_transaction_df.iterrows():    
    if value.counter_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_after_amount': value.counter_wallet_amount}
        wallet_after_list.append(data_dict)
    elif value.bank_wallet_amount is not None:
        data_dict = {
            'business_code': value.business_code,
            'wallet_after_amount': value.bank_wallet_amount}
        wallet_after_list.append(data_dict)
    else:
        data_dict = {
            'business_code': value.business_code,
            'wallet_after_amount': value.wallet_transaction_amount}
        wallet_after_list.append(data_dict)
        
wallet_before_df = pd.DataFrame(wallet_before_list)
wallet_after_df = pd.DataFrame(wallet_after_list)

wallet_df = pd.merge(wallet_before_df, wallet_after_df, left_on='business_code', right_on='business_code', how='outer')
wallet_df = wallet_df.fillna(0)

# merging supersale_df, wallet_df 
wallet_super_sale_group_df = pd.merge(super_sale_group_df, wallet_df, left_on='business_code', right_on='business_code', how='left')

# product_wise_sale_df
agent_sale_list = list(Sale.objects.filter(sale_group__business__business_type_id__in=business_type_ids, sale_group__date__range=[from_date, to_date]).order_by('product_id').values_list('sale_group__business__code', 'product__short_name', 'count', 'cost'))
agent_sale_column = ['business_code', 'product_name', 'product_count', 'product_cost']
agent_sale_df = pd.DataFrame(agent_sale_list, columns=agent_sale_column)
agent_sale_df = agent_sale_df.groupby(['business_code', 'product_name']).agg({'product_cost': 'sum'})
#convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
agent_product_cost_df = pd.pivot_table(agent_sale_df, index='business_code', columns='product_name', aggfunc=min, fill_value=0)

#convert_pivot_table_to_normal_df
agent_product_cost_df.columns = agent_product_cost_df.columns.droplevel(0) #remove amount
agent_product_cost_df.columns.name = None  #remove categories
agent_product_cost_df = agent_product_cost_df.reset_index() #index to columns
# merging wallet_super_sale_group_df and agent_product_cost_df

final_df = pd.merge(wallet_super_sale_group_df, agent_product_cost_df, left_on='business_code', right_on='business_code', how='left')
final_df = final_df.reindex(columns=['business_code', 'agent_name', 'wallet_before_amount', 'counter_amount', 'online_amount', 'wallet_after_amount', 'Total Amount Paid To Aavin For This Month', 'BM 200', 'Curd 100', 'Curd 150', 'Curd 500', 'FCM 1000', 'FCM 500', 'STD 250', 'STD 500', 'TM 500', 'Total Cost Of The Products Orderd For This Month', 'Difference'])
final_df = final_df.fillna(0)

final_df['Total Amount Paid To Aavin For This Month'] = final_df['wallet_before_amount'].astype('float') + final_df['counter_amount'].astype('float') + final_df['online_amount'].astype('float') - final_df['wallet_after_amount'].astype('float')
final_df['Total Cost Of The Products Orderd For This Month'] = final_df['BM 200'].astype('float') + final_df['Curd 100'].astype('float') + final_df['Curd 150'].astype('float') + final_df['Curd 500'].astype('float') + final_df['FCM 1000'].astype('float') + final_df['FCM 500'].astype('float') + final_df['STD 250'].astype('float') + final_df['STD 500'].astype('float') + final_df['TM 500'].astype('float')

final_df['Difference'] =  final_df['Total Cost Of The Products Orderd For This Month'].astype('float') - final_df['Total Amount Paid To Aavin For This Month'].astype('float')
final_df=final_df.rename(columns={'business_code': 'Booth Code', 'agent_name': 'Agent Name', 'wallet_before_amount': 'Wallet At The Start Of Month', 'counter_amount': 'Paid At Counter For This Month', 'online_amount': 'Paid Via Online For This Month', 'wallet_after_amount': 'Wallet At The End Of This Month'})

# final_df.to_excel(file_name, index=False)
# df = pd.read_excel(file_name)
# df.to_excel(file_name, index=False)
display(HTML(final_df[final_df['Difference'] != 0].to_html()))
# display(HTML(final_df.to_html()))


,Booth Code,Agent Name,Wallet At The Start Of Month,Paid At Counter For This Month,Paid Via Online For This Month,Wallet At The End Of This Month,Total Amount Paid To Aavin For This Month,BM 200,Curd 100,Curd 150,Curd 500,FCM 1000,FCM 500,STD 250,STD 500,TM 500,Total Cost Of The Products Orderd For This Month,Difference
21,1107,SURESH KUMAR .D,0.0,0.00,25219.00,0.00,25219.0,0,0,1280.00,1617.00,0,1887.00,12504.00,8977.00,0,26265.0,1046.0
31,114,DHARAMARAJ.M,0.0,6204.00,0.00,0.00,6204.0,0,0,0,0,0,0,0,5640.00,0,5640.0,-564.0
38,1162,S.Radha,0.0,0.00,69191.00,0.00,69191.0,0,0,2800.00,6296.50,0,10812.00,16920.00,32383.00,0,69211.5,20.5
150,1821,KAVIRAJ.R,0.0,69480.00,17264.00,499.00,86245.0,0,0,960.00,0,0,6834.00,39000.00,39950.00,0,86744.0,499.0
238,2175,V.R.Jothimani,0.0,0.00,9765.00,1395.00,8370.0,0,0,0,0,0,1785.00,4032.00,3948.00,0,9765.0,1395.0
274,2275,E.PALANISAMY,0.0,0.00,58442.00,3989.00,54453.0,0,0,0,0,29682.00,25755.00,5616.00,34780.00,0,95833.0,41380.0
280,2293,UDAYAKUMAR .K,0.0,29328.00,3948.00,0.00,33276.0,0,0,0,0,0,0,0,37224.00,0,37224.0,3948.0
283,2296,R AKILA,0.0,0.00,131485.00,864.00,130621.0,0,0,0,3528.00,0,51102.00,31968.00,52828.00,0,139426.0,8805.0
338,2439,KANNA MOBILES,0.0,0.00,7398.00,2574.00,4824.0,0,0,0,0,0,1224.00,912.00,5922.00,0,8058.0,3234.0
392,2637,R.Sivamanikandan,0.0,0.00,25922.00,0.00,25922.0,0,0,5160.00,0,0,2295.00,4224.00,14100.00,989.00,26768.0,846.0


In [45]:
Business.objects.get(code=112).id

1296

In [1]:
list1 = list(wallet_df['business_code']) 
list2 = list(agent_product_cost_df['business_id'])
list(set(list2) - set(list1))

NameError: name 'wallet_df' is not defined

## -------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ##

## Nwe Report 

### Complete Transaction Version 2

In [49]:
from_date = '2020-11-1'
to_date = '2020-11-15'
business_id = 1296
super_sale_group_obj = SaleGroupTransactionTrace.objects.filter(delivery_date__range=[from_date, to_date], super_sale_group__business_id=business_id)

super_sale_group_values = list(super_sale_group_obj.values_list('super_sale_group__business__code', 'delivery_date', 'super_sale_group__time_created', 'super_sale_group__time_modified', 'super_sale_group__mor_sale_group__total_cost', 'super_sale_group__eve_sale_group__total_cost', 'counter_amount', 'transacted_amount', 'wallet_amount'))                              
super_sale_group_columns = ['business_code', 'delivery_date', 'time_created', 'time_modified', 'morning_sale_group', 'evening_sale_group', 'counter_amount', 'online_amount', 'wallet_amount']

agent_super_sale_group_df = pd.DataFrame(super_sale_group_values, columns=super_sale_group_columns)
agent_super_sale_group_df = agent_super_sale_group_df.groupby('delivery_date').agg({'time_created': 'last',  'time_modified': 'last', 'morning_sale_group': 'last', 'evening_sale_group': 'last', 'counter_amount': 'sum', 'online_amount': 'sum', 'wallet_amount': 'sum'})               
agent_super_sale_group_df = agent_super_sale_group_df.reset_index().fillna(0)
agent_super_sale_group_df['final_order_value'] = agent_super_sale_group_df['morning_sale_group'] + agent_super_sale_group_df['evening_sale_group']
agent_super_sale_group_df['total_amount_paid'] = agent_super_sale_group_df['counter_amount'] + agent_super_sale_group_df['online_amount'] + agent_super_sale_group_df['wallet_amount']
agent_super_sale_group_df['difference'] = agent_super_sale_group_df['final_order_value'] - agent_super_sale_group_df['total_amount_paid']
agent_super_sale_group_df = agent_super_sale_group_df.drop(columns=['morning_sale_group', 'evening_sale_group', 'counter_amount', 'online_amount', 'wallet_amount'])

agent_super_sale_group_df

,delivery_date,time_created,time_modified,final_order_value,total_amount_paid,difference
0,2020-11-05,2020-11-04 03:45:39.001589+00:00,2020-11-04 03:45:39.001628+00:00,3534.50,3534.50,0.00
1,2020-11-06,2020-11-04 03:48:21.875167+00:00,2020-11-04 03:48:21.875196+00:00,3534.50,3534.50,0.00
2,2020-11-07,2020-11-04 03:50:53.564134+00:00,2020-11-04 03:50:53.564166+00:00,3534.50,3534.50,0.00
3,2020-11-08,2020-11-07 03:40:33.393181+00:00,2020-11-07 03:40:33.393211+00:00,3370.00,3370.00,0.00
4,2020-11-09,2020-11-08 05:59:45.262420+00:00,2020-11-08 05:59:45.262472+00:00,3370.00,3370.00,0.00
5,2020-11-10,2020-11-09 03:15:55.274900+00:00,2020-11-09 03:15:55.274949+00:00,3816.50,3816.50,0.00
6,2020-11-11,2020-11-09 03:18:36.647440+00:00,2020-11-09 03:18:36.647471+00:00,3816.50,3816.50,0.00


In [50]:
data_dict = {}
# trase_value = super_sale_group_obj.values_list('delivery_date', 'salegrouptransactiontrace__counter_amount', 'salegrouptransactiontrace__transacted_amount',     
for trace in super_sale_group_obj:
    if not trace.delivery_date in data_dict:
        data_dict[trace.delivery_date] = {'transaction_details':[]}                     
    
    if trace.counter_transaction is not None:
        data_dict[trace.delivery_date]['transaction_details'].append({'scop': 'Counter', 'amount': trace.counter_amount, 'time_created': trace.counter_transaction.time_created})    
        
    if trace.bank_transaction is not None:
        data_dict[trace.delivery_date]['transaction_details'].append({'scop': 'Online', 'amount': trace.transacted_amount, 'time_created': trace.bank_transaction.time_created})
        
    if trace.wallet_transaction is not None:
        data_dict[trace.delivery_date]['transaction_details'].append({'scop': 'Wallet', 'amount': trace.wallet_amount, 'time_created': trace.wallet_transaction.time_created})

transaction_df = pd.DataFrame(data_dict)

transaction_df = transaction_df.T
transaction_df.columns


Index(['transaction_details'], dtype='object')

In [56]:
super_sale_group_df = pd.merge(agent_super_sale_group_df, transaction_df, left_on='delivery_date', right_index=True, how='left')
super_sale_group_df

,delivery_date,time_created,time_modified,final_order_value,total_amount_paid,difference,transaction_details
0,2020-11-05,2020-11-04 03:45:39.001589+00:00,2020-11-04 03:45:39.001628+00:00,3534.50,3534.50,0.00,"[{'scop': 'Online', 'amount': 3534.50, 'time_c..."
1,2020-11-06,2020-11-04 03:48:21.875167+00:00,2020-11-04 03:48:21.875196+00:00,3534.50,3534.50,0.00,"[{'scop': 'Online', 'amount': 3534.50, 'time_c..."
2,2020-11-07,2020-11-04 03:50:53.564134+00:00,2020-11-04 03:50:53.564166+00:00,3534.50,3534.50,0.00,"[{'scop': 'Online', 'amount': 3534.50, 'time_c..."
3,2020-11-08,2020-11-07 03:40:33.393181+00:00,2020-11-07 03:40:33.393211+00:00,3370.00,3370.00,0.00,"[{'scop': 'Online', 'amount': 3370.00, 'time_c..."
4,2020-11-09,2020-11-08 05:59:45.262420+00:00,2020-11-08 05:59:45.262472+00:00,3370.00,3370.00,0.00,"[{'scop': 'Online', 'amount': 3370.00, 'time_c..."
5,2020-11-10,2020-11-09 03:15:55.274900+00:00,2020-11-09 03:15:55.274949+00:00,3816.50,3816.50,0.00,"[{'scop': 'Online', 'amount': 3816.50, 'time_c..."
6,2020-11-11,2020-11-09 03:18:36.647440+00:00,2020-11-09 03:18:36.647471+00:00,3816.50,3816.50,0.00,"[{'scop': 'Online', 'amount': 3816.50, 'time_c..."


In [57]:
#total daywise sale details all 
#---------------------
sale_group_ids = list(SaleGroup.objects.filter(business_id=business_id, date__range=[from_date, to_date]).values_list('id', flat=True))
sale_obj = Sale.objects.filter(sale_group_id__in=sale_group_ids, product__is_active=True)
sale_list = list(sale_obj.values_list('id', 'sale_group', 'sale_group__business_id',
                                      'sale_group__date', 'sale_group__session',
                                      'sale_group__session__display_name', 'sale_group__ordered_via',
                                      'sale_group__ordered_via__name', 'sale_group__payment_status__name',
                                      'sale_group__sale_status__name', 'sale_group__total_cost', 'product',
                                      'product__name', 'count', 'cost'))
sale_column = ['sale_id', 'sale_group_id', 'business_id', 'date', 'session_id', 'session_name',
              'ordered_via_id', 'ordered_via_name', 'payment_status',
              'sale_status',
              'session_wise_price', 'product_id', 'product_name', 'quantity', 'product_cost']
sale_df = pd.DataFrame(sale_list, columns=sale_column)

sale_df

,sale_id,sale_group_id,business_id,date,session_id,session_name,ordered_via_id,ordered_via_name,payment_status,sale_status,session_wise_price,product_id,product_name,quantity,product_cost
0,1560536,507762,1296,2020-11-01,1,Mor,1,Mobile,Paid,Ordered,3534.50,2,STD,144.0,3384.00
1,1560537,507762,1296,2020-11-01,1,Mor,1,Mobile,Paid,Ordered,3534.50,1,Tonned,7.0,150.50
2,1562923,508601,1296,2020-11-02,1,Mor,1,Mobile,Paid,Ordered,3534.50,2,STD,144.0,3384.00
3,1562924,508601,1296,2020-11-02,1,Mor,1,Mobile,Paid,Ordered,3534.50,1,Tonned,7.0,150.50
4,1562925,508602,1296,2020-11-03,1,Mor,1,Mobile,Paid,Ordered,3534.50,2,STD,144.0,3384.00
5,1562926,508602,1296,2020-11-03,1,Mor,1,Mobile,Paid,Ordered,3534.50,1,Tonned,7.0,150.50
6,1562927,508603,1296,2020-11-04,1,Mor,1,Mobile,Paid,Ordered,3534.50,2,STD,144.0,3384.00
7,1562928,508603,1296,2020-11-04,1,Mor,1,Mobile,Paid,Ordered,3534.50,1,Tonned,7.0,150.50
8,1571840,511760,1296,2020-11-05,1,Mor,1,Mobile,Paid,Ordered,3534.50,2,STD,144.0,3384.00
9,1571841,511760,1296,2020-11-05,1,Mor,1,Mobile,Paid,Ordered,3534.50,1,Tonned,7.0,150.50


In [58]:
# daywise sale details date_seprated
sale_value_list = []
for date in super_sale_group_df['delivery_date']:
    date_wise_df = sale_df[sale_df['date']==date]
    data_dict = {
        'delivery_date': date,
        'total_sale_details': date_wise_df.to_dict('r')
    }
    sale_value_list.append(data_dict)
day_wise_sale_df = pd.DataFrame(sale_value_list)
day_wise_sale_df

/home/suneshrajan/Software/Virtualenv/aavin/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,delivery_date,total_sale_details
0,2020-11-05,"[{'sale_id': 1571840, 'sale_group_id': 511760,..."
1,2020-11-06,"[{'sale_id': 1571842, 'sale_group_id': 511761,..."
2,2020-11-07,"[{'sale_id': 1571844, 'sale_group_id': 511762,..."
3,2020-11-08,"[{'sale_id': 1580665, 'sale_group_id': 514896,..."
4,2020-11-09,"[{'sale_id': 1583693, 'sale_group_id': 515979,..."
5,2020-11-10,"[{'sale_id': 1586644, 'sale_group_id': 517001,..."
6,2020-11-11,"[{'sale_id': 1586646, 'sale_group_id': 517002,..."


In [59]:
final_df = pd.merge(super_sale_group_df, day_wise_sale_df, left_on='delivery_date', right_on='delivery_date')

In [60]:
final_df = final_df.reindex(columns=['delivery_date', 'time_created', 'time_modified', 'total_sale_details', 'final_order_value', 'total_amount_paid', 'difference', 'transaction_details'])

In [61]:
final_df

,delivery_date,time_created,time_modified,total_sale_details,final_order_value,total_amount_paid,difference,transaction_details
0,2020-11-05,2020-11-04 03:45:39.001589+00:00,2020-11-04 03:45:39.001628+00:00,"[{'sale_id': 1571840, 'sale_group_id': 511760,...",3534.50,3534.50,0.00,"[{'scop': 'Online', 'amount': 3534.50, 'time_c..."
1,2020-11-06,2020-11-04 03:48:21.875167+00:00,2020-11-04 03:48:21.875196+00:00,"[{'sale_id': 1571842, 'sale_group_id': 511761,...",3534.50,3534.50,0.00,"[{'scop': 'Online', 'amount': 3534.50, 'time_c..."
2,2020-11-07,2020-11-04 03:50:53.564134+00:00,2020-11-04 03:50:53.564166+00:00,"[{'sale_id': 1571844, 'sale_group_id': 511762,...",3534.50,3534.50,0.00,"[{'scop': 'Online', 'amount': 3534.50, 'time_c..."
3,2020-11-08,2020-11-07 03:40:33.393181+00:00,2020-11-07 03:40:33.393211+00:00,"[{'sale_id': 1580665, 'sale_group_id': 514896,...",3370.00,3370.00,0.00,"[{'scop': 'Online', 'amount': 3370.00, 'time_c..."
4,2020-11-09,2020-11-08 05:59:45.262420+00:00,2020-11-08 05:59:45.262472+00:00,"[{'sale_id': 1583693, 'sale_group_id': 515979,...",3370.00,3370.00,0.00,"[{'scop': 'Online', 'amount': 3370.00, 'time_c..."
5,2020-11-10,2020-11-09 03:15:55.274900+00:00,2020-11-09 03:15:55.274949+00:00,"[{'sale_id': 1586644, 'sale_group_id': 517001,...",3816.50,3816.50,0.00,"[{'scop': 'Online', 'amount': 3816.50, 'time_c..."
6,2020-11-11,2020-11-09 03:18:36.647440+00:00,2020-11-09 03:18:36.647471+00:00,"[{'sale_id': 1586646, 'sale_group_id': 517002,...",3816.50,3816.50,0.00,"[{'scop': 'Online', 'amount': 3816.50, 'time_c..."


###  Super Sale Group Data Append

In [11]:
sale_group_obj = SaleGroup.objects.filter(date__month=11).order_by('session_id')
super_sale_group_obj = SuperSaleGroup.objects.all()
sale_group_transaction_trace_obj = SaleGroupTransactionTrace.objects.filter(delivery_date__month=11)
counter_employee_trace_sale_group_map_obj = CounterEmployeeTraceSaleGroupMap.objects.all()
count = 0
for sale_group in sale_group_obj:
    if super_sale_group_obj.filter(delivery_date=sale_group.date, business_id=sale_group.business_id).exists():     
        temp_super_sale_group_obj = super_sale_group_obj.get(delivery_date=sale_group.date, business_id=sale_group.business_id) 
        if sale_group.session_id == 1:
            if temp_super_sale_group_obj.mor_sale_group_id is None:  
                temp_super_sale_group_obj.mor_sale_group_id = sale_group.id
#                 temp_super_sale_group_obj.save()
        else:
            if temp_super_sale_group_obj.eve_sale_group_id is None:  
                temp_super_sale_group_obj.eve_sale_group_id = sale_group.id
#                 temp_super_sale_group_obj.save()
    else:
        count += 1
        temp_super_sale_group_obj = SuperSaleGroup(business_id=sale_group.business_id, 
                                                       delivery_date=sale_group.date, 
                                                       time_created=sale_group.time_created, 
                                                       time_modified=sale_group.time_modified)
        if sale_group.session_id == 1:
            temp_super_sale_group_obj.mor_sale_group_id=sale_group.id
        else:
            temp_super_sale_group_obj.eve_sale_group_id=sale_group.id

        temp_super_sale_group_obj.save()
            
            
for temp_super_sale_group_obj in super_sale_group_obj:        
    if not sale_group_transaction_trace_obj.filter(delivery_date=temp_super_sale_group_obj.delivery_date, super_sale_group_id=temp_super_sale_group_obj.id).exists():                   
        counter_amount = 0
        online_amount = 0
        wallet_amount = 0
        time_created = None
        time_modified = None
        sale_group_ids = []
        
        if temp_super_sale_group_obj.mor_sale_group is not None:
            sale_group_ids.append(temp_super_sale_group_obj.mor_sale_group.id)
            if temp_super_sale_group_obj.mor_sale_group.ordered_via_id == 2:
                counter_id = counter_employee_trace_sale_group_map_obj.get(sale_group=temp_super_sale_group_obj.mor_sale_group.id).counter_employee_trace.counter.id
                counter_amount += temp_super_sale_group_obj.mor_sale_group.total_cost
            else:
                counter_id = 23
                online_amount += temp_super_sale_group_obj.mor_sale_group.total_cost
                
            time_created = temp_super_sale_group_obj.mor_sale_group.time_created
            time_modified = temp_super_sale_group_obj.mor_sale_group.time_modified 
                
        if temp_super_sale_group_obj.eve_sale_group is not None:
            sale_group_ids.append(temp_super_sale_group_obj.eve_sale_group.id)
            if temp_super_sale_group_obj.eve_sale_group.ordered_via_id == 2:
                counter_id = counter_employee_trace_sale_group_map_obj.get(sale_group=temp_super_sale_group_obj.eve_sale_group.id).counter_employee_trace.counter.id
                counter_amount += temp_super_sale_group_obj.eve_sale_group.total_cost
            else:
                counter_id = 23
                online_amount += temp_super_sale_group_obj.eve_sale_group.total_cost
            
            time_created = temp_super_sale_group_obj.eve_sale_group.time_created
            time_modified = temp_super_sale_group_obj.eve_sale_group.time_modified
                
                
        temp_sale_group_transaction_trace_obj = SaleGroupTransactionTrace(delivery_date=temp_super_sale_group_obj.delivery_date, 
                                                                          super_sale_group_id=temp_super_sale_group_obj.id, 
                                                                          sale_group_order_type_id=1, 
                                                                          counter_amount=counter_amount, 
                                                                          transacted_amount=online_amount, 
                                                                          wallet_amount=wallet_amount, 
                                                                          bank_transaction=None, 
                                                                          wallet_transaction=None, 
                                                                          counter_transaction=None, 
                                                                          is_wallet_used=False, 
                                                                          order_sale_group_json=None, 
                                                                          counter_id=counter_id, 
                                                                          time_created=time_created,
                                                                          time_modified=time_modified)
        
#         sale_group_ids = list(SaleGroup.objects.filter(business_id=business_id, date__range=[from_date, to_date]).values_list('id', flat=True))
        sale_obj = Sale.objects.filter(sale_group_id__in=sale_group_ids, product__is_active=True)
        sale_list = list(sale_obj.values_list('id', 'sale_group', 'sale_group__business_id',
                                              'sale_group__date', 'sale_group__session',
                                              'sale_group__session__display_name', 'sale_group__ordered_via',
                                              'sale_group__ordered_via__name', 'sale_group__payment_status__name',
                                              'sale_group__sale_status__name', 'sale_group__total_cost', 'product',
                                              'product__name', 'count', 'cost'))
        sale_column = ['sale_id', 'sale_group_id', 'business_id', 'date', 'session_id', 'session_name',
                      'ordered_via_id', 'ordered_via_name', 'payment_status',
                      'sale_status',
                      'session_wise_price', 'product_id', 'product_name', 'quantity', 'product_cost']
        sale_df = pd.DataFrame(sale_list, columns=sale_column)
        
        temp_sale_group_transaction_trace_obj.order_sale_group_json = sale_df.to_dict('r')
        
#         temp_sale_group_transaction_trace_obj.save()
        
        print(temp_sale_group_transaction_trace_obj)
            
sale_df

/home/suneshrajan/Software/Virtualenv/aavin/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupT

SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupT

SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupT

SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupT

SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupT

SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupT

SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupTransactionTrace object
SaleGroupT

,sale_id,sale_group_id,business_id,date,session_id,session_name,ordered_via_id,ordered_via_name,payment_status,sale_status,session_wise_price,product_id,product_name,quantity,product_cost
0,1568867,510681,2791,2020-11-03,2,Eve,1,Mobile,Paid,Ordered,2161.00,25,Curd,10.0,245.00
1,1568868,510681,2791,2020-11-03,2,Eve,1,Mobile,Paid,Ordered,2161.00,7,Curd,10.0,80.00
2,1568869,510681,2791,2020-11-03,2,Eve,1,Mobile,Paid,Ordered,2161.00,3,STD,12.0,144.00
3,1568870,510681,2791,2020-11-03,2,Eve,1,Mobile,Paid,Ordered,2161.00,2,STD,72.0,1692.00


In [28]:
for i in super_sale_group_obj:
    if i.mor_sale_group is None:
            print(i.id)

2
27
42
44
55
58
67
71
73
74
75
97
136
138
175
185
201
202
218
227
229
272
308
321
397
408
465
468
474
481
528
720
686
702
703
708
710
711
722
728
732
752
794
801
814
821
833
835
822
849
852
907
867
873
885
909
939
1051
1058
1069
1187
1122
1196
1200
1228
1252
1406
1417
1440
1459
1463
1464
1466
1468
1482
1484
1469
1485
1480
1499
1537
1541
1549
1551
1556
1563
1573
1582
1697
1711
1712
1718
1748
1752
1755
1765
1775
1787
1790
1806
1824
1859
1869
1935
2021
2049
2071
2187
2191
2195
2197
2198
2199
2200
2201
2212
2214
2218
2229
2245
2267
2299
2416
2337
2313
2316
2338
2339
2343
2350
2374
2417
2421
2433
2443
2491
2505
2506
2517
2520
2525
2537
2543
2565
2569
2574
2653
2726
2728
2782
2804
2956
2931
2934
2935
2936
2937
2938
2940
2950
2954
2955
2967
2972
2989
3010
2948
3024
3084
3122
3128
3137
3154
3156
3169
3177
3180
3188
3193
3202
3203
3215
3226
3236
3271
3277
3291
3292
3337
3409
3419
3422
3575
3635
3639
3676
3711
3667
3675
3688
3691
3694
3696
3698
3697
3710
3723
3724
3728
3736
3744
3746
3748
3749


In [22]:
count = 0
transaction_obj = TransactionLog.objects.all()
for stt in SaleGroupTransactionTrace.objects.filter(counter_transaction__isnull=True): 
#     print(stt.id,stt.super_sale_group.business.code, stt.super_sale_group.time_created, stt.transacted_amount, stt.super_sale_group.id)
    if not stt.counter_amount == 0:
        print(stt.counter_amount)
        try:
            i = transaction_obj.get(date=stt.time_created, transacted_by_id=stt.super_sale_group.business.user_profile.user.id, transaction_status_id=2, amount=stt.counter_amount, transaction_direction_id=1)
            stt.bank_transaction_id=i.id
#             stt.save()
            print(i.amount) 
        except Exception as e:
            print(e)
            count +=1
        print('---------------------------------------------------------------------------')
print(count)

1175.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
1175.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
141.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
3172.50
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
94.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
1645.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
1645.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
235.00
TransactionLog matching query does not exist.
----------------------------------------

TransactionLog matching query does not exist.
---------------------------------------------------------------------------
141.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
235.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
235.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
235.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
141.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
47.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
94.00
TransactionLog matching query does not exist.
-----------------------------------------------------

TransactionLog matching query does not exist.
---------------------------------------------------------------------------
7990.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
94.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
141.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
188.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
-76.50
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
3290.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
235.00
TransactionLog matching query does not exist.
--------------------------------------------------

TransactionLog matching query does not exist.
---------------------------------------------------------------------------
117236.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
235.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
501.64
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
705.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
52004.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
15480.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
11940.00
TransactionLog matching query does not exist.
------------------------------------------

TransactionLog matching query does not exist.
---------------------------------------------------------------------------
9400.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
3655.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
141.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
423.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
235.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
141.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
188.00
TransactionLog matching query does not exist.
-------------------------------------------------

...

TransactionLog matching query does not exist.
---------------------------------------------------------------------------
5278.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
27005.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
17702.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
7068.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
23911.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
729.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
13124.00
TransactionLog matching query does not exist.
-----------------------------------------

TransactionLog matching query does not exist.
---------------------------------------------------------------------------
15410.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
19906.50
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
26987.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
376.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
20487.50
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
1882.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
20078.00
TransactionLog matching query does not exist.
----------------------------------------

TransactionLog matching query does not exist.
---------------------------------------------------------------------------
9669.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
16008.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
564.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
6768.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
5000.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
1400.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
31920.00
TransactionLog matching query does not exist.
-------------------------------------------

TransactionLog matching query does not exist.
---------------------------------------------------------------------------
496.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
4485.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
5179.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
12932.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
15817.50
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
40012.50
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
5670.00
TransactionLog matching query does not exist.
------------------------------------------

TransactionLog matching query does not exist.
---------------------------------------------------------------------------
9464.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
10290.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
25125.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
188.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
1082.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
6828.00
TransactionLog matching query does not exist.
---------------------------------------------------------------------------
17395.00
TransactionLog matching query does not exist.
------------------------------------------

### Customer Report 1 data upload script

In [10]:

# before run this need to load the json
import pandas as pd
import numpy as np
import requests
import io

try:
    from StringIO import StringIO
except ImportError:
    
    from io import StringIO
    from io import BytesIO
from decimal import Decimal
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1Ec0GeoGikLQgkTMvjRrvJyRyU2xONBowjrcHu592fgw/export?format=xlsx'
r = requests.get(google_sheet_url)
data = r.content
df = pd.read_excel(BytesIO(data),sheet_name='Sheet1')
df = df.fillna(0)
group_by_df = df.dropna().dropna().drop(columns=['S.No']).groupby('UNION')
# group_by_df = pd.read_excel('AAVIN .xlsx').dropna().drop(columns=['S.No']).groupby('UNION')

iunion_dict = group_by_df.apply(lambda x:x.to_dict('r')).to_dict() 
# pd.read_excel('AAVIN .xlsx').dropna()
iunion_dict
# group_by_df

{'EDCMPU': [{'NAME': 'L.KANNAN',
   'CUSTOMER CODE': 'CBE1485',
   'UNION': 'EDCMPU'},
  {'NAME': 'K.NACHIMUTHU', 'CUSTOMER CODE': 'CBE1120270', 'UNION': 'EDCMPU'}],
 'NJF.OOTY': [{'NAME': 'T.R.CAHNDRAN',
   'CUSTOMER CODE': 'CBE10695',
   'UNION': 'NJF.OOTY'},
  {'NAME': 'M.KARTHIKEYAN', 'CUSTOMER CODE': 'CBE10694', 'UNION': 'NJF.OOTY'},
  {'NAME': 'V.VIGNARAJAN', 'CUSTOMER CODE': 'CBE10692', 'UNION': 'NJF.OOTY'},
  {'NAME': 'ALPHONS MARIA JOSEPH',
   'CUSTOMER CODE': 'CBE10691',
   'UNION': 'NJF.OOTY'}],
 'TCMPF': [{'NAME': 'ER.RAVIKUMAR',
   'CUSTOMER CODE': 'CBE1120599',
   'UNION': 'TCMPF'},
  {'NAME': ' U.V.RAJSEKER', 'CUSTOMER CODE': 'CBE1120601', 'UNION': 'TCMPF'}],
 'TCMPF,RTD': [{'NAME': 'DR.S.HARIHARAN',
   'CUSTOMER CODE': 'CBE1593',
   'UNION': 'TCMPF,RTD'},
  {'NAME': 'R,NARAYANSAMY', 'CUSTOMER CODE': 'CBE10078', 'UNION': 'TCMPF,RTD'},
  {'NAME': 'G.VISVANATHAN', 'CUSTOMER CODE': 'CBE10219', 'UNION': 'TCMPF,RTD'},
  {'NAME': 'N.GANESAN', 'CUSTOMER CODE': 'CBE91', 'UNION':

In [12]:
union_for_icustomer_obj = UnionForIcustomer.objects.all()
icustomer_obj = ICustomer.objects.all()
for union in iunion_dict:
    customer_union_id = UnionForIcustomer.objects.get(name=union)
    for customer in iunion_dict[union]:
        emp_customer_code = 'EXEMP' + str(customer['CUSTOMER CODE'])[3:]

        if icustomer_obj.filter(customer_code=customer['CUSTOMER CODE']).exists():
            single_customer_obj = icustomer_obj.get(customer_code=customer['CUSTOMER CODE'])
            single_customer_obj.union_for_icustomer_id = customer_union_id
            single_customer_obj.save()
            print('in employee', customer['CUSTOMER CODE'], union_for_icustomer_obj.get(name=union).id, union)

        elif icustomer_obj.filter(customer_code=emp_customer_code).exists():
            single_customer_obj = icustomer_obj.get(customer_code=str(emp_customer_code))
            single_customer_obj.union_for_icustomer_id = customer_union_id
            single_customer_obj.save()
            print(emp_customer_code, union_for_icustomer_obj.get(name=union).id, union)

        else:
            print(f'Unmapped Customers = { emp_customer_code }')
    print()
    


EXEMP1485 12 EDCMPU
in employee CBE1120270 12 EDCMPU

EXEMP10695 9 NJF.OOTY
EXEMP10694 9 NJF.OOTY
EXEMP10692 9 NJF.OOTY
EXEMP10691 9 NJF.OOTY

in employee CBE1120599 6 TCMPF
in employee CBE1120601 6 TCMPF

EXEMP1593 7 TCMPF,RTD
EXEMP10078 7 TCMPF,RTD
EXEMP10219 7 TCMPF,RTD
EXEMP91 7 TCMPF,RTD
EXEMP422 7 TCMPF,RTD
EXEMP10218 7 TCMPF,RTD
EXEMP198 7 TCMPF,RTD
EXEMP1722 7 TCMPF,RTD
EXEMP10232 7 TCMPF,RTD
EXEMP10211 7 TCMPF,RTD
EXEMP245 7 TCMPF,RTD
EXEMP20093 7 TCMPF,RTD
EXEMP1223 7 TCMPF,RTD
EXEMP10690 7 TCMPF,RTD
EXEMP2505 7 TCMPF,RTD
EXEMP10164 7 TCMPF,RTD
EXEMP20095 7 TCMPF,RTD
EXEMP20094 7 TCMPF,RTD
EXEMP1707 7 TCMPF,RTD
EXEMP20096 7 TCMPF,RTD
EXEMP10217 7 TCMPF,RTD
EXEMP10220 7 TCMPF,RTD
EXEMP91 7 TCMPF,RTD
EXEMP10693 7 TCMPF,RTD
EXEMP1223 7 TCMPF,RTD



In [13]:
for customer in icustomer_obj.filter(union_for_icustomer_id__isnull=True, customer_type_id__in=[2,3]):
    if customer.customer_type_id == 2:
        customer.union_for_icustomer_id = 5
        customer.save()
    else:
        customer.union_for_icustomer_id = 2
        customer.save()


In [81]:
set(DailySessionllyBusinessllySale.objects.filter(business__code='8123').values_list('business_type__name'))

{('Union Parlour With Discount',)}

### Script for socity data upload

In [6]:
#sosity product price dict
import pandas as pd
business_wise_product_discount_list = list(BusinessWiseProductDiscount.objects.filter(business__business_type_id__in=[4, 5, 6, 10, 12]).values_list('business_id', 'product_id', 'discounted_price'))
business_wise_product_discount_column = ['business_id', 'product_id', 'discounted_price']
business_wise_product_discount_df = pd.DataFrame(business_wise_product_discount_list, columns=business_wise_product_discount_column)
business_wise_product_discount_df = business_wise_product_discount_df.groupby(['business_id', 'product_id']).agg({'discounted_price':max})

# convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
business_wise_product_discount_df = pd.pivot_table(business_wise_product_discount_df, index='business_id', columns='product_id', aggfunc=min, fill_value=0)

#convert_pivot_table_to_normal_df
business_wise_product_discount_df.columns = business_wise_product_discount_df.columns.droplevel(0) #remove amount
business_wise_product_discount_df.columns.name = None  #remove categories
business_wise_product_discount_df = business_wise_product_discount_df.reset_index() #index to columns

#sosity product price dict
business_wise_product_discount_df = business_wise_product_discount_df.groupby('business_id')
business_wise_product_discount_dict = business_wise_product_discount_df.apply(lambda x:x.to_dict('r')[0]).to_dict() 

business_wise_product_discount_dict
# business_wise_product_discount_df
# business_wise_product_discount_df

/Users/hari/Software/virtualenv/aavin/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


{1439: {'business_id': 1439,
  1: Decimal('20.00'),
  2: Decimal('22.00'),
  3: Decimal('11.50'),
  4: Decimal('24.00'),
  6: Decimal('48.00'),
  7: Decimal('10.00'),
  8: Decimal('8.00'),
  9: 0,
  10: Decimal('12.00'),
  21: 0,
  23: 0,
  24: 0,
  25: Decimal('30.00'),
  26: 0},
 1440: {'business_id': 1440,
  1: Decimal('20.00'),
  2: Decimal('22.00'),
  3: Decimal('11.50'),
  4: Decimal('24.00'),
  6: Decimal('48.00'),
  7: Decimal('10.00'),
  8: Decimal('8.00'),
  9: 0,
  10: Decimal('12.00'),
  21: 0,
  23: 0,
  24: 0,
  25: Decimal('30.00'),
  26: 0},
 1441: {'business_id': 1441,
  1: Decimal('20.00'),
  2: Decimal('22.00'),
  3: Decimal('11.50'),
  4: Decimal('24.00'),
  6: Decimal('48.00'),
  7: Decimal('10.00'),
  8: Decimal('8.00'),
  9: 0,
  10: Decimal('12.00'),
  21: 0,
  23: 0,
  24: 0,
  25: Decimal('30.00'),
  26: 0},
 1442: {'business_id': 1442,
  1: Decimal('19.18'),
  2: Decimal('21.10'),
  3: Decimal('11.03'),
  4: Decimal('23.02'),
  6: Decimal('46.04'),
  7: Decim

In [4]:
def serve_product_and_session_list(user_profile_id):
    business_id = BusinessAgentMap.objects.get(business__user_profile_id=user_profile_id, is_active=True).business_id
    business_type_id = Business.objects.get(id=business_id).business_type_id
    commission_product_ids = list(
        BusinessProductConcessionMap.objects.filter(business_type_id=business_type_id, product__is_active=True,
                                                    concession_type_id=1).values_list(
            'product_id', flat=True))
    discount_product_ids = list(
        BusinessProductConcessionMap.objects.filter(business_type_id=business_type_id, product__is_active=True,
                                                    concession_type_id=2).values_list(
            'product_id', flat=True))
    product_obj = Product.objects.filter(is_active=True, id__in=commission_product_ids).order_by(
        'display_ordinal').exclude(group_id=3)
    product_list = list(product_obj.values_list('id', 'group', 'group__name', 'name', 'short_name', 'code', 'unit',
                                                'unit__display_name', 'description', 'quantity', 'mrp',
                                                'gst_percent', 'color', 'minimum_quantity'))
    product_column = ['product_id', 'product_group_id', 'product_group_name', 'product_name', 'product_short_name',
                      'product_code', 'unit_id', 'unit_name', 'description', 'product_quantity', 'product_mrp',
                      'gst_percent', 'color', 'minimum_quantity']
    product_df = pd.DataFrame(product_list, columns=product_column)
    if business_type_id == 1 or business_type_id == 2 or business_type_id == 3 or business_type_id == 9 or business_type_id == 11 or business_type_id == 16:
        discounted_product_obj = BusinessTypeWiseProductDiscount.objects.filter(business_type_id=business_type_id,
                                                                                product__is_active=True,
                                                                                product_id__in=discount_product_ids).order_by(
            'product__display_ordinal').exclude(product__group_id=3)
        discounted_product_list = list(
            discounted_product_obj.values_list('product_id', 'product__group', 'product__group__name',
                                               'product__name', 'product__short_name', 'product__code',
                                               'product__unit',
                                               'product__unit__display_name', 'product__description',
                                               'product__quantity', 'discounted_price', 'product__gst_percent',
                                               'product__color', 'product__minimum_quantity'))
        discounted_product_column = ['product_id', 'product_group_id', 'product_group_name', 'product_name',
                                     'product_short_name',
                                     'product_code', 'unit_id', 'unit_name', 'description', 'product_quantity',
                                     'product_mrp',
                                     'gst_percent', 'color', 'minimum_quantity']
        discounted_product_df = pd.DataFrame(discounted_product_list, columns=discounted_product_column)
    else:
        discounted_product_obj = BusinessWiseProductDiscount.objects.filter(business_id=business_id,
                                                                            product__is_active=True,
                                                                            product_id__in=discount_product_ids).order_by(
            'product__display_ordinal').exclude(product__group_id=3)
        discounted_product_list = list(
            discounted_product_obj.values_list('product_id', 'product__group', 'product__group__name',
                                               'product__name', 'product__short_name', 'product__code',
                                               'product__unit',
                                               'product__unit__display_name', 'product__description',
                                               'product__quantity', 'discounted_price', 'product__gst_percent',
                                               'product__color', 'product__minimum_quantity'))
        discounted_product_column = ['product_id', 'product_group_id', 'product_group_name', 'product_name',
                                     'product_short_name',
                                     'product_code', 'unit_id', 'unit_name', 'description', 'product_quantity',
                                     'product_mrp',
                                     'gst_percent', 'color', 'minimum_quantity']
        discounted_product_df = pd.DataFrame(discounted_product_list, columns=discounted_product_column)
    final_product_df = product_df.merge(discounted_product_df, how="outer")
#     loose_milk_obj = ProductQuantityVariationPrice.objects.filter(min_quantity=1, product__is_active=True).order_by(
#         'product__display_ordinal')
#     loose_milk_list = list(
#         loose_milk_obj.values_list('product_id', 'product__group', 'product__group__name', 'product__name',
#                                    'product__short_name', 'product__code', 'product__unit',
#                                    'product__unit__display_name', 'product__description', 'product__quantity', 'mrp',
#                                    'product__gst_percent', 'product__color', 'product__minimum_quantity'))
#     loose_milk_column = ['product_id', 'product_group_id', 'product_group_name', 'product_name', 'product_short_name',
#                          'product_code', 'unit_id', 'unit_name', 'description', 'product_quantity', 'product_mrp',
#                          'gst_percent', 'color', 'minimum_quantity']
#     loose_milk_df = pd.DataFrame(loose_milk_list, columns=loose_milk_column)
#     final_product_df = final_product_df.merge(loose_milk_df, how="outer")
    return final_product_df

In [5]:
business_wise_product_discount_dict = {}
for business in Business.objects.filter(business_type_id=16):
    if not business.code in business_wise_product_discount_dict:
        business_wise_product_discount_dict[business.id] = {}
    data = serve_product_and_session_list(business.user_profile.id)
    business_wise_product_discount_dict[business.id] = data.set_index('product_id')['product_mrp'].to_dict()
business_wise_product_discount_dict 

{3214: {1: Decimal('20.00'),
  2: Decimal('22.00'),
  3: Decimal('11.50'),
  4: Decimal('24.00'),
  6: Decimal('48.00'),
  7: Decimal('8.00'),
  25: Decimal('27.26'),
  8: Decimal('7.27'),
  26: Decimal('10.00')}}

In [7]:
# sale and salegroup data upload
from decimal import Decimal
start_date = '2022-03-07'
end_date = '2022-03-31'
sale_group_obj = SaleGroup.objects.filter(business__business_type__in=[4, 5, 6, 10, 12], date__gte=start_date, date__lte=end_date)
sale_obj = Sale.objects.filter(sale_group_id__in=sale_group_obj)
for sale_group in sale_group_obj:
    print(sale_group.date, sale_group.business.code)
    sale_group_total_cost = 0
    sale_group_sale = sale_obj.filter(sale_group_id=sale_group.id)
    for sale in sale_group_sale:
        if sale.cost == 0 and sale.count == 0:
            print('hi')
            product_cost = 0
        else:
            product_cost = round(Decimal(float(sale.cost)/sale.count),2)
            sale_wise_product_cost = round(business_wise_product_discount_dict[sale_group.business_id][sale.product_id], 2)
            if product_cost != sale_wise_product_cost:
                print(sale.id, product_cost, sale_wise_product_cost)
                temp_sale_cost = round(sale_wise_product_cost * Decimal(sale.count),2)
                sale.cost = temp_sale_cost
                sale.save()

                sale_group_total_cost += temp_sale_cost
            else:
                sale_group_total_cost += sale.cost
    sale_group.total_cost = sale_group_total_cost
    sale_group.product_amount = sale_group_total_cost
    sale_group.save()

2022-03-07 4302
3065244 8.00 10.00
3065245 24.50 30.00
2022-03-07 3076
2022-03-07 3092
2022-03-07 5018
2022-03-07 3057
2022-03-07 3057
2022-03-07 3034
2022-03-07 3076
2022-03-07 3026
2022-03-07 3026
2022-03-07 4172
2022-03-07 4172
2022-03-07 3043
2022-03-07 4200
2022-03-07 4202
2022-03-07 4213
2022-03-07 4199
2022-03-07 4245
2022-03-07 4237
2022-03-07 4048
2022-03-07 4239
2022-03-07 4303
2022-03-07 3039
2022-03-07 3039
2022-03-07 3008
2022-03-07 3008
2022-03-07 3051
2022-03-07 3051
2022-03-07 4173
2022-03-07 4173
2022-03-07 4249
2022-03-07 3089
2022-03-07 3089
2022-03-07 2385
2022-03-07 2385
2022-03-07 5003
2022-03-07 5003
2022-03-07 4203
2022-03-07 4204
2022-03-07 4205
2022-03-07 3020
2022-03-07 4248
2022-03-07 4261
2022-03-07 4077
2022-03-07 4195
2022-03-07 3147
2022-03-07 3147
2022-03-07 4256
2022-03-07 4054
2022-03-07 4054
2022-03-07 4144
2022-03-07 4144
2022-03-07 4251
2022-03-07 4019
2022-03-07 4048
2022-03-07 3004
2022-03-07 3004
2022-03-07 3148
2022-03-07 3148
2022-03-07 4250
2

2022-03-10 4200
2022-03-10 4202
2022-03-10 4213
2022-03-10 4199
2022-03-10 4215
2022-03-10 4078
2022-03-10 4025
2022-03-10 5007
2022-03-10 5007
2022-03-10 54298
2022-03-10 54298
2022-03-10 4024
2022-03-10 4024
2022-03-10 3037
2022-03-10 4251
2022-03-10 4245
2022-03-10 3057
2022-03-10 3057
2022-03-10 4187
2022-03-10 4187
2022-03-10 4173
2022-03-10 4173
2022-03-10 4303
2022-03-10 4237
2022-03-10 3039
2022-03-10 3039
2022-03-10 3051
2022-03-10 3051
2022-03-10 4015
2022-03-10 3151
2022-03-10 5002
2022-03-10 3029
2022-03-10 3029
2022-03-10 4240
2022-03-10 4240
2022-03-10 4016
2022-03-10 4016
2022-03-10 3037
2022-03-10 4249
2022-03-10 3004
2022-03-10 3004
2022-03-10 4048
2022-03-10 4048
2022-03-10 4246
2022-03-10 4246
2022-03-10 4186
2022-03-10 4186
2022-03-10 3044
2022-03-10 3069
2022-03-10 5003
2022-03-10 5003
2022-03-10 4203
2022-03-10 4204
2022-03-10 4205
2022-03-10 4226
3074170 29.00 30.00
2022-03-10 4248
2022-03-10 4261
2022-03-10 4023
2022-03-10 4195
2022-03-10 3147
2022-03-10 4077
20

2022-03-13 4203
2022-03-13 4204
2022-03-13 4205
2022-03-13 4226
2022-03-13 4248
2022-03-13 4261
2022-03-13 4077
2022-03-13 4195
2022-03-13 3042
2022-03-13 4016
2022-03-13 4016
2022-03-13 3040
2022-03-13 3040
2022-03-13 3057
2022-03-13 4172
2022-03-13 4172
2022-03-13 4067
2022-03-13 4162
2022-03-13 4186
2022-03-13 4186
2022-03-13 4246
2022-03-13 4246
2022-03-13 4021
2022-03-13 4021
2022-03-13 5030
2022-03-13 3151
2022-03-13 5530
2022-03-13 4250
2022-03-13 4190
2022-03-13 4223
2022-03-13 5019
2022-03-13 2385
2022-03-13 2385
2022-03-13 5004
2022-03-13 4054
2022-03-13 4054
2022-03-13 5008
2022-03-13 3005
2022-03-13 3147
2022-03-13 3147
2022-03-13 6010
2022-03-13 6020
2022-03-13 3145
2022-03-13 5005
2022-03-13 5005
2022-03-13 4157
2022-03-13 4301
2022-03-13 3027
2022-03-13 3027
2022-03-13 4175
2022-03-13 4176
2022-03-13 5018
2022-03-13 4260
2022-03-13 5031
2022-03-13 4196
2022-03-13 4197
2022-03-13 4198
2022-03-13 3132
2022-03-13 4215
2022-03-13 4128
2022-03-13 4128
2022-03-13 2425
2022-03-

2022-03-16 4024
2022-03-16 4024
2022-03-16 5019
2022-03-16 4246
2022-03-16 4246
2022-03-16 4186
2022-03-16 4186
2022-03-16 4021
2022-03-16 4021
2022-03-16 5010
2022-03-16 5530
2022-03-16 4223
2022-03-16 6010
2022-03-16 4128
2022-03-16 4128
2022-03-16 6020
2022-03-16 4224
2022-03-16 4224
2022-03-16 3029
2022-03-16 3029
2022-03-16 3027
2022-03-16 3027
2022-03-16 5008
2022-03-16 4190
2022-03-16 5004
2022-03-16 5018
2022-03-16 5007
2022-03-16 5007
2022-03-16 4175
2022-03-16 4176
2022-03-16 2425
2022-03-16 2425
2022-03-16 4196
2022-03-16 4197
2022-03-16 4198
2022-03-16 3132
2022-03-16 4215
2022-03-16 4253
2022-03-16 4015
2022-03-16 3147
2022-03-16 3147
2022-03-16 4256
2022-03-16 5005
2022-03-16 5005
2022-03-16 99109
2022-03-16 4157
2022-03-16 4157
2022-03-16 3020
2022-03-16 6009
3096209 24.78 27.26
2022-03-16 6008
3096230 24.78 27.26
2022-03-16 6007
3096315 24.78 27.26
2022-03-16 6006
3096338 24.78 27.26
2022-03-16 6005
3096377 24.78 27.26
2022-03-16 4048
2022-03-16 4048
2022-03-16 3040
202

2022-03-19 4260
2022-03-19 4128
2022-03-19 4128
2022-03-19 4224
2022-03-19 4224
2022-03-19 4190
2022-03-19 5004
2022-03-19 5030
2022-03-19 4250
2022-03-19 4019
2022-03-19 4157
2022-03-19 4157
2022-03-19 3027
2022-03-19 3027
2022-03-19 4176
2022-03-19 4175
2022-03-19 5005
2022-03-19 5005
2022-03-19 4196
2022-03-19 4197
2022-03-19 4198
2022-03-19 3132
2022-03-19 4215
2022-03-19 4253
2022-03-19 5010
2022-03-19 4272
2022-03-19 2425
2022-03-19 2425
2022-03-19 4054
2022-03-19 4054
2022-03-19 3050
2022-03-19 3050
2022-03-19 3020
2022-03-19 6005
2022-03-19 6006
2022-03-19 6007
2022-03-19 6008
2022-03-19 6009
2022-03-19 4014
2022-03-19 5018
2022-03-19 4251
2022-03-19 21152
2022-03-20 3008
2022-03-20 3092
2022-03-20 4172
2022-03-20 4172
2022-03-20 4245
2022-03-20 4212
2022-03-20 4149
2022-03-20 4201
2022-03-20 4026
2022-03-20 4031
2022-03-20 4031
2022-03-20 4032
2022-03-20 4258
2022-03-20 54298
2022-03-20 54298
2022-03-20 5007
2022-03-20 4200
2022-03-20 4202
2022-03-20 4213
2022-03-20 4199
2022-

2022-03-23 4212
2022-03-23 4149
2022-03-23 4201
2022-03-23 4025
2022-03-23 4024
2022-03-23 5007
2022-03-23 4024
2022-03-23 4031
2022-03-23 4031
2022-03-23 4032
2022-03-23 54298
2022-03-23 54298
2022-03-23 3076
2022-03-23 3076
2022-03-23 4237
2022-03-23 4251
2022-03-23 4240
2022-03-23 4240
2022-03-23 4301
2022-03-23 2385
2022-03-23 5003
2022-03-23 5003
2022-03-23 4203
2022-03-23 4204
2022-03-23 4205
2022-03-23 4226
2022-03-23 5002
2022-03-23 4248
2022-03-23 4077
2022-03-23 4023
2022-03-23 4023
2022-03-23 4195
2022-03-23 3043
2022-03-23 3089
2022-03-23 3089
2022-03-23 3004
2022-03-23 3004
2022-03-23 4144
2022-03-23 4144
2022-03-23 4173
2022-03-23 4173
2022-03-23 4249
2022-03-23 3148
2022-03-23 3148
2022-03-23 4179
2022-03-23 4179
2022-03-23 3002
2022-03-23 3002
2022-03-23 4303
2022-03-23 3044
2022-03-23 3069
2022-03-23 4245
2022-03-23 4172
2022-03-23 4172
2022-03-23 4048
2022-03-23 4162
2022-03-23 4261
2022-03-23 4216
2022-03-23 4258
2022-03-23 4261
2022-03-23 4048
2022-03-23 4021
2022-0

2022-03-26 4302
2022-03-26 4023
2022-03-26 4023
2022-03-26 4195
2022-03-26 4251
2022-03-26 4261
2022-03-26 4240
2022-03-26 4240
2022-03-26 4249
2022-03-26 4016
2022-03-26 4016
2022-03-26 3004
2022-03-26 3004
2022-03-26 4173
2022-03-26 4173
2022-03-26 3044
2022-03-26 3069
2022-03-26 3089
2022-03-26 3089
2022-03-26 4040
2022-03-26 3037
2022-03-26 3037
2022-03-26 54295
2022-03-26 4179
2022-03-26 4179
2022-03-26 4020
2022-03-26 4020
2022-03-26 3002
2022-03-26 3002
2022-03-26 3042
2022-03-26 3029
2022-03-26 3029
2022-03-26 4067
2022-03-26 4067
2022-03-26 4054
2022-03-26 4054
2022-03-26 4172
2022-03-26 4172
2022-03-26 4077
2022-03-26 4162
2022-03-26 3005
2022-03-26 5010
2022-03-26 5530
2022-03-26 5030
2022-03-26 4128
2022-03-26 4128
2022-03-26 4224
2022-03-26 4224
2022-03-26 4223
2022-03-26 5031
2022-03-26 5004
2022-03-26 6020
2022-03-26 6010
2022-03-26 5019
2022-03-26 5008
2022-03-26 4260
2022-03-26 3040
2022-03-26 3040
2022-03-26 5002
2022-03-26 4190
2022-03-26 3027
2022-03-26 4302
2022-03

2022-03-29 6010
2022-03-29 4176
2022-03-29 4175
2022-03-29 4196
2022-03-29 4197
2022-03-29 4198
2022-03-29 3132
2022-03-29 4215
2022-03-29 4253
2022-03-29 5005
2022-03-29 5005
2022-03-29 4157
2022-03-29 4157
2022-03-29 5018
2022-03-29 4015
2022-03-29 2425
2022-03-29 6011
2022-03-29 6009
2022-03-29 6008
2022-03-29 6007
2022-03-29 6006
2022-03-29 6005
2022-03-29 4048
2022-03-29 3076
2022-03-29 3076
2022-03-29 4014
2022-03-29 5002
2022-03-29 2425
2022-03-29 3023
2022-03-29 21152
2022-03-30 3034
2022-03-30 3092
2022-03-30 4187
2022-03-30 3039
2022-03-30 3039
2022-03-30 3051
2022-03-30 3051
2022-03-30 3145
2022-03-30 4187
2022-03-30 3026
2022-03-30 3026
2022-03-30 99109
2022-03-30 3148
2022-03-30 3148
2022-03-30 4048
2022-03-30 4048
2022-03-30 3057
2022-03-30 3008
2022-03-30 3008
2022-03-30 2007
2022-03-30 4200
2022-03-30 4202
2022-03-30 4213
2022-03-30 4199
2022-03-30 4078
2022-03-30 3057
2022-03-30 5007
2022-03-30 5007
2022-03-30 4212
2022-03-30 4149
2022-03-30 4201
2022-03-30 4025
2022-0

In [4]:
# dailysessionlybusinessly socity data upload
start_date = '2022-03-07'
end_date = '2022-03-31'
daily_businessly_obj = list(DailySessionllyBusinessllySale.objects.filter(business__business_type_id=10, delivery_date__gte=start_date, delivery_date__lte=end_date, sold_to='Agent').values())
daily_businessly_obj[0].keys()

dict_keys(['id', 'delivery_date', 'session_id', 'business_id', 'route_id', 'zone_id', 'union', 'business_type_id', 'sold_to', 'tm500_pkt', 'tm500_litre', 'tm500_cost', 'tm500_unit_price', 'std250_pkt', 'std250_litre', 'std250_cost', 'std250_unit_price', 'std500_pkt', 'std500_litre', 'std500_cost', 'std500_unit_price', 'fcm500_pkt', 'fcm500_litre', 'fcm500_cost', 'fcm500_unit_price', 'fcm1000_pkt', 'fcm1000_litre', 'fcm1000_cost', 'fcm1000_unit_price', 'tmcan', 'tmcan_litre', 'tmcan_cost', 'tmcan_unit_price', 'smcan', 'smcan_litre', 'smcan_cost', 'smcan_unit_price', 'fcmcan', 'fcmcan_litre', 'fcmcan_cost', 'fcmcan_unit_price', 'curd500_pkt', 'curd500_kgs', 'curd500_cost', 'curd500_unit_price', 'curd150_pkt', 'curd150_kgs', 'curd150_cost', 'curd150_unit_price', 'cupcurd_box', 'cupcurd_count', 'cupcurd_kgs', 'cupcurd_cost', 'cupcurd_unit_price', 'curd_bucket', 'curd_bucket_kgs', 'curd_bucket_cost', 'curd_bucket_unit_price', 'lassi200_pkt', 'lassi200_kgs', 'lassi200_cost', 'lassi200_unit_p

In [8]:
# dailysessionlybusinessly socity data upload
start_date = '2022-03-07'
end_date = '2022-03-31'
daily_businessly_obj = DailySessionllyBusinessllySale.objects.filter(business__business_type_id__in=[4, 5, 6, 10, 12], delivery_date__gte=start_date, delivery_date__lte=end_date, sold_to='Agent')
for i in daily_businessly_obj:
    print(i.delivery_date)
    milk_cost = 0
    curd_cost = 0
    frg_cost = 0
    total_cost = 0
    
    fcm_cost = 0
    sm_cost = 0
    tm_cost = 0
    
    ## please make to remove this line for diffrent purpose
    i.business_type_id = i.business.business_type.id
    
    
    
    print(i.tm500_cost, i.std250_cost, i.std500_cost, i.fcm500_cost, i.fcm1000_cost, i.curd500_cost, i.curd150_cost, i.cupcurd_cost,   i.milk_cost,  i.curd_cost, i.total_cost)     
    #milk
    #tm 
    if i.tm500_pkt != 0:
        cost = i.tm500_pkt * business_wise_product_discount_dict[i.business_id][1]
        i.tm500_cost = cost
        milk_cost += cost
        tm_cost += cost
        
    #std
    if i.std250_pkt != 0:
        cost = i.std250_pkt * business_wise_product_discount_dict[i.business_id][3]
        i.std250_cost = cost
        milk_cost += cost
        sm_cost += cost
    
    if i.std500_pkt != 0:
        cost = i.std500_pkt * business_wise_product_discount_dict[i.business_id][2]
        i.std500_cost = cost
        milk_cost += cost
        sm_cost += cost
    
    #fcm
    if i.fcm500_pkt != 0:
        cost = i.fcm500_pkt * business_wise_product_discount_dict[i.business_id][4]
        i.fcm500_cost = cost
        milk_cost += cost
        fcm_cost += cost
    
    if i.fcm1000_pkt != 0:
        cost = i.fcm1000_pkt * business_wise_product_discount_dict[i.business_id][6]
        i.fcm1000_cost = cost
        milk_cost += cost
        fcm_cost += cost
        
    #curd 
    if i.curd500_pkt != 0:
        cost = i.curd500_pkt * business_wise_product_discount_dict[i.business_id][25]
        i.curd500_cost = cost
        curd_cost += cost
    
    if i.curd150_pkt != 0:
        cost = i.curd150_pkt * business_wise_product_discount_dict[i.business_id][7]
        i.curd150_cost = cost
        curd_cost += cost
    
    if i.cupcurd_count != 0:
        if business_wise_product_discount_dict[i.business_id][10] != 0:
            cost = i.cupcurd_count * business_wise_product_discount_dict[i.business_id][10]
            i.cupcurd_cost = cost
            curd_cost += cost
            
    #butter_milk
    if i.buttermilk200_pkt != 0:
        if business_wise_product_discount_dict[i.business_id][8] != 0:
            cost = i.buttermilk200_pkt * business_wise_product_discount_dict[i.business_id][8]
            i.buttermilk200_cost = cost
            frg_cost += cost
            
    i.fcm_cost = fcm_cost
    i.sm_cost = sm_cost
    i.tm_cost = tm_cost
    
    i.milk_cost = milk_cost
    i.curd_cost = curd_cost 
    i.fermented_products_cost = frg_cost + i.lassi_cost + curd_cost
    i.total_cost = i.milk_cost + i.fermented_products_cost
    
    i.save()
    print(i.tm500_cost, i.std250_cost, i.std500_cost, i.fcm500_cost, i.fcm1000_cost, i.curd500_cost, i.curd150_cost, i.cupcurd_cost,   i.milk_cost,  i.curd_cost, i.total_cost, i.id)      
    print()
    

2022-03-07
0.00 0.00 6330.00 0.00 0.00 0.00 0.00 0.00 6330.00 0.00 6330.00
0.00 0.00 6330.00 0.00 0.00 0.00 0.00 0.00 6330.00 0 6330.00 2427545

2022-03-07
0.00 0.00 3165.00 0.00 0.00 0.00 0.00 0.00 3165.00 0.00 3165.00
0.00 0.00 3165.00 0.00 0.00 0.00 0.00 0.00 3165.00 0 3165.00 2427548

2022-03-07
0.00 0.00 88.00 0.00 0.00 0.00 0.00 0.00 88.00 0.00 88.00
0.00 0.00 88.00 0.00 0.00 0.00 0.00 0.00 88.00 0 88.00 2427552

2022-03-07
0.00 0.00 352.00 0.00 0.00 0.00 0.00 0.00 352.00 0.00 352.00
0.00 0.00 352.00 0.00 0.00 0.00 0.00 0.00 352.00 0 352.00 2427616

2022-03-07
0.00 0.00 150.00 0.00 0.00 0.00 0.00 0.00 150.00 0.00 150.00
0.00 0.00 150.00 0.00 0.00 0.00 0.00 0.00 150.00 0 150.00 2427629

2022-03-07
0.00 0.00 1760.00 0.00 0.00 0.00 0.00 0.00 1760.00 0.00 1760.00
0.00 0.00 1760.00 0.00 0.00 0.00 0.00 0.00 1760.00 0 1760.00 2427630

2022-03-07
0.00 0.00 88.00 0.00 0.00 0.00 0.00 0.00 88.00 0.00 88.00
0.00 0.00 88.00 0.00 0.00 0.00 0.00 0.00 88.00 0 88.00 2427656

2022-03-07
0.00 0.00 

0.00 0.00 1688.00 0.00 0.00 0.00 0.00 0.00 1688.00 0 1688.00 2428390

2022-03-07
2440.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 2440.00 0.00 2440.00
2440.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 2440.00 0 2440.00 2428400

2022-03-07
0.00 0.00 0.00 211000.00 0.00 0.00 0.00 0.00 211000.00 0.00 211000.00
0.00 0.00 0.00 211000.00 0.00 0.00 0.00 0.00 211000.00 0 211000.00 2428410

2022-03-07
0.00 0.00 0.00 253200.00 0.00 0.00 0.00 0.00 253200.00 0.00 253200.00
0.00 0.00 0.00 253200.00 0.00 0.00 0.00 0.00 253200.00 0 253200.00 2428413

2022-03-07
0.00 0.00 19794.40 0.00 0.00 0.00 0.00 0.00 19794.40 0.00 19794.40
0.00 0.00 19794.40 0.00 0.00 0.00 0.00 0.00 19794.40 0 19794.40 2428431

2022-03-07
0.00 0.00 1688.00 0.00 0.00 0.00 0.00 0.00 1688.00 0.00 1688.00
0.00 0.00 1688.00 0.00 0.00 0.00 0.00 0.00 1688.00 0 1688.00 2428436

2022-03-07
0.00 0.00 1210.00 0.00 0.00 0.00 0.00 0.00 1210.00 0.00 1210.00
0.00 0.00 1210.00 0.00 0.00 0.00 0.00 0.00 1210.00 0 1210.00 2428446

2022-03-07
0.00 0.00 1100.00 0

95.90 55.15 506.40 690.60 0.00 300.00 50.00 0.00 1348.05 350.00 1698.05 2428631

2022-03-07
0.00 0.00 352.00 0.00 0.00 58.00 0.00 0.00 352.00 58.00 410.00
0.00 0.00 352.00 0.00 0.00 60.00 0.00 0.00 352.00 60.00 412.00 2428659

2022-03-07
0.00 0.00 132.00 0.00 0.00 0.00 0.00 0.00 132.00 0.00 132.00
0.00 0.00 132.00 0.00 0.00 0.00 0.00 0.00 132.00 0 132.00 2428660

2022-03-07
0.00 0.00 132.00 0.00 0.00 0.00 0.00 0.00 132.00 0.00 132.00
0.00 0.00 132.00 0.00 0.00 0.00 0.00 0.00 132.00 0 132.00 2428661

2022-03-07
0.00 0.00 6600.00 0.00 0.00 0.00 0.00 0.00 6600.00 0.00 6600.00
0.00 0.00 6600.00 0.00 0.00 0.00 0.00 0.00 6600.00 0 6600.00 2428662

2022-03-07
0.00 0.00 880.00 0.00 0.00 0.00 0.00 0.00 880.00 0.00 880.00
0.00 0.00 880.00 0.00 0.00 0.00 0.00 0.00 880.00 0 880.00 2428663

2022-03-07
0.00 0.00 132.00 0.00 0.00 0.00 0.00 0.00 132.00 0.00 132.00
0.00 0.00 132.00 0.00 0.00 0.00 0.00 0.00 132.00 0 132.00 2428664

2022-03-07
0.00 0.00 928.40 0.00 0.00 0.00 0.00 0.00 928.40 0.00 928.40


0.00 0.00 3520.00 0.00 0.00 0.00 0.00 0.00 3520.00 0 3520.00 2431044

2022-03-08
0.00 0.00 1760.00 0.00 0.00 0.00 0.00 0.00 1760.00 0.00 1760.00
0.00 0.00 1760.00 0.00 0.00 0.00 0.00 0.00 1760.00 0 1760.00 2431045

2022-03-08
0.00 0.00 880.00 0.00 0.00 0.00 0.00 0.00 880.00 0.00 880.00
0.00 0.00 880.00 0.00 0.00 0.00 0.00 0.00 880.00 0 880.00 2431046

2022-03-08
0.00 0.00 1688.00 0.00 0.00 0.00 0.00 0.00 1688.00 0.00 1688.00
0.00 0.00 1688.00 0.00 0.00 0.00 0.00 0.00 1688.00 0 1688.00 2431047

2022-03-08
0.00 0.00 211.00 0.00 0.00 0.00 0.00 0.00 211.00 0.00 211.00
0.00 0.00 211.00 0.00 0.00 0.00 0.00 0.00 211.00 0 211.00 2431048

2022-03-08
0.00 0.00 792.00 0.00 0.00 0.00 0.00 0.00 792.00 0.00 792.00
0.00 0.00 792.00 0.00 0.00 0.00 0.00 0.00 792.00 0 792.00 2431049

2022-03-08
0.00 0.00 880.00 0.00 0.00 0.00 0.00 0.00 880.00 0.00 880.00
0.00 0.00 880.00 0.00 0.00 0.00 0.00 0.00 880.00 0 880.00 2431050

2022-03-08
0.00 0.00 5908.00 0.00 0.00 0.00 0.00 0.00 5908.00 0.00 5908.00
0.00 0.00

0.00 0.00 440.00 0.00 0.00 0.00 0.00 0.00 440.00 0 440.00 2431770

2022-03-08
0.00 0.00 660.00 0.00 0.00 0.00 0.00 0.00 660.00 0.00 660.00
0.00 0.00 660.00 0.00 0.00 0.00 0.00 0.00 660.00 0 660.00 2431782

2022-03-08
0.00 0.00 968.00 0.00 0.00 0.00 0.00 0.00 968.00 0.00 968.00
0.00 0.00 968.00 0.00 0.00 0.00 0.00 0.00 968.00 0 968.00 2431783

2022-03-08
0.00 0.00 6330.00 0.00 0.00 0.00 0.00 0.00 6330.00 0.00 6330.00
0.00 0.00 6330.00 0.00 0.00 0.00 0.00 0.00 6330.00 0 6330.00 2431786

2022-03-08
1280.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1280.00 0.00 1280.00
1280.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1280.00 0 1280.00 2431787

2022-03-08
0.00 0.00 633.00 0.00 0.00 0.00 0.00 0.00 633.00 0.00 633.00
0.00 0.00 633.00 0.00 0.00 0.00 0.00 0.00 633.00 0 633.00 2431788

2022-03-08
0.00 0.00 8440.00 0.00 0.00 0.00 0.00 0.00 8440.00 0.00 8440.00
0.00 0.00 8440.00 0.00 0.00 0.00 0.00 0.00 8440.00 0 8440.00 2431789

2022-03-08
0.00 0.00 6752.00 0.00 0.00 0.00 0.00 0.00 6752.00 0.00 6752.00
0.00 0

...

0.00 0.00 22846.40 0.00 0.00 0.00 0.00 0.00 22846.40 0 22846.40 2509487

2022-03-30
0.00 0.00 1298.00 0.00 0.00 0.00 0.00 0.00 1298.00 0.00 1298.00
0.00 0.00 1298.00 0.00 0.00 0.00 0.00 0.00 1298.00 0 1298.00 2509512

2022-03-30
0.00 0.00 440.00 0.00 0.00 0.00 0.00 0.00 440.00 0.00 440.00
0.00 0.00 440.00 0.00 0.00 0.00 0.00 0.00 440.00 0 440.00 2509533

2022-03-30
0.00 0.00 1320.00 0.00 0.00 0.00 0.00 0.00 1320.00 0.00 1320.00
0.00 0.00 1320.00 0.00 0.00 0.00 0.00 0.00 1320.00 0 1320.00 2509546

2022-03-30
0.00 0.00 0.00 198340.00 0.00 0.00 0.00 0.00 198340.00 0.00 198340.00
0.00 0.00 0.00 198340.00 0.00 0.00 0.00 0.00 198340.00 0 198340.00 2509551

2022-03-30
0.00 0.00 1477.00 0.00 0.00 600.00 0.00 0.00 1477.00 600.00 2077.00
0.00 0.00 1477.00 0.00 0.00 600.00 0.00 0.00 1477.00 600.00 2077.00 2509571

2022-03-30
0.00 0.00 0.00 253200.00 0.00 0.00 0.00 0.00 253200.00 0.00 253200.00
0.00 0.00 0.00 253200.00 0.00 0.00 0.00 0.00 253200.00 0 253200.00 2509638

2022-03-30
0.00 0.00 844.00 

0.00 0.00 1980.00 0.00 0.00 0.00 0.00 0.00 1980.00 0.00 1980.00
0.00 0.00 1980.00 0.00 0.00 0.00 0.00 0.00 1980.00 0 1980.00 2509808

2022-03-30
0.00 0.00 352.00 0.00 0.00 0.00 0.00 0.00 352.00 0.00 352.00
0.00 0.00 352.00 0.00 0.00 0.00 0.00 0.00 352.00 0 352.00 2509809

2022-03-30
0.00 0.00 132.00 0.00 0.00 0.00 0.00 0.00 132.00 0.00 132.00
0.00 0.00 132.00 0.00 0.00 0.00 0.00 0.00 132.00 0 132.00 2509810

2022-03-30
0.00 0.00 176.00 0.00 0.00 0.00 0.00 0.00 176.00 0.00 176.00
0.00 0.00 176.00 0.00 0.00 0.00 0.00 0.00 176.00 0 176.00 2509811

2022-03-30
0.00 0.00 220.00 0.00 0.00 0.00 0.00 0.00 220.00 0.00 220.00
0.00 0.00 220.00 0.00 0.00 0.00 0.00 0.00 220.00 0 220.00 2509813

2022-03-30
0.00 0.00 150.00 0.00 0.00 0.00 0.00 0.00 150.00 0.00 150.00
0.00 0.00 150.00 0.00 0.00 0.00 0.00 0.00 150.00 0 150.00 2509816

2022-03-30
0.00 0.00 264.00 0.00 0.00 0.00 0.00 0.00 264.00 0.00 264.00
0.00 0.00 264.00 0.00 0.00 0.00 0.00 0.00 264.00 0 264.00 2509817

2022-03-30
0.00 0.00 660.00 0.00

0.00 0.00 5064.00 0.00 0.00 0.00 0.00 0.00 5064.00 0 5064.00 2512204

2022-03-31
0.00 0.00 2532.00 0.00 0.00 0.00 0.00 0.00 2532.00 0.00 2532.00
0.00 0.00 2532.00 0.00 0.00 0.00 0.00 0.00 2532.00 0 2532.00 2512205

2022-03-31
0.00 0.00 9284.00 0.00 0.00 0.00 0.00 0.00 9284.00 0.00 9284.00
0.00 0.00 9284.00 0.00 0.00 0.00 0.00 0.00 9284.00 0 9284.00 2512206

2022-03-31
0.00 0.00 211.00 0.00 0.00 0.00 0.00 0.00 211.00 0.00 211.00
0.00 0.00 211.00 0.00 0.00 0.00 0.00 0.00 211.00 0 211.00 2512207

2022-03-31
0.00 0.00 1100.00 0.00 0.00 0.00 0.00 0.00 1100.00 0.00 1100.00
0.00 0.00 1100.00 0.00 0.00 0.00 0.00 0.00 1100.00 0 1100.00 2512208

2022-03-31
0.00 0.00 2532.00 0.00 0.00 0.00 0.00 0.00 2532.00 0.00 2532.00
0.00 0.00 2532.00 0.00 0.00 0.00 0.00 0.00 2532.00 0 2532.00 2512209

2022-03-31
0.00 0.00 440.00 0.00 0.00 0.00 0.00 0.00 440.00 0.00 440.00
0.00 0.00 440.00 0.00 0.00 0.00 0.00 0.00 440.00 0 440.00 2512210

2022-03-31
0.00 0.00 132.00 0.00 0.00 0.00 0.00 0.00 132.00 0.00 132.00


0.00 0.00 844.00 0.00 0.00 0.00 0.00 0.00 844.00 0 844.00 2512900

2022-03-31
0.00 0.00 1320.00 0.00 0.00 0.00 0.00 0.00 1320.00 0.00 1320.00
0.00 0.00 1320.00 0.00 0.00 0.00 0.00 0.00 1320.00 0 1320.00 2512918

2022-03-31
0.00 0.00 1266.00 0.00 0.00 0.00 0.00 0.00 1266.00 0.00 1266.00
0.00 0.00 1266.00 0.00 0.00 0.00 0.00 0.00 1266.00 0 1266.00 2512921

2022-03-31
0.00 0.00 1899.00 0.00 0.00 0.00 0.00 0.00 1899.00 0.00 1899.00
0.00 0.00 1899.00 0.00 0.00 0.00 0.00 0.00 1899.00 0 1899.00 2512924

2022-03-31
0.00 0.00 594.00 0.00 0.00 0.00 0.00 0.00 594.00 0.00 594.00
0.00 0.00 594.00 0.00 0.00 0.00 0.00 0.00 594.00 0 594.00 2512926

2022-03-31
0.00 0.00 1540.00 0.00 0.00 0.00 0.00 0.00 1540.00 0.00 1540.00
0.00 0.00 1540.00 0.00 0.00 0.00 0.00 0.00 1540.00 0 1540.00 2512928

2022-03-31
383.60 165.45 2110.00 1151.00 0.00 900.00 200.00 0.00 3810.05 1100.00 4910.05
383.60 165.45 2110.00 1151.00 0.00 900.00 200.00 0.00 3810.05 1100.00 4910.05 2512930

2022-03-31
0.00 0.00 440.00 0.00 0.00 

0.00 0.00 7174.00 0.00 0.00 0.00 0.00 0.00 7174.00 0 7174.00 2513024

2022-03-31
0.00 0.00 440.00 0.00 0.00 0.00 0.00 0.00 440.00 0.00 440.00
0.00 0.00 440.00 0.00 0.00 0.00 0.00 0.00 440.00 0 440.00 2513026

2022-03-31
0.00 0.00 274.30 0.00 0.00 0.00 0.00 0.00 274.30 0.00 274.30
0.00 0.00 274.30 0.00 0.00 0.00 0.00 0.00 274.30 0 274.30 2513027

2022-03-31
0.00 0.00 1477.00 0.00 0.00 0.00 0.00 0.00 1477.00 0.00 1477.00
0.00 0.00 1477.00 0.00 0.00 0.00 0.00 0.00 1477.00 0 1477.00 2513028

2022-03-31
0.00 0.00 506.16 0.00 0.00 0.00 0.00 0.00 506.16 0.00 506.16
0.00 0.00 506.16 0.00 0.00 0.00 0.00 0.00 506.16 0 506.16 2513019



In [83]:
Business.objects.get(code='8123').id

3214

In [ ]:
route_trace_sale_obj = RouteTraceWiseSaleSummary.objects.filter(route_trace__route__is_temp_route=True, route_trace__date__month=10).values_list('route_trace__route_id', 'product_id', )            

### Temp Tray Count Updated

In [23]:
route_obj = Route.objects.filter(is_temp_route=True, session_id=1)
name_dict = {}
for route in route_obj:
    short_name = route.name.split(' -')[0]
    if not short_name in name_dict:
        name_dict[short_name] = {
            'route_name': route.name,
            'route_id': route.id
        }
name_dict

{'M2': {'route_name': 'M2 - VALPA_MOR_temp', 'route_id': 223},
 'P1': {'route_name': 'P1 - THOND_MOR_temp', 'route_id': 219},
 'CURD SUPPL_MOR_temp': {'route_name': 'CURD SUPPL_MOR_temp', 'route_id': 237},
 'B': {'route_name': 'B - SAIBAB_MOR_temp', 'route_id': 207},
 'A': {'route_name': 'A - PODANU_MOR_temp', 'route_id': 215},
 'V': {'route_name': 'V - ROUTE_MOR_temp', 'route_id': 209},
 '3': {'route_name': '3 - PEELAM_MOR_temp', 'route_id': 206},
 'M3': {'route_name': 'M3 - POLLA_MOR_temp', 'route_id': 222},
 'S': {'route_name': 'S - GANDHI_MOR_temp', 'route_id': 226},
 'E': {'route_name': 'E - TOWN_MOR_temp', 'route_id': 214},
 'C': {'route_name': 'C - GANDHI_MOR_temp', 'route_id': 225},
 'Y': {'route_name': 'Y - Route_MOR_temp', 'route_id': 242},
 'L': {'route_name': 'L - GANAPA_MOR_temp', 'route_id': 229},
 'LEAKAGE AL_MOR_temp': {'route_name': 'LEAKAGE AL_MOR_temp', 'route_id': 236},
 'F': {'route_name': 'F - RSPURA_MOR_temp', 'route_id': 208},
 'K': {'route_name': 'K - METTUP_MO

In [37]:
import pandas as pd
import numpy as np
import requests
import io

try:
    from StringIO import StringIO
except ImportError:
    
    from io import StringIO
    from io import BytesIO
from decimal import Decimal
route_trace_sale_obj = RouteTraceWiseSaleSummary.objects.filter(route_trace__route__is_temp_route=True)
product_list = list(Product.objects.all().values('short_name', 'id'))
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1CR00EeXXKMuawOrksSPKGy9eZqcEAp89SYGSlYk513E/export?format=xlsx'
r = requests.get(google_sheet_url)
# sheet_list = ['May-2020', 'June-2020', 'july-2020', 'August - 2020', 'September - 2020']
# for sheet in sheet_list:
#     df = pd.read_excel(BytesIO(data),sheet_name=sheet)
#     month_df = df.fillna(0)
#     print(sheet)
#     for index,df_value in month_df.iterrows():
#         temp_route_id = name_dict[str(df_value['Temp Route']).upper().strip()]['route_id']
#         route_trace_id = RouteTrace.objects.get(date=df_value['Date'], route_id=temp_route_id).id
        
#         for product in product_list:
#             if product['short_name'] in df_value:
#                 if route_trace_sale_obj.filter(route_trace_id=route_trace_id, product_id=product['id']).exists():
#                     temp_route_trace_sale_obj = route_trace_sale_obj.get(route_trace_id=route_trace_id, product_id=product['id'])    
                    
#                     if temp_route_trace_sale_obj.tray_count == None or temp_route_trace_sale_obj.tray_count == 0:
#                         temp_route_trace_sale_obj.tray_count = df_value[product['short_name']]
                        
#                 else:
#                     temp_route_trace_sale_obj = RouteTraceWiseSaleSummary(quantity=0, product_id=product['id'], 
#                                                                           route_trace_id=route_trace_id, 
#                                                                           tray_count=df_value[product['short_name']])
#                 temp_route_trace_sale_obj.save()
                
#     print(f'done - {sheet.title}')
#     print(df)

b'PK\x03\x04\x14\x00\x08\x08\x08\x00\x96\x02\x82Q\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00xl/drawings/drawing1.xml\x9d\xd0]n\xc20\x0c\x07\xf0\x13\xec\x0eU\xdeiZ\x18\x13C\x14^\xd0N0\x0e\xe0%n\x1b\x91\x8f\xca\x0e\xa3\xdc~\xd1J6i{\x01\x1em\xcb?\xf9\xef\xcdnt\xb6\xf8Db\x13|#\xea\xb2\x12\x05z\x15\xb4\xf1]#\x0e\xefo\xb3\x95(8\x82\xd7`\x83\xc7F\\\x90\xc5n\xfb\xb4\x195\xad\xcf\xbc\xa7"\xed{^\xa7\xb2\x11}\x8c\xc3ZJV=:\xe02\x0c\xe8\xd3\xb4\r\xe4 \xa6\x92:\xa9\t\xceIvV\xce\xab\xeaE\xf2@\x08\x9a{\xc4\xb8\x9f&\xe2\xea\xc1\x03\x9a\x03\xe3\xf3\xfeM\xd7\x84\xb65\n\xf7A\x9d\x1c\xfa8!\x84\x16b\xfa\x05\xf7f\xe0\xac\xa9\x07\xaeQ=P\xfc\x01\xc6\x7f\x823\x8a\x02\x876\x96*\xb8\xeb)\xd9HB\xfd<\t8\xfe\x1a\xf5\xdd\xc8R\xbe\xca\xd5_\xc8\xdd\x14\xc7\x01\x1dO\xc3,\xb9Cz\xc8\x87\xb1&^\xbe\x93eFw\xee\x81\xb7h\x03\x1d\x81\xcb\xc8\xb88\xf8\xe3\xdd\xb1*\xc96\xb5(+l\xb1^\xde\xad\xcc\xb3"\xb7_PK\x07\x08\x07bi\x83\x05\x01\x00\x00\x07\x03\x00\x00PK\x03\x04\x14\x00\x08\x08\x08\x00\x96\x02\x82Q\x00\x00\

In [138]:
import openpyxl
route_trace_sale_obj = RouteTraceWiseSaleSummary.objects.filter(route_trace__route__is_temp_route=True)
product_list = list(Product.objects.all().values('short_name', 'id'))
excel_name = 'Tray Back Calculation.xlsx'
wb = openpyxl.load_workbook(excel_name)

for sheet in wb.worksheets:
    month_df = pd.read_excel(excel_name, sheet_name=sheet.title).fillna(0)
    
    for index,df_value in month_df.iterrows():
        temp_route_id = name_dict[str(df_value['Temp Route']).upper()]['route_id']
        route_trace_id = RouteTrace.objects.get(date=df_value['Date'], route_id=temp_route_id).id
        
        for product in product_list:
            if product['short_name'] in df_value:
                if route_trace_sale_obj.filter(route_trace_id=route_trace_id, product_id=product['id']).exists():
                    temp_route_trace_sale_obj = route_trace_sale_obj.get(route_trace_id=route_trace_id, product_id=product['id'])    
                    
                    if temp_route_trace_sale_obj.tray_count == None or temp_route_trace_sale_obj.tray_count == 0:
                        temp_route_trace_sale_obj.tray_count = df_value[product['short_name']]
                        
                else:
                    temp_route_trace_sale_obj = RouteTraceWiseSaleSummary(quantity=0, product_id=product['id'], 
                                                                          route_trace_id=route_trace_id, 
                                                                          tray_count=df_value[product['short_name']])
                temp_route_trace_sale_obj.save()
                
    print(f'done - {sheet.title}')
month_df

done - June-2020
done - july-2020
done - August - 2020
done - September - 2020
done - October 2020


,Date,Temp Route,TM 500,STD 500,STD 250,FCM 500,FCM 1000,Curd 150,Curd 500,BM 200
0,2020-10-01,A,0.0,67,6.0,4.0,0.0,1.0,0.0,0.0
1,2020-10-01,K1,0.0,4,10.0,47.0,0.0,3.0,3.0,0.0
2,2020-10-01,Q,0.0,26,9.0,23.0,1.0,2.0,1.0,0.0
3,2020-10-02,A,0.0,53,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-10-02,K1,0.0,10,11.0,30.0,0.0,2.0,0.0,0.0
5,2020-10-02,Q,0.0,25,9.0,23.0,1.0,4.0,1.0,0.0
6,2020-10-03,A,0.0,68,6.0,4.0,0.0,1.0,0.0,0.0
7,2020-10-03,k1,0.0,5,11.0,49.0,0.0,3.0,3.0,0.0
8,2020-10-03,Q,0.0,25,9.0,23.0,1.0,3.0,1.0,0.0
9,2020-10-04,A,0.0,52,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
product_list

[{'short_name': 'CBUK', 'id': 21},
 {'short_name': 'Lassi 200', 'id': 9},
 {'short_name': 'TNcan', 'id': 22},
 {'short_name': 'SMcan', 'id': 24},
 {'short_name': 'FMcan', 'id': 23},
 {'short_name': 'Curd 100', 'id': 10},
 {'short_name': 'BM 200', 'id': 8},
 {'short_name': 'TM 500', 'id': 1},
 {'short_name': 'STD 500', 'id': 2},
 {'short_name': 'STD 250', 'id': 3},
 {'short_name': 'FCM 500', 'id': 4},
 {'short_name': 'FCM 1000', 'id': 6},
 {'short_name': 'Curd 150', 'id': 7},
 {'short_name': 'Curd 500', 'id': 25}]

In [4]:
import pandas as pd
from_date = "2020-05-1"
to_date = "2020-05-1"
date_list = pd.date_range(start=from_date, end=to_date)
date_list

DatetimeIndex(['2020-05-01'], dtype='datetime64[ns]', freq='D')

In [14]:
mor_route_obj = Route.objects.filter(is_temp_route=False,session_id=1, id__in=[85,99,145,143,241,123,126,147,149,151,153,136]).order_by("name")
eve_route_obj = Route.objects.filter(is_temp_route=False,session_id=2, id__in=[86,100,144,146,127,120,148,150,154,152,134]).order_by("name")
data_dict={
  "final_dict" : {
      "tm500":{
          "qty":0,
          "rate":0,
          "liter":0,
          "cost":0,
      },
      "sm250":{
          "qty":0,
          "rate":0,
          "liter":0,
          "cost":0,
      },
      "sm500":{
          "qty":0,
          "rate":0,
          "liter":0,
          "cost":0,
      },
      "fcm500":{
          "qty":0,
          "rate":0,
          "liter":0,
          "cost":0,
      },
      "fcm1000":{
          "qty":0,
          "rate":0,
          "liter":0,
          "cost":0,
      },
      "total":{
          "qty":0,
          "rate":0,
          "liter":0,
          "cost":0,
      },
  }
}
from_date = '2020-11-10'
to_date = '2020-11-10'
date_list = pd.date_range(start=from_date, end=to_date)


for date in date_list:
  dates = str(date)[:10]
  for route in mor_route_obj:
      sale_obj = DailySessionllyBusinessllySale.objects.filter(sold_to="Leakage",route_id=route.id,delivery_date=dates)
      if sale_obj:
          format_date = datetime.datetime.strptime(dates,'%Y-%m-%d')
          date = str(datetime.datetime.strftime(format_date, '%d-%m-%Y'))

          if not route.name[:-4] in data_dict:
              data_dict[route.name[:-4]] = {
                  "grand_total":0,
              }
          if not date in data_dict[route.name[:-4]]:
            data_dict[route.name[:-4]][date] = {
                "tm500": 0,
                "sm250": 0,
                "sm500": 0,
                "fcm500": 0,
                "fcm1000": 0,
                "total" : 0,
            }
          tm500 = sale_obj.aggregate(Sum('tm500_litre'))['tm500_litre__sum']
          sm250 = sale_obj.aggregate(Sum('std250_litre'))['std250_litre__sum']
          sm500 = sale_obj.aggregate(Sum('std500_litre'))['std500_litre__sum']
          fcm500 = sale_obj.aggregate(Sum('fcm500_litre'))['fcm500_litre__sum']
          fcm1000 = sale_obj.aggregate(Sum('fcm1000_litre'))['fcm1000_litre__sum']

          data_dict[route.name[:-4]]["grand_total"] += tm500 + sm250 + sm500 + fcm500 + fcm1000

          data_dict[route.name[:-4]][date]["tm500"] += sale_obj.aggregate(Sum('tm500_litre'))['tm500_litre__sum']
          data_dict[route.name[:-4]][date]["sm250"] += sale_obj.aggregate(Sum('std250_litre'))['std250_litre__sum']
          data_dict[route.name[:-4]][date]["sm500"] += sale_obj.aggregate(Sum('std500_litre'))['std500_litre__sum']
          data_dict[route.name[:-4]][date]["fcm500"] += sale_obj.aggregate(Sum('fcm500_litre'))['fcm500_litre__sum']
          data_dict[route.name[:-4]][date]["fcm1000"] += sale_obj.aggregate(Sum('fcm1000_litre'))['fcm1000_litre__sum']
          data_dict[route.name[:-4]][date]["total"] += tm500+sm250+sm500+fcm500+fcm1000

  for route in eve_route_obj:
      sale_obj = DailySessionllyBusinessllySale.objects.filter(sold_to="Leakage",route_id=route.id,delivery_date=dates)
      if sale_obj:
          format_date = datetime.datetime.strptime(dates,'%Y-%m-%d')
          date = str(datetime.datetime.strftime(format_date, '%d-%m-%Y'))
          if not route.name[:-4] in data_dict:
              data_dict[route.name[:-4]] = {
                  "grand_total":0,
              }
          if not date in data_dict[route.name[:-4]]:
            data_dict[route.name[:-4]][date] = {
                "tm500": 0,
                "sm250": 0,
                "sm500": 0,
                "fcm500": 0,
                "fcm1000": 0,
                "total" : 0,
            }

          tm500 = sale_obj.aggregate(Sum('tm500_litre'))['tm500_litre__sum']
          sm250 = sale_obj.aggregate(Sum('std250_litre'))['std250_litre__sum']
          sm500 = sale_obj.aggregate(Sum('std500_litre'))['std500_litre__sum']
          fcm500 = sale_obj.aggregate(Sum('fcm500_litre'))['fcm500_litre__sum']
          fcm1000 = sale_obj.aggregate(Sum('fcm1000_litre'))['fcm1000_litre__sum']

          data_dict[route.name[:-4]]["grand_total"] += tm500 + sm250 + sm500 + fcm500 + fcm1000

          data_dict[route.name[:-4]][date]["tm500"] += sale_obj.aggregate(Sum('tm500_litre'))['tm500_litre__sum']
          data_dict[route.name[:-4]][date]["sm250"] += sale_obj.aggregate(Sum('std250_litre'))['std250_litre__sum']
          data_dict[route.name[:-4]][date]["sm500"] += sale_obj.aggregate(Sum('std500_litre'))['std500_litre__sum']
          data_dict[route.name[:-4]][date]["fcm500"] += sale_obj.aggregate(Sum('fcm500_litre'))['fcm500_litre__sum']
          data_dict[route.name[:-4]][date]["fcm1000"] += sale_obj.aggregate(Sum('fcm1000_litre'))['fcm1000_litre__sum']
          data_dict[route.name[:-4]][date]["total"] += tm500+sm250+sm500+fcm500+fcm1000

final_obj = DailySessionllyBusinessllySale.objects.filter(sold_to="Leakage", delivery_date__in=date_list, route_id__in=[86,100,144,146,127,120,148,150,154,152,85,99,145,143,241,123,126,147,149,151,153,136,134])

data_dict["final_dict"]["tm500"]["qty"] = final_obj.aggregate(Sum('tm500_pkt'))['tm500_pkt__sum']
data_dict["final_dict"]["tm500"]["rate"] = Product.objects.get(id=1).mrp
data_dict["final_dict"]["tm500"]["liter"] = final_obj.aggregate(Sum('tm500_litre'))['tm500_litre__sum']
data_dict["final_dict"]["tm500"]["cost"] = final_obj.aggregate(Sum('tm500_pkt'))['tm500_pkt__sum'] * Product.objects.get(id=1).mrp

data_dict["final_dict"]["sm250"]["qty"] = final_obj.aggregate(Sum('std250_pkt'))['std250_pkt__sum']
data_dict["final_dict"]["sm250"]["rate"] = Product.objects.get(id=3).mrp
data_dict["final_dict"]["sm250"]["liter"] = final_obj.aggregate(Sum('std250_litre'))['std250_litre__sum']
data_dict["final_dict"]["sm250"]["cost"] = final_obj.aggregate(Sum('std250_pkt'))['std250_pkt__sum'] * Product.objects.get(id=3).mrp

data_dict["final_dict"]["sm500"]["qty"] = final_obj.aggregate(Sum('std500_pkt'))['std500_pkt__sum']
data_dict["final_dict"]["sm500"]["rate"] = Product.objects.get(id=2).mrp
data_dict["final_dict"]["sm500"]["liter"] = final_obj.aggregate(Sum('std500_litre'))['std500_litre__sum']
data_dict["final_dict"]["sm500"]["cost"] = final_obj.aggregate(Sum('std500_pkt'))['std500_pkt__sum'] * Product.objects.get(id=2).mrp

data_dict["final_dict"]["fcm500"]["qty"] = final_obj.aggregate(Sum('fcm500_pkt'))['fcm500_pkt__sum']
data_dict["final_dict"]["fcm500"]["rate"] = Product.objects.get(id=4).mrp
data_dict["final_dict"]["fcm500"]["liter"] = final_obj.aggregate(Sum('fcm500_litre'))['fcm500_litre__sum']
data_dict["final_dict"]["fcm500"]["cost"] = final_obj.aggregate(Sum('fcm500_pkt'))['fcm500_pkt__sum'] * Product.objects.get(id=4).mrp

data_dict["final_dict"]["fcm1000"]["qty"] = final_obj.aggregate(Sum('fcm1000_pkt'))['fcm1000_pkt__sum']
data_dict["final_dict"]["fcm1000"]["rate"] = Product.objects.get(id=6).mrp
data_dict["final_dict"]["fcm1000"]["liter"] = final_obj.aggregate(Sum('fcm1000_litre'))['fcm1000_litre__sum']
data_dict["final_dict"]["fcm1000"]["cost"] = final_obj.aggregate(Sum('fcm1000_pkt'))['fcm1000_pkt__sum'] * Product.objects.get(id=6).mrp

data_dict["final_dict"]["total"]["qty"] = final_obj.aggregate(Sum('tm500_pkt'))['tm500_pkt__sum'] + final_obj.aggregate(Sum('std250_pkt'))['std250_pkt__sum'] + final_obj.aggregate(Sum('std500_pkt'))['std500_pkt__sum'] + final_obj.aggregate(Sum('fcm500_pkt'))['fcm500_pkt__sum'] + final_obj.aggregate(Sum('fcm1000_pkt'))['fcm1000_pkt__sum']
data_dict["final_dict"]["total"]["rate"] = 0
data_dict["final_dict"]["total"]["liter"] = final_obj.aggregate(Sum('tm500_litre'))['tm500_litre__sum'] + final_obj.aggregate(Sum('std250_litre'))['std250_litre__sum'] + final_obj.aggregate(Sum('std500_litre'))['std500_litre__sum'] + final_obj.aggregate(Sum('fcm500_litre'))['fcm500_litre__sum'] + final_obj.aggregate(Sum('fcm1000_litre'))['fcm1000_litre__sum']
data_dict["final_dict"]["total"]["cost"] = data_dict["final_dict"]["tm500"]["cost"] + data_dict["final_dict"]["sm250"]["cost"] + data_dict["final_dict"]["sm500"]["cost"] + data_dict["final_dict"]["fcm500"]["cost"] + data_dict["final_dict"]["fcm1000"]["cost"]

#   data = create_canvas_report_for_leakage_allowance(data_dict,date_list)
#   return Response(data=data, status=status.HTTP_200_OK)

data_dict

{'final_dict': {'tm500': {'qty': 0,
   'rate': Decimal('21.50'),
   'liter': Decimal('0.000'),
   'cost': Decimal('0.00')},
  'sm250': {'qty': 29,
   'rate': Decimal('12.00'),
   'liter': Decimal('7.250'),
   'cost': Decimal('348.00')},
  'sm500': {'qty': 77,
   'rate': Decimal('23.50'),
   'liter': Decimal('38.500'),
   'cost': Decimal('1809.50')},
  'fcm500': {'qty': 55,
   'rate': Decimal('25.50'),
   'liter': Decimal('27.500'),
   'cost': Decimal('1402.50')},
  'fcm1000': {'qty': 1,
   'rate': Decimal('51.00'),
   'liter': Decimal('1.000'),
   'cost': Decimal('51.00')},
  'total': {'qty': 162,
   'rate': 0,
   'liter': Decimal('74.250'),
   'cost': Decimal('3611.00')}},
 '1 - SULUR': {'grand_total': Decimal('10.000'),
  '10-11-2020': {'tm500': Decimal('0.000'),
   'sm250': Decimal('0.000'),
   'sm500': Decimal('10.000'),
   'fcm500': Decimal('0.000'),
   'fcm1000': Decimal('0.000'),
   'total': Decimal('10.000')}},
 'F - RSPURA': {'grand_total': Decimal('18.000'),
  '10-11-2020': {

### Order closing check

In [2]:
business_id = 1424 
order_date = '2020-10-11'
session_list = [2]

indent_status = True
for session_id in session_list:
    route_trace_obj = RouteTrace.objects.get(date=order_date, route__routebusinessmap__business_id=business_id, session_id=session_id)
    if session_id == 1:
        if route_trace_obj.indent_status_id == 3:
            indent_status = False
            break            
    else:
        if route_trace_obj.indent_status_id == 3:
            indent_status = False
print(indent_status)

False


In [17]:
date_list = pd.date_range(start='2020-09-28', end='2020-11-28')
date_list

DatetimeIndex(['2020-09-28', '2020-09-29', '2020-09-30', '2020-10-01',
               '2020-10-02', '2020-10-03', '2020-10-04', '2020-10-05',
               '2020-10-06', '2020-10-07', '2020-10-08', '2020-10-09',
               '2020-10-10', '2020-10-11', '2020-10-12', '2020-10-13',
               '2020-10-14', '2020-10-15', '2020-10-16', '2020-10-17',
               '2020-10-18', '2020-10-19', '2020-10-20', '2020-10-21',
               '2020-10-22', '2020-10-23', '2020-10-24', '2020-10-25',
               '2020-10-26', '2020-10-27', '2020-10-28', '2020-10-29',
               '2020-10-30', '2020-10-31', '2020-11-01', '2020-11-02',
               '2020-11-03', '2020-11-04', '2020-11-05', '2020-11-06',
               '2020-11-07', '2020-11-08', '2020-11-09', '2020-11-10',
               '2020-11-11', '2020-11-12', '2020-11-13', '2020-11-14',
               '2020-11-15', '2020-11-16', '2020-11-17', '2020-11-18',
               '2020-11-19', '2020-11-20', '2020-11-21', '2020-11-22',
      

In [20]:
for product_trace in BusinessWiseProductDiscountTrace.objects.filter(business_wise_discount__business_id=2780, business_wise_discount__product_id=9):
    start_date = str(product_trace.start_date)[:10]
    end_date = str(product_trace.end_date)[:10]
    print(end_date)
    date_list = pd.date_range(start=start_date, end=end_date)
    if from_date in date_list :
        trace_ids.append(product_trace.id)
    if to_date in date_list:
        trace_ids.append(product_trace.id)

In [6]:
def find_out_booth_or_customer_code(string):
    string = str(string)
    index = string.index('b')
    return string[index + 1:]

In [8]:
from_date = '2020-08-01'
to_date = '2020-08-31'
rid = None
order_status = 'Success'

payment_request_obj = PaymentRequest.objects.filter(paymentrequestresponse__time_created__date__range=[from_date,to_date]).order_by('-time_created')
payment_request_list = list(
    payment_request_obj.values_list('id', 'rid', 'status__description', 'enquiry_response_status_latest__name',
                                    'time_modified', 'encrypted_string', 'decrypted_string',
                                    'payment_request_for__name', 'crn', 'amt', 'ppi', 'cks', 'is_wallet_selected',
                                    'is_amount_returened_to_wallet', 'wallet_balance_after_this_transaction',
                                    'time_created'))
payment_request_column = ['id', 'rid', 'request_status_name', 'enquiry_response_status_latest',
                          'latest_enquiryed_time', 'request_encrypted_string', 'request_decrypted_string',
                          'payment_request_for', 'crn', 'amt', 'ppi', 'cks', 'is_wallet_selected',
                          'is_amount_returened_to_wallet', 'wallet_balance_after_this_transaction',
                          'request_time_created']
payment_request_df = pd.DataFrame(payment_request_list, columns=payment_request_column)
# responce
payement_request_ids = list(
    PaymentRequest.objects.filter(time_created__date__range=[from_date,to_date]).values_list(
        'id', flat=True))
payment_responce_obj = PaymentRequestResponse.objects.filter(time_created__date__range=[from_date,to_date])
payment_responce_list = list(
    payment_responce_obj.values_list('payment_request', 'status__name', 'is_enquired', 'encrypted_string',
                                     'decrypted_string', 'brn', 'trn', 'tet', 'pmd', 'stc', 'rmk', 'time_created'))
payment_responce_column = ['payment_request', 'responce_status_name', 'is_enquired', 'responce_encrypted_string',
                           'responce_decrypted_string', 'brn', 'trn', 'tet', 'pmd', 'stc', 'rmk',
                           'responce_time_created']
payment_responce_df = pd.DataFrame(payment_responce_list, columns=payment_responce_column)
# requested user
payment_request_user = PaymentRequestUserMap.objects.filter(payment_request_id__in=payement_request_ids)
payment_request_user_list = list(
    payment_request_user.values_list('payment_request', 'payment_intitated_by', 'payment_intitated_by__first_name',
                                     'payment_intitated_by__last_name'))
payment_request_user_column = ['request_map_request_id', 'user_id', 'first_name', 'last_name']
payment_request_user_df = pd.DataFrame(payment_request_user_list, columns=payment_request_user_column)
# merge user with request table
payment_request_df = pd.merge(payment_request_df, payment_request_user_df, how='left', left_on='id',
                              right_on='request_map_request_id')

# merge request with responce
final_df = pd.merge(payment_request_df, payment_responce_df, left_on='id', right_on='payment_request', how='left')
if not final_df.empty:
    final_df['booth_or_customer_code'] = final_df.apply(lambda x: find_out_booth_or_customer_code(x['crn']), axis=1)

final_df = final_df.fillna(0)

# excel part
final_df = final_df.reindex(columns=[
    'first_name', 'last_name', 'booth_or_customer_code', 'amt', 'request_time_created', 
    'payment_request_for', 'responce_status_name', 'responce_time_created', 'enquiry_response_status_latest', 
    'latest_enquiryed_time', 'is_wallet_selected', 'rid', 'crn', 'trn', 'tet', 'brn', 'pmd'
])


final_df=final_df.rename(columns={
    'first_name':'First Name', 'last_name':'Last Name', 'booth_or_customer_code': 'Booth/Customer Code', 'amt': 'Amount', 'request_time_created': 'Requested Time', 
    'payment_request_for': 'Requested For', 'responce_status_name': 'Responce Status', 'responce_time_created': 'Responce Time', 'enquiry_response_status_latest': 'Enquiry Status', 
    'latest_enquiryed_time': 'Enquiry Time', 'is_wallet_selected': 'Is Wallet Used', 'rid': 'RID', 'crn': 'CRN', 'trn': 'Transaction Number', 'tet': 'Transaction Time', 'brn': 'BRN', 'pmd': 'Payment Mode'
})

file_name = ''
file_path = ''
if rid == None:
    if order_status == 'Success':
        final_df = final_df[(final_df['Responce Status'] == 'Success') | (final_df['Enquiry Status'] == 'Success')]
    elif order_status == 'Failed':
        final_df = final_df[(final_df['Enquiry Status'] == 'Failed') & (final_df['Responce Status'] != 'Success')]
    elif order_status == 'Pending':
        final_df = final_df[(final_df['Enquiry Status'] == 'Pending') & (final_df['Responce Status'] != 'Success')]
    else:
        final_df = final_df[(final_df['Responce Status'] == 0) & (final_df['Enquiry Status'] == 0)]

    final_df['Requested Time'] = final_df['Requested Time'].astype(str)
    final_df['Enquiry Time'] = final_df['Enquiry Time'].astype(str)

    final_df.index = pd.RangeIndex(start=1, stop=final_df.shape[0]+1, step=1)

    final_df['Responce Time'] = final_df['Responce Time'].astype(str)
    final_df['Transaction Time'] = final_df['Transaction Time'].astype(str)

    file_name = from_date + '_to_' + to_date + '_'+order_status+"_status.xlsx"
    file_path = os.path.join('static/media/zone_wise_report', file_name)
    final_df.to_excel(file_path)
    alert = "Data Not Avaliable"
    show_alert = False
    if final_df.empty:
        show_alert = True
else:
    final_df = final_df[(final_df['RID'] == str(rid))]

    final_df['Requested Time'] = final_df['Requested Time'].astype(str)
    final_df['Enquiry Time'] = final_df['Enquiry Time'].astype(str)

    final_df.index = pd.RangeIndex(start=1, stop=final_df.shape[0]+1, step=1)

    final_df['Responce Time'] = final_df['Responce Time'].astype(str)
    final_df['Transaction Time'] = final_df['Transaction Time'].astype(str)

    file_name = from_date + '_to_' + to_date + '_'+str(rid)+"_status.xlsx"
    file_path = os.path.join('static/media/zone_wise_report', file_name)
    final_df.to_excel(file_path)

In [10]:
from_date = '2020-08-01'
to_date = '2020-08-31'

payment_responce_obj = PaymentRequestResponse.objects.filter(time_created__date__range=[from_date,to_date])
amount = payment_responce_obj.aggregate(Sum('amt'))['amt__sum']
amount

Decimal('11811687.81')

In [5]:
def serve_product_price_for_date_range(business_codes, from_date, to_date, group_id):
    price_per_user_profile_dict = {}
    for business_code in business_codes:
        business_obj = Business.objects.get(code=business_code)
        if not business_obj.user_profile.id in price_per_user_profile_dict:
            price_per_user_profile_dict[business_obj.user_profile.id] = []
        
        commission_product_ids = list(BusinessProductConcessionMap.objects.filter(business_type_id=business_obj.business_type_id, product__is_active=True, concession_type_id=1, product__group_id=group_id).values_list('product_id', flat=True))
        discount_product_ids = list(BusinessProductConcessionMap.objects.filter(business_type_id=business_obj.business_type_id, product__is_active=True, concession_type_id=2, product__group_id=group_id).values_list('product_id', flat=True))
        price_per_user_profile_dict[business_obj.user_profile.id] = []
        if len(commission_product_ids) != 0:
            for commission_product_id in commission_product_ids:
                if ProductTrace.objects.filter(product_id=commission_product_id).exists():
                    trace_ids = []
                    for product_trace in ProductTrace.objects.filter(product_id=commission_product_id):
                        date_list = pd.date_range(product_trace.start_date, product_trace.end_date)
                        if from_date in date_list :
                            trace_ids.append(product_trace.id)
                        if to_date in date_list:
                            trace_ids.append(product_trace.id)
                    if len(set(trace_ids)) == 1:
                        product_trace_id = trace_ids[0]
                        product_trace_obj = ProductTrace.objects.get(id=product_trace_id)
                        data_dict = {
                            'product_id': product_trace_obj.product.id,
                            'product_mrp': product_trace_obj.mrp
                        }
                        price_per_user_profile_dict[business_obj.user_profile.id].append(data_dict)
                    else:
                        data_dict = {
                            'is_multiple_price': True,
                            'return_data': serve_product_diffrence_list(business_obj, 'product_trace', group_id)
                        }
                        return data_dict
        business_type_id = business_obj.business_type_id
        if len(discount_product_ids) != 0:
            if business_type_id == 1 or business_type_id == 2 or business_type_id == 3 or business_type_id == 9 or business_type_id == 11:  
                for discount_product_id in discount_product_ids:
                    if BusinessTypeWiseProductDiscountTrace.objects.filter(business_type_wise_discount__business_type_id=business_type_id, business_type_wise_discount__product_id=discount_product_id).exists():
                        trace_ids = []
                        for product_trace in BusinessTypeWiseProductDiscountTrace.objects.filter(business_type_wise_discount__business_type_id=business_type_id, business_type_wise_discount__product_id=discount_product_id):
                            date_list = pd.date_range(product_trace.start_date, product_trace.end_date)
                            if from_date in date_list :
                                trace_ids.append(product_trace.id)
                            if to_date in date_list:
                                trace_ids.append(product_trace.id)
                        if len(set(trace_ids)) == 1:
                            product_trace_id = trace_ids[0]
                            business_type_wise_trace_obj = BusinessTypeWiseProductDiscountTrace.objects.get(id=product_trace_id)
                            data_dict = {
                                'product_id': business_type_wise_trace_obj.business_type_wise_discount.product.id,
                                'product_mrp': business_type_wise_trace_obj.discounted_price
                            }
                            price_per_user_profile_dict[business_obj.user_profile.id].append(data_dict)
                        else:
                            data_dict = {
                                'is_multiple_price': True,
                                'return_data': serve_product_diffrence_list(business_obj, 'business_type_wise', group_id)
                            }
                            return data_dict
            else:
                for discount_product_id in discount_product_ids:
                    if BusinessWiseProductDiscountTrace.objects.filter(business_wise_discount__business_id=business_obj.id, business_wise_discount__product_id=discount_product_id).exists():
                        trace_ids = []
                        for product_trace in BusinessWiseProductDiscountTrace.objects.filter(business_wise_discount__business_id=business_obj.id, business_wise_discount__product_id=discount_product_id):
                            date_list = pd.date_range(product_trace.start_date, product_trace.end_date)
                            if from_date in date_list :
                                trace_ids.append(product_trace.id)
                            if to_date in date_list:
                                trace_ids.append(product_trace.id)
                        if len(set(trace_ids)) == 1:
                            product_trace_id = trace_ids[0]
                            business_wise_trace_obj = BusinessWiseProductDiscountTrace.objects.get(id=product_trace_id)
                            data_dict = {
                                'product_id': business_wise_trace_obj.business_wise_discount.product.id,
                                'product_mrp': business_wise_trace_obj.discounted_price
                            }
                            price_per_user_profile_dict[business_obj.user_profile.id].append(data_dict)
                        else:
                            data_dict = {
                                'is_multiple_price': True,
                                'return_data': serve_product_diffrence_list(business_obj, 'business_wise', group_id)
                            }
                            return data_dict
    data_dict = {
        'is_multiple_price': False,
        'return_data': price_per_user_profile_dict
    }
    return data_dict

In [9]:
business_codes = []
business_types = {}
from_date = '2021-05-01'
to_date = '2021-05-15'
option_type = 'by_business'

date_list = pd.date_range(start=from_date, end=to_date)
if option_type == 'by_business':
  business_codes = [8101,]
  business_types['business']= business_codes
  business_names = business_codes
elif request.data['option_type'] == 'by_business_type':
  for business_type in request.data['business_type_list']:
      if business_type == 'private_institute':
          pvt_business_code_list = list(Business.objects.filter(business_type_id=4).values_list('code', flat=True))
          business_codes += pvt_business_code_list
          business_types['private_institute'] = pvt_business_code_list
      elif business_type == 'govt_institute':
          gvt_business_code_list = list(Business.objects.filter(business_type_id=5).values_list('code', flat=True))
          business_codes += gvt_business_code_list
          business_types['govt_institute'] = gvt_business_code_list
      elif business_type == 'union_parlour':
          union_parlour_business_code_list = ['8105', '8115', '8104', '8101', '8117', '8116', '8109'] 
          business_codes += union_parlour_business_code_list
          business_types['union_parlour'] = union_parlour_business_code_list
      elif business_type == 'union_booth':
          union_booth_business_code_list = ['1', '1999', '2000']
          business_types['union_booth'] = union_booth_business_code_list
          business_codes += union_booth_business_code_list
      elif business_type == 'nilgris':
          nilgris_booth_business_code_list = list(Business.objects.filter(zone_id=11).values_list('code', flat=True))
          business_codes += nilgris_booth_business_code_list
          business_types['nilgris'] = nilgris_booth_business_code_list
      elif business_type == 'tirupur':
          tirupur_booth_business_code_list = list(Business.objects.filter(zone_id=13).values_list('code', flat=True))
          business_codes += tirupur_booth_business_code_list
          business_types['tirupur'] = tirupur_booth_business_code_list
      elif business_type == 'society':
          society_business_code_list = list(Business.objects.filter(business_type_id=10).values_list('code', flat=True))
          business_codes += society_business_code_list
          business_types['society'] = society_business_code_list
business_codes_for_check = set(list(DailySessionllyBusinessllySale.objects.filter(delivery_date__in=date_list, sold_to='Agent', business__code__in=business_codes).values_list('business__code', flat=True)))

check_dict = serve_product_price_for_date_range(business_codes_for_check, from_date, to_date, 2)
if check_dict['is_multiple_price']:
    print(check_dict)

daily_sessionly_obj = DailySessionllyBusinessllySale.objects.filter(delivery_date__in=date_list, sold_to='Agent')

xlxs_file_name = "all_booth_curd_sale_"+str(date_list[0])[:10]+ " to " +str(date_list[-1])[:10] + f"_{business_names}" + ".xlsx"
xlxs_file_path = os.path.join('static/media/zone_wise_report/', xlxs_file_name)

df_dict = {}
for instution in business_types:
    instution_sale_list = daily_sessionly_obj.filter(business__code__in=business_types[instution]).values_list('business__name', 'curd500_kgs', 'curd500_cost', 'curd150_kgs', 'curd150_cost', 
                                                                                                                'cupcurd_kgs', 'cupcurd_cost', 'buttermilk200_litre', 'buttermilk200_cost', 'curd_kgs', 
                                                                                                                'curd_cost', 'buttermilk_litre', 'buttermilk_cost', 'bm_jar200_litre', 'bm_jar200_cost')  

    instution_sale_column = ['Business', 'CURD500 Kgs', 'CURD500 Cost', 'CURD150 Kgs', 'CURD150 Cost', 'CUPCURD Kgs', 'CUPCURD Cost', 'BUTTERMILK200 Litre', 'BUTTERMILK200 Cost', 'CURD Kgs', 'CURD Cost', 'BUTTERMILK Litre', 'BUTTERMILK Cost', 'BMJAR Litre', 'BMJAR Cost']  
    instution_sale_df = pd.DataFrame(instution_sale_list, columns=instution_sale_column)
    instution_sale_df = instution_sale_df.groupby('Business').agg({'CURD500 Kgs': sum, 'CURD500 Cost': sum, 'CURD150 Kgs': sum, 'CURD150 Cost': sum, 'CUPCURD Kgs': sum, 'CUPCURD Cost': sum, 'BUTTERMILK200 Litre': sum, 'BUTTERMILK200 Cost': sum, 'CURD Kgs': sum, 'CURD Cost': sum, 'BUTTERMILK Litre': sum, 'BUTTERMILK Cost': sum, 'BMJAR Litre': sum, 'BMJAR Cost': sum}).reset_index() 

    instution_sale_df['TOTAL Kgs'] = instution_sale_df['BUTTERMILK Litre'] + instution_sale_df['CURD Kgs']
    instution_sale_df['TOTAL Cost'] = instution_sale_df['BUTTERMILK Cost'] + instution_sale_df['CURD Cost']

    instution_sale_df = instution_sale_df.drop(columns=['BUTTERMILK Litre', 'BUTTERMILK Cost', 'CURD Kgs', 'CURD Cost'])

    instution_sale_df = instution_sale_df.append(instution_sale_df.sum(), ignore_index=True)
    instution_sale_df.iloc[-1, instution_sale_df.columns.get_loc('Business')] = 'Total'

    instution_sale_df = instution_sale_df[instution_sale_df['TOTAL Cost'] != 0]

    for i in list(instution_sale_df.columns)[1:]:
        if i[-3:] != 'Kgs' and i[-5:] != 'Litre':
            instution_sale_df[i] = round(instution_sale_df[i].astype(float))
        else:
            instution_sale_df[i] = instution_sale_df[i].astype(float)

    instution_sale_df = instution_sale_df.reset_index(drop=True)
    instution_sale_df.index += 1
    df_dict[instution] = instution_sale_df

with pd.ExcelWriter(xlxs_file_path) as writer:  
    for sheet_name in df_dict:
        df_dict[sheet_name].to_excel(writer, sheet_name=sheet_name)

last_bill_number_count = InstititionBillNumberIdBank.objects.filter(id=1)[0].last_count
bill_no = int(last_bill_number_count) + 1

business_code_list = business_codes

date_list = list(set(list(DailySessionllyBusinessllySale.objects.filter(delivery_date__in=date_list).values_list('delivery_date', flat=True))))
date_list =  sorted(date_list)

#----------------------------------input getting---------------------------------------------#
bill_data_obj = DailySessionllyBusinessllySale.objects.filter(delivery_date__in=date_list, business__code__in=business_code_list, sold_to='Agent' )

#fill missing date
date_fill_list = list(set(list(bill_data_obj.values_list('business__name','business_id', 'business__code', 'business__zone__name', 'business__user_profile_id'))))
date_fill_column = ['business_name', 'business_id', 'business_code', 'zone_name', 'user_profile_id']
date_fill_df = pd.DataFrame(date_fill_list, columns=date_fill_column)
date_fill_df = date_fill_df.assign(date =  [date_list for i in date_fill_df.index])
date_fill_df = date_fill_df.explode('date').reset_index(drop=True)

#morning_df
mor_bill_data_list = list(bill_data_obj.filter(session_id=1).values_list('business__name', 'delivery_date', 'curd500_kgs', 'curd150_kgs', 'cupcurd_kgs', 'buttermilk200_litre', 'bm_jar200_litre'))
mor_bill_data_column = ['business_name', 'date', 'curd500_mor', 'curd150_mor', 'cupcurd_mor', 'buttermilk200_mor', 'bm_jar200_mor']

mor_bill_data_df = pd.DataFrame(mor_bill_data_list, columns=mor_bill_data_column)

mor_group_by_df = mor_bill_data_df.groupby(['business_name', 'date']).agg({'curd500_mor': sum, 'curd150_mor': sum, 'cupcurd_mor': sum, 'buttermilk200_mor': sum, 'bm_jar200_mor': sum}).reset_index()                 

#evening_df
eve_bill_data_list = list(bill_data_obj.filter(session_id=2).values_list('business__name', 'delivery_date', 'curd500_kgs', 'curd150_kgs', 'cupcurd_kgs', 'buttermilk200_litre', 'bm_jar200_litre'))
eve_bill_data_column = ['business_name', 'date', 'curd500_eve', 'curd150_eve', 'cupcurd_eve', 'buttermilk200_eve', 'bm_jar200_eve']

eve_bill_data_df = pd.DataFrame(eve_bill_data_list, columns=eve_bill_data_column)
if eve_bill_data_df.empty:
    eve_bill_data_df['business_name'] = mor_group_by_df['business_name']
    eve_bill_data_df['date'] = mor_group_by_df['date']
    eve_bill_data_df = eve_bill_data_df.fillna(0)

eve_group_by_df = eve_bill_data_df.groupby(['business_name', 'date']).agg({'curd500_eve': sum, 'curd150_eve': sum, 'cupcurd_eve': sum, 'buttermilk200_eve': sum, 'bm_jar200_eve': sum}).reset_index()                 
# merg mor and eve df
bill_df = pd.merge(mor_group_by_df, eve_group_by_df, left_on=['business_name', 'date'], right_on=['business_name', 'date'], how='outer')
# bill df and datefill_df
bill_df = pd.merge(date_fill_df, bill_df, left_on=['business_name', 'date'], right_on=['business_name', 'date'], how='outer')

bill_df = bill_df.fillna(Decimal("0.000"))

bill_df["mor_total"] = bill_df['curd500_mor'] + bill_df['curd150_mor'] + bill_df['cupcurd_mor'] + bill_df['buttermilk200_mor'] + bill_df['bm_jar200_mor']
bill_df["eve_total"] = bill_df['curd500_eve'] + bill_df['curd150_eve'] + bill_df['cupcurd_eve'] + bill_df['buttermilk200_eve'] + bill_df['bm_jar200_eve']

bill_df['mor_total'] = bill_df['mor_total'].astype(float)
bill_df['eve_total'] = bill_df['eve_total'].astype(float)

bill_filter_df = bill_df.groupby(['business_name']).agg({'mor_total': sum, 'eve_total':sum}).reset_index()
bill_filter_df = bill_filter_df[(bill_filter_df['mor_total'] != 0) | (bill_filter_df['eve_total'] != 0)]
valied = bill_df['business_name'].isin(bill_filter_df['business_name'])
bill_df = bill_df[valied]

bill_df_dict = bill_df.groupby(['business_name']).apply(lambda x:x.to_dict('r')).to_dict()
bill_df_dict

{'8101 - Own parlour': [{'business_name': '8101 - Own parlour',
   'business_id': 1775,
   'business_code': '8101',
   'zone_name': 'WEST',
   'user_profile_id': 1786,
   'date': datetime.date(2021, 5, 1),
   'curd500_mor': Decimal('24.000'),
   'curd150_mor': Decimal('7.500'),
   'cupcurd_mor': Decimal('0.000'),
   'buttermilk200_mor': Decimal('10.000'),
   'bm_jar200_mor': Decimal('12.000'),
   'curd500_eve': Decimal('48.000'),
   'curd150_eve': Decimal('0.000'),
   'cupcurd_eve': Decimal('0.000'),
   'buttermilk200_eve': Decimal('0.000'),
   'bm_jar200_eve': Decimal('0.000'),
   'mor_total': 53.5,
   'eve_total': 48.0},
  {'business_name': '8101 - Own parlour',
   'business_id': 1775,
   'business_code': '8101',
   'zone_name': 'WEST',
   'user_profile_id': 1786,
   'date': datetime.date(2021, 5, 2),
   'curd500_mor': Decimal('24.000'),
   'curd150_mor': Decimal('7.500'),
   'cupcurd_mor': Decimal('0.000'),
   'buttermilk200_mor': Decimal('10.000'),
   'bm_jar200_mor': Decimal('12.0

In [88]:
from datetime import datetime 
from num2words import num2words
import pytz
indian = pytz.timezone('Asia/Kolkata')
    #---------------------------------------------------------------pdf-----------------------------------------------------------------------------------#

    file_name = "all_booth_curd_sale_"+str(date_list[0])[:10]+ " to " +str(date_list[-1])[:10] + ".pdf"
    # file_path = os.path.join('static/media', file_name)
    file_path = os.path.join('static/media/zone_wise_report/', file_name)

    mycanvas = canvas.Canvas(file_path, pagesize=A4)

    route_obj = RouteBusinessMap.objects.all()

    zone_num_dict = {
              "EAST": "(94890 43713)",
              "WEST": "(94890 43711)",
              "SOUTH": "(94890 43708)",
              "POLLACHI": "(94890 43708)",
              "NORTH": "(94890 43700)",
              "MTP": "(94890 43711)",
              "NILGIRIS": "",
              "MPCS": "",
              "TIRUPPUR": "",
              "CURD Zone": "",
              "CHENNAI Aavin": "",
          }

    light_color = 0x000000
    dark_color = 0x000000

    for datas in bill_df_dict:
        if BusinessWiseBillNumber.objects.filter(from_date=from_date, to_date=to_date, business_id=bill_df_dict[datas][0]['business_id'], product_group_type_id=2).exists():
            temp_bill_number = BusinessWiseBillNumber.objects.filter(from_date=from_date, to_date=to_date, business_id=bill_df_dict[datas][0]['business_id'], product_group_type_id=2)[0].bill_number
        else:
            temp_bill_number = bill_no
            business_wise_bill_number = BusinessWiseBillNumber(from_date=from_date, to_date=to_date, business_id=bill_df_dict[datas][0]['business_id'], bill_number=temp_bill_number, created_by=request.user, product_group_type_id=2)
            business_wise_bill_number.save()
            bill_no +=1

        business_code = bill_df_dict[datas][0]['business_code']
        business_name = bill_df_dict[datas][0]['business_name']
        business_zone = bill_df_dict[datas][0]['zone_name']
        user_profile_id = bill_df_dict[datas][0]['user_profile_id']
        bill_no_of = temp_bill_number
        curd500 = 0
        curd150 = 0
        curd100 = 0
        butter_milk = 0
        product_dict = check_dict['return_data'][user_profile_id]
        for product in product_dict:
            if product["product_id"] == 25:
                curd500 = product["product_mrp"] * 2
            if product["product_id"] == 7:
                curd150 = product["product_mrp"] * Decimal(6.667)
            if product["product_id"] == 10:
                curd100 = product["product_mrp"] * 10
            if product["product_id"] == 8:
                butter_milk = product["product_mrp"] * 5 
            if product["product_id"] == 26:
                butter_milk_jar = product["product_mrp"] * 5

        route_name = route_obj.filter(business__code=business_code).order_by('route__session_id')[0].route.name[:-4]

        mycanvas.setFillColor(HexColor(dark_color))
        mycanvas.setFont('Helvetica', 12.5)
        mycanvas.drawCentredString(300, 820,'The Coimbatore District Co-Operative Milk Producers Union Ltd, Pachapalayam, Coimbatore - 641 010')
        mycanvas.setFont('Helvetica', 13)
        # mycanvas.drawCentredString(500-150, 560+ 40+200+2, str(business_name.upper()))

        mycanvas.setDash(6,3)
        mycanvas.setLineWidth(0)
    #     mycanvas.line(406-150, 557+ 40+200+2, 593-150, 557+ 40+200+2)
        mycanvas.drawCentredString(300, 560+ 41+200+1, 'GST NO.: ' + str("33AAAAT7787L2ZU"))

        # top_double_line
        mycanvas.line(10, 530+ 40+ 10+200+15, 585, 530+ 40+ 10+200+15)
    #     mycanvas.line(10, 527+ 40+ 10+200-5, 710, 527+ 40+ 10+200-5)
        # under_double_line
    #     mycanvas.line(10, 510+ 40+ 10+200-5, 710, 510+ 40+ 10+200-5)
        mycanvas.line(10, 507+ 40+ 10+200+15, 585, 507+ 40+ 10+200+15)

        from_date = datetime.strptime(str(date_list[0])[:10], '%Y-%m-%d')
        to_date = datetime.strptime(str(date_list[-1])[:10], '%Y-%m-%d')

    #     # BILL FOR THE SUPPLY OF MILK
        mycanvas.setFont('Helvetica', 12)
        mycanvas.setFillColor(HexColor(dark_color))
        mycanvas.drawString(10,514+ 60+ 10+200-5,"BILL FOR THE SUPPLY OF CURD & BUTTER MILK ")

        mycanvas.drawRightString(450,514+ 60+ 10+200-5,"ROUTE : "+str(route_name))

        mycanvas.drawRightString(585,514+ 60+ 10+200-5,"Bill No.: "+str(bill_no_of))

        mycanvas.drawRightString(585,507+ 40+ 10+200-8,"  PERIOD : FROM "+str(datetime.strftime(from_date, '%d/%m/%Y'))+" TO "+str(datetime.strftime(to_date, '%d/%m/%Y')))


        mycanvas.setFont('Helvetica', 13)
        mycanvas.drawString(10, 507+ 40+ 10+200-8, str(business_name.upper()))
        mycanvas.setFont('Helvetica', 8)

        # UNDER BILL FOR THE SUPPLY OF MILK
        mycanvas.line(10, 475+ 40+ 10+200+10, 585, 475+ 40+ 10+200+10)
        mycanvas.line(10, 473+ 40+ 10+200+10, 585, 473+ 40+ 10+200+10)


        x_axis_line = 10
        y_axis_line = 473+ 40+ 10+200+10
        y_axis = 443+ 40+ 10+200+10

        # Heading
        mycanvas.drawString(x_axis_line+5,y_axis-3,"Date")
        x_adjust = 45
        head_list = ["CURD500","CURD150","CUPCURD","BUTTER MILK","BM JAR", "TOTAL"]
        i = 0
        for head in head_list:
            mycanvas.drawString(x_axis_line +x_adjust,y_axis-3,"Mor")
            x_adjust += 45
            mycanvas.drawCentredString(x_axis_line +x_adjust-15,y_axis+15,str(head))
            mycanvas.drawString(x_axis_line +x_adjust,y_axis-3,"Eve")
            x_adjust += 45

        mycanvas.line(x_axis_line, y_axis+11, x_axis_line+575, y_axis + 11)
        mycanvas.line(x_axis_line, y_axis-8, x_axis_line+575, y_axis-8)
        mycanvas.line(x_axis_line, y_axis-10, x_axis_line+575, y_axis-10)

        curd500_mor = 0
        curd150_mor = 0
        cupcurd_mor = 0
        buttermilk200_mor = 0
        bm_jar200_mor = 0

        curd500_eve = 0
        curd150_eve = 0
        cupcurd_eve = 0
        buttermilk200_eve = 0
        bm_jar200_eve = 0
        mor_total = 0
        eve_total = 0

        x_adjust = 0

        for value in bill_df_dict[datas]:
            mycanvas.setFont('Helvetica', 7)
            mycanvas.drawString(x_axis_line+10 +x_adjust,y_axis-25,str(value['date'])[-2:])
            x_adjust += 45


            if value['curd500_mor'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-25,str(round(value['curd500_mor'],3)))
                curd500_mor += value['curd500_mor']
            x_adjust += 45

            if value['curd500_eve'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-25,str(round(value['curd500_eve'],3)))
                curd500_eve += value['curd500_eve']
            x_adjust += 45

            if value['curd150_mor'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-25,str(round(value['curd150_mor'],3)))
                curd150_mor += value['curd150_mor']
            x_adjust += 45

            if value['curd150_eve'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-25,str(round(value['curd150_eve'],3)))
                curd150_eve += value['curd150_eve']
            x_adjust += 45


            if value['cupcurd_mor'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-25,str(round(value['cupcurd_mor'],3)))
                cupcurd_mor += value['cupcurd_mor']
            x_adjust += 45

            if value['cupcurd_eve'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+23,y_axis-25,str(round(value['cupcurd_eve'],3)))
                cupcurd_eve += value['cupcurd_eve']
            x_adjust += 45

            if value['buttermilk200_mor'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+23,y_axis-25,str(round(value['buttermilk200_mor'],3)))
                buttermilk200_mor += value['buttermilk200_mor']
            x_adjust += 45

            if value['buttermilk200_eve'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+22,y_axis-25,str(round(value['buttermilk200_eve'],3)))
                buttermilk200_eve += value['buttermilk200_eve']
            x_adjust += 45

            if value['bm_jar200_mor'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+23,y_axis-25,str(round(value['bm_jar200_mor'],3)))
                bm_jar200_mor += value['bm_jar200_mor']
            x_adjust += 45

            if value['bm_jar200_eve'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+22,y_axis-25,str(round(value['bm_jar200_eve'],3)))
                bm_jar200_eve += value['bm_jar200_eve']
            x_adjust += 45

            if value['mor_total'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+20,y_axis-25,str(round(value['mor_total'],3)))
                mor_total += value['mor_total']
            x_adjust += 45

            if value['eve_total'] !=0:
                mycanvas.drawRightString(x_axis_line +x_adjust+29,y_axis-25,str(round(value['eve_total'],3)))
                eve_total += value['eve_total']
            x_adjust += 45


            y_axis -=12
            x_adjust = 0
        mycanvas.setFont('Helvetica', 7)
        x_adjust = 45

        y_axis -=5
        mycanvas.drawRightString(x_axis_line +x_adjust,y_axis-45, "KGs")
        mycanvas.drawRightString(x_axis_line +x_adjust+15,y_axis-65, "MRP Rs")
        mycanvas.drawRightString(x_axis_line +x_adjust+7,y_axis-85, "COST")
        mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-25,str(round(curd500_mor,3)))
        x_adjust += 45

        mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-45,str(round(curd500_mor + curd500_eve,3)))
        mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-65,str(round(curd500,2)))
        curd500_cost = (curd500_mor + curd500_eve) * curd500
        mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-85,str(round(curd500_cost)))
        mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-25,str(round(curd500_eve,3)))
        x_adjust += 45

        mycanvas.drawRightString(x_axis_line +x_adjust,y_axis-45, "KGs")
        mycanvas.drawRightString(x_axis_line +x_adjust+15,y_axis-65, "MRP Rs")
        mycanvas.drawRightString(x_axis_line +x_adjust+7,y_axis-85, "COST")
        mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-25,str(round(curd150_mor,3)))
        x_adjust += 45


        mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-45,str(round(curd150_eve + curd150_mor,3)))
        mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-65,str(round(curd150,2)))
        curd150_cost = (curd150_eve + curd150_mor) * curd150
        mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-85,str(round(curd150_cost)))
        mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-25,str(round(curd150_eve,3)))
        x_adjust += 45


        mycanvas.drawRightString(x_axis_line +x_adjust,y_axis-45, "KGs")
        mycanvas.drawRightString(x_axis_line +x_adjust+15,y_axis-65, "MRP Rs")
        mycanvas.drawRightString(x_axis_line +x_adjust+7,y_axis-85, "COST")
        mycanvas.drawRightString(x_axis_line +x_adjust+25,y_axis-25,str(round(cupcurd_mor,3)))
        x_adjust += 45

        mycanvas.drawRightString(x_axis_line +x_adjust+23,y_axis-45,str(round(cupcurd_mor + cupcurd_eve,3)))
        mycanvas.drawRightString(x_axis_line +x_adjust+23,y_axis-65,str(round(curd100,2)))
        cupcurd_cost = (cupcurd_mor + cupcurd_eve) * curd100
        mycanvas.drawRightString(x_axis_line +x_adjust+23,y_axis-85,str(round(cupcurd_cost)))
        mycanvas.drawRightString(x_axis_line +x_adjust+23,y_axis-25,str(round(cupcurd_eve,3)))
        x_adjust += 45


        mycanvas.drawRightString(x_axis_line +x_adjust,y_axis-45, "KGs")
        mycanvas.drawRightString(x_axis_line +x_adjust+15,y_axis-65, "MRP Rs")
        mycanvas.drawRightString(x_axis_line +x_adjust+7,y_axis-85, "COST")
        mycanvas.drawRightString(x_axis_line +x_adjust+23,y_axis-25,str(round(buttermilk200_mor,3)))
        x_adjust += 45


        mycanvas.drawRightString(x_axis_line +x_adjust+22,y_axis-45,str(round(buttermilk200_eve + buttermilk200_mor,3)))
        mycanvas.drawRightString(x_axis_line +x_adjust+22,y_axis-65,str(round(butter_milk,2)))
        buttermilk200_cost = (buttermilk200_eve + buttermilk200_mor) * butter_milk
        mycanvas.drawRightString(x_axis_line +x_adjust+22,y_axis-85,str(round(buttermilk200_cost)))
        mycanvas.drawRightString(x_axis_line +x_adjust+22,y_axis-25,str(round(buttermilk200_eve,3)))
        x_adjust += 45


        mycanvas.drawRightString(x_axis_line +x_adjust,y_axis-45, "KGs")
        mycanvas.drawRightString(x_axis_line +x_adjust+15,y_axis-65, "MRP Rs")
        mycanvas.drawRightString(x_axis_line +x_adjust+7,y_axis-85, "COST")
        mycanvas.drawRightString(x_axis_line +x_adjust+23,y_axis-25,str(round(bm_jar200_mor,3)))
        x_adjust += 45

        mycanvas.drawRightString(x_axis_line +x_adjust+22,y_axis-45,str(round(bm_jar200_eve + bm_jar200_mor,3)))
        mycanvas.drawRightString(x_axis_line +x_adjust+22,y_axis-65,str(round(butter_milk_jar,2)))
        buttermilk200jar_cost = (bm_jar200_eve + bm_jar200_mor) * butter_milk_jar
        mycanvas.drawRightString(x_axis_line +x_adjust+22,y_axis-85,str(round(buttermilk200jar_cost)))
        mycanvas.drawRightString(x_axis_line +x_adjust+22,y_axis-25,str(round(bm_jar200_eve,3)))
        x_adjust += 45


        mycanvas.drawRightString(x_axis_line +x_adjust,y_axis-45, "KGs")
        mycanvas.drawRightString(x_axis_line +x_adjust+7,y_axis-85, "COST")
        mycanvas.drawRightString(x_axis_line +x_adjust+20,y_axis-25,str(round(mor_total,3)))
        x_adjust += 45


        total_cost = curd500_cost + curd150_cost + cupcurd_cost + buttermilk200_cost+buttermilk200jar_cost
        mycanvas.drawRightString(x_axis_line +x_adjust+29,y_axis-45,str(round(eve_total + mor_total,3)))
        mycanvas.drawRightString(x_axis_line +x_adjust+29,y_axis-85,str(round(total_cost)))
        mycanvas.drawRightString(x_axis_line +x_adjust+29,y_axis-25,str(round(eve_total,3)))

        y_axis +=5


        mycanvas.line(x_axis_line, y_axis-17, x_axis_line+575, y_axis-17)
        mycanvas.line(x_axis_line, y_axis-37, x_axis_line+575, y_axis-37)
        mycanvas.line(x_axis_line, y_axis-57, x_axis_line+575, y_axis-57)
        mycanvas.line(x_axis_line, y_axis-77, x_axis_line+575, y_axis-77)
        mycanvas.line(x_axis_line, y_axis-97, x_axis_line+575, y_axis-97)

        #--lines--
        mycanvas.line(x_axis_line, y_axis_line, x_axis_line, y_axis-17-40-20-20)
        mycanvas.line(x_axis_line+50-20, y_axis_line, x_axis_line+50-20, y_axis-17-40)
        mycanvas.line(x_axis_line+132-25-32, y_axis_line-20, x_axis_line+132-25-32, y_axis-17-20)
        mycanvas.line(x_axis_line+213-35-60, y_axis_line, x_axis_line+213-35-60, y_axis-17-40-20-20)

        mycanvas.line(x_axis_line+294-55-76, y_axis_line-20, x_axis_line+294-55-76, y_axis-17-20)
        mycanvas.line(x_axis_line+375-60-107, y_axis_line, x_axis_line+375-60-107, y_axis-17-40-20-20)
        mycanvas.line(x_axis_line+456-65-136, y_axis_line-20, x_axis_line+456-65-136, y_axis-17-20)

        mycanvas.line(x_axis_line+537-80-159, y_axis_line, x_axis_line+537-80-159, y_axis-17-40-20-20)
        mycanvas.line(x_axis_line+618-95-182, y_axis_line-20, x_axis_line+618-95-182, y_axis-17-20)
        mycanvas.line(x_axis_line+699-100-213, y_axis_line, x_axis_line+699-100-213, y_axis-17-40-20-20)

        mycanvas.line(x_axis_line+700-113-155, y_axis_line-20, x_axis_line+700-113-155, y_axis-17-40+20)
        mycanvas.line(x_axis_line+700-69-155, y_axis_line, x_axis_line+700-69-155, y_axis-17-40-20-20)
        mycanvas.line(x_axis_line+700-180, y_axis_line-20, x_axis_line+700-180, y_axis-17-40+20)

        mycanvas.line(585, y_axis_line,585, y_axis-17-40-20-20)



        words = num2words(round(total_cost), lang='en_IN')
        mycanvas.setFont('Helvetica', 11)
        mycanvas.drawString(x_axis_line,y_axis-37-40-20-15-5,"GST=0%, " + "  Rupees.  "+str(words.upper())+" "+"Only")
        mycanvas.drawString(x_axis_line,y_axis-57-40-25+2-15,"Terms and Conditions:")
        mycanvas.drawRightString(585,y_axis-57-40-25+2-15,"for GENERAL MANAGER")
        mycanvas.line(x_axis_line, y_axis-57-45-25+2-15, x_axis_line + 130, y_axis-57-45-25+2-15)
        mycanvas.drawString(x_axis_line,y_axis-67-45-7-25+2-5-10,"1. The bills should be settled within 15 days of the receipt of bill.")
        mycanvas.drawString(x_axis_line,y_axis-77-50-9-25+2-10,"2. Any difference in the bill should be informed immediately to Zonal DM. " + str(zone_num_dict[business_zone]) +",")
        mycanvas.drawString(x_axis_line,y_axis-87-55-11-25+2-5-5,"    or to this office 0422-2607971,0422-2544777, aavincbemkg@gmail.com")
        mycanvas.drawString(x_axis_line,y_axis-97-60-13-25+2-5,"3. Enclose the payment slip alongwith the payment without fail.")
        mycanvas.drawString(x_axis_line,y_axis-107-65-15-25+2-5,"E & OE")

        #Payement Slip double line
        mycanvas.line(10, y_axis-107-65-5-25-20+2, 585, y_axis-107-65-5-25-20+2)
        mycanvas.line(10, y_axis-107-65-5-25-22+2, 585, y_axis-107-65-5-25-22+2)

        mycanvas.drawString(x_axis_line,y_axis-107-65-5-25-42+4,"P  A  Y  M  E  N  T     S  L  I  P     T  0     A  A  V  I  N     -  C  O  I  M  B  A  T  O  R  E")
        mycanvas.line(10, y_axis-107-65-5-25-45+4, 500-80, y_axis-107-65-5-25-45+4)

        mycanvas.setFont('Helvetica', 11)
        mycanvas.drawString(x_axis_line,y_axis-107-65-5-25-45-15+4,"Code                                         :     " +str(business_code))
        # mycanvas.drawString(x_axis_line,y_axis-107-65-5-25-45-30+4,"Name of the institution             :     "+str(business_name))
        # route = RouteBusinessMap.objects.filter(business__code=business_code, route__session_id=1)[0].route.name[:-4]
        mycanvas.drawString(x_axis_line,y_axis-107-65-5-25-45-30+4,"Name of the institution             :     "+str(business_name))
        mycanvas.drawString(x_axis_line,y_axis-107-65-5-25-45-45+4,"Bill No/Month/Amount              :     "+str(bill_no_of)+" / "+str(datetime.strftime(from_date, '%b-%Y'))+ " / "+"Rs. "+str(round(total_cost)))
        mycanvas.drawString(x_axis_line,y_axis-107-65-5-25-45-60+4,"Details of Payment                    :      "+"Ch.No.: "+"                 "+"Date :")
        mycanvas.setFont('Helvetica', 11)
    #     mycanvas.drawString(x_axis_line,y_axis-107-65-5-25-45-75+4,str(datetime.strftime(datetime.now(timezone('Asia/Kolkata')), '%d/%m/%Y - %I:%M %p')))
        mycanvas.showPage()

mycanvas.save() 

In [55]:
BankTransaction.objects.all().delete()
BankTransactionMaster.objects.all().delete()
BankTransactionCarryForward.objects.all().delete()

(0, {'transaction.BankTransactionCarryForward': 0})

In [126]:
PaymentRequestResponse.objects.filter(payment_request__time_created__date__range=['2020-08-01', '2020-08-31'], status_id=1).count()

1225

In [6]:
from_date = '2020-08-1'
to_date = '2020-08-31'


bank_transaction_payment_responce_obj = BankTransactionPaymentResponseMap.objects.filter(payment_response__payment_request__time_created__date__range=[from_date, to_date])
payment_responce_obj = PaymentRequestResponse.objects.filter(payment_request__time_created__date__range=[from_date, to_date], status_id=1)

bank_transaction_rid_list = list(bank_transaction_payment_responce_obj.values_list('payment_response__rid', flat=True))                     
not_included_payment_responce_rid_list = list(payment_responce_obj.exclude(rid__in=bank_transaction_rid_list).values_list('id', 'rid', 'amt', 'payment_request__payment_request_for__name', 'status__name', 'tet', 'trn', 'crn', 'payment_request__time_created', 'time_created'))
not_included_payment_responce_rid_columns = ['ID', 'RID', 'Amount', 'Order For', 'Status', 'Transaction Time', 'Transaction Id', 'crn', 'Request Time', 'Responce Time']
payment_response_df = pd.DataFrame(not_included_payment_responce_rid_list, columns=not_included_payment_responce_rid_columns)

rid_list = (payment_response_df['RID'])

In [7]:
PaymentRequest.objects.filter(time_created__date__range=['2020-08-1', '2020-08-31']).count()

2381

In [ ]:
miss_list = ['2093000102', '2093000103', '2093000104', '2093000105', '2093000108', '2093000109', '2093000110', '2093000111', '2093000112', '2093000113', '2093000114', '2093000115', '2093000122', '2093000116', '2093000123', '2093000125', '2093000121', '2093000129', '2093000128', '2093000138', '2093000133', '2093000134', '2093000136', '2093000132', '2093000137', '2093000147', '2093000142', '2093000150', '2093000149', '2093000148', '2093000159', '2093000154', '2093000156', '2093000152', '2093000153', '2093000155', '2093000158', '2093000160', '2093000161', '2093000162', '2093000163', '2093000165', '2093000164', '2093000167', '2093000166', '2093000172', '2093000170', '2093000171', '2093000173', '2093000177', '2093000174', '2093000175', '2093000176', '2092800062', '2093000180', '2092400147', '2092800022', '2093000157', '2093000193', '2092800023', '2092800067', '2092900013', '2092800122', '2092900006', '2092800081', '2092800097', '2092800133', '2092800135', '2092900002', '2092900001', '2092900014', '2092900004', '2092900010', '2092900015', '2092900018', '2092900023', '2092900034', '2092900029', '2092900019', '2092900035', '2092900038', '2092900030', '2092900036', '2092900031', '2092900032', '2092900039', '2092900041', '2092900055', '2092900046', '2092900054', '2092900050', '2092900058', '2092900073', '2092900059', '2092900077', '2092900065', '2092900074', '2092900069', '2092900081', '2092900099', '2092900082', '2092900091', '2092900083', '2092900085', '2092900100', '2092900087', '2092900092', '2092900101', '2092900093', '2092900088', '2092900103', '2092900089', '2092900094', '2092900104', '2092900096', '2092900105', '2092900115', '2092900106', '2092900110', '2092900116', '2092900111', '2092900107', '2092900117', '2092900108', '2092900112', '2092900118', '2092900113', '2092900123', '2092900119', '2092900124', '2092900114', '2092900121', '2092900133', '2092900127', '2092900143', '2092900136', '2092900134', '2092900145', '2092900152', '2092900139', '2092900148', '2092900140', '2092900156', '2092900165', '2092900159', '2092900141', '2092900166', '2093000013', '2093000001', '2092900167', '2093000020', '2093000014', '2093000035', '2093000015', '2093000026', '2093000038', '2093000028', '2093000039', '2093000057', '2093000051', '2093000045', '2093000065', '2093000046', '2093000052', '2093000071', '2093000053', '2093000050', '2093000083', '2093000081', '2093000082', '2093000047', '2093000084', '2093000055', '2093000087', '2093000056', '2093000085', '2093000094', '2093000092', '2093000086', '2093000093', '2093000095', '2093000097', '2093000098', '2093000099', '2093000100']

In [36]:
payment_log_list = ['2083100113', '2083100110', '2083100109', '2083100108', '2083100106', '2083100101', '2083100099', '2083100098', '2083100097', '2083100096', '2083100093', '2083100092', '2083100090', '2083100089', '2083100088', '2083100087', '2083100086', '2083100085', '2083100084', '2083100083', '2083100081', '2083100080', '2083100078', '2083100074', '2083100073', '2083100069', '2083100068', '2083100067', '2083100066', '2083100065', '2083100064', '2083100063', '2083100062', '2083100061', '2083100056', '2083100053', '2083100051', '2083100050', '2083100049', '2083100048', '2083100047', '2083100046', '2083100043', '2083100040', '2083100038', '2083100035', '2083100033', '2083100031', '2083100030', '2083100029', '2083100027', '2083100025', '2083100021', '2083100017', '2083100015', '2083100014', '2083100009', '2083100008', '2083100007', '2083100005', '2083100002', '2083000098', '2083000095', '2083000094', '2083000093', '2083000092', '2083000087', '2083000086', '2083000085', '2083000084', '2083000083', '2083000082', '2083000081', '2083000080', '2083000079', '2083000078', '2083000077', '2083000076', '2083000075', '2083000074', '2083000073', '2083000070', '2083000069', '2083000068', '2083000067', '2083000063', '2083000062', '2083000061', '2083000060', '2083000059', '2083000058', '2083000057', '2083000056', '2083000054', '2083000053', '2083000051', '2083000050', '2083000049', '2083000048', '2083000047', '2083000046', '2083000045', '2083000044', '2083000043', '2083000041', '2083000040', '2083000039', '2083000038', '2083000037', '2083000036', '2083000035', '2083000034', '2083000033', '2083000032', '2083000027', '2083000026', '2083000024', '2083000022', '2083000020', '2083000016', '2083000015', '2083000014', '2083000011', '2083000009', '2083000007', '2083000006', '2083000005', '2082900091', '2082900089', '2082900087', '2082900086', '2082900085', '2082900081', '2082900080', '2082900078', '2082900077', '2082900076', '2082900074', '2082900073', '2082900072', '2082900069', '2082900068', '2082900067', '2082900066', '2082900064', '2082900060', '2082900057', '2082900056', '2082900054', '2082900052', '2082900051', '2082900050', '2082900049', '2082900048', '2082900047', '2082900046', '2082900045', '2082900044', '2082900043', '2082900042', '2082900041', '2082900040', '2082900038', '2082900036', '2082900034', '2082900032', '2082900029', '2082900026', '2082900024', '2082900023', '2082900020', '2082900019', '2082900018', '2082900017', '2082900016', '2082900015', '2082900009', '2082900008', '2082900007', '2082900005', '2082900004', '2082900003', '2082900002', '2082900001', '2082800127', '2082800126', '2082800125', '2082800123', '2082800122', '2082800120', '2082800119', '2082800118', '2082800117', '2082800116', '2082800115', '2082800114', '2082800113', '2082800111', '2082800110', '2082800109', '2082800107', '2082800106', '2082800105', '2082800104', '2082800103', '2082800102', '2082800101', '2082800100', '2082800099', '2082800098', '2082800097', '2082800096', '2082800095', '2082800093', '2082800090', '2082800089', '2082800088', '2082800087', '2082800086', '2082800085', '2082800084', '2082800083', '2082800082', '2082800081', '2082800080', '2082800079', '2082800078', '2082800077', '2082800074', '2082800073', '2082800071', '2082800070', '2082800069', '2082800066', '2082800065', '2082800063', '2082800062', '2082800061', '2082800060', '2082800059', '2082800058', '2082800056', '2082800054', '2082800052', '2082800051', '2082800050', '2082800049', '2082800047', '2082800046', '2082800045', '2082800043', '2082800042', '2082800037', '2082800033', '2082800028', '2082800026', '2082800025', '2082800022', '2082800021', '2082800007', '2082800006', '2082800004', '2082800003', '2082800002', '2082800001', '2082700176', '2082700175', '2082700173', '2082700172', '2082700171', '2082700170', '2082700169', '2082700168', '2082700167', '2082700166', '2082700163', '2082700161', '2082700160', '2082700159', '2082700158', '2082700155', '2082700154', '2082700151', '2082700150', '2082700149', '2082700148', '2082700146', '2082700145', '2082700143', '2082700142', '2082700141', '2082700140', '2082700138', '2082700137', '2082700132', '2082700130', '2082700129', '2082700127', '2082700124', '2082700122', '2082700121', '2082700120', '2082700119', '2082700118', '2082700117', '2082700116', '2082700115', '2082700112', '2082700111', '2082700110', '2082700109', '2082700108', '2082700105', '2082700102', '2082700061', '2082700060', '2082700059', '2082700057', '2082700056', '2082700055', '2082700054', '2082700052', '2082700050', '2082700042', '2082700040', '2082700038', '2082700018', '2082700017', '2082700009', '2082700008', '2082700007', '2082700006', '2082700004', '2082700003', '2082700001', '2082600124', '2082600121', '2082600119', '2082600114', '2082600107', '2082600104', '2082600102', '2082600098', '2082600097', '2082600096', '2082600095', '2082600094', '2082600091', '2082600090', '2082600089', '2082600088', '2082600087', '2082600086', '2082600083', '2082600081', '2082600077', '2082600075', '2082600074', '2082600073', '2082600072', '2082600070', '2082600069', '2082600068', '2082600067', '2082600066', '2082600065', '2082600064', '2082600063', '2082600062', '2082600061', '2082600060', '2082600058', '2082600057', '2082600054', '2082600052', '2082600049', '2082600048', '2082600047', '2082600046', '2082600045', '2082600042', '2082600041', '2082600040', '2082600038', '2082600037', '2082600036', '2082600035', '2082600034', '2082600033', '2082600031', '2082600030', '2082600027', '2082600026', '2082600017', '2082600016', '2082600015', '2082600014', '2082600010', '2082600005', '2082600002', '2082600001', '2082500105', '2082500103', '2082500102', '2082500101', '2082500100', '2082500099', '2082500097', '2082500095', '2082500094', '2082500091', '2082500090', '2082500089', '2082500087', '2082500086', '2082500085', '2082500084', '2082500083', '2082500082', '2082500081', '2082500080', '2082500079', '2082500078', '2082500076', '2082500074', '2082500072', '2082500066', '2082500065', '2082500061', '2082500060', '2082500058', '2082500056', '2082500054', '2082500051', '2082500050', '2082500049', '2082500048', '2082500047', '2082500046', '2082500044', '2082500043', '2082500042', '2082500041', '2082500040', '2082500039', '2082500038', '2082500037', '2082500034', '2082500029', '2082500027', '2082500026', '2082500024', '2082500023', '2082500020', '2082500018', '2082500017', '2082500016', '2082500013', '2082500012', '2082500010', '2082500009', '2082500006', '2082500005', '2082500004', '2082500003', '2082500002', '2082500001', '2082400103', '2082400101', '2082400099', '2082400096', '2082400087', '2082400086', '2082400084', '2082400083', '2082400082', '2082400079', '2082400078', '2082400075', '2082400073', '2082400072', '2082400070', '2082400069', '2082400067', '2082400065', '2082400064', '2082400063', '2082400062', '2082400061', '2082400060', '2082400059', '2082400058', '2082400057', '2082400056', '2082400052', '2082400050', '2082400049', '2082400048', '2082400047', '2082400046', '2082400045', '2082400041', '2082400039', '2082400038', '2082400037', '2082400034', '2082400032', '2082400030', '2082400021', '2082400020', '2082400018', '2082400016', '2082400012', '2082400009', '2082400008', '2082400007', '2082400005', '2082400004', '2082400003', '2082400002', '2082400001', '2082300055', '2082300054', '2082300052', '2082300051', '2082300050', '2082300047', '2082300046', '2082300041', '2082300040', '2082300039', '2082300037', '2082300035', '2082300034', '2082300033', '2082300032', '2082300031', '2082300029', '2082300028', '2082300026', '2082300024', '2082300022', '2082300020', '2082300019', '2082300018', '2082300017', '2082300016', '2082300015', '2082300014', '2082300013', '2082300012', '2082300010', '2082300009', '2082300008', '2082300007', '2082300006', '2082300004', '2082300003', '2082300002', '2082200076', '2082200074', '2082200072', '2082200071', '2082200070', '2082200069', '2082200065', '2082200064', '2082200060', '2082200059', '2082200058', '2082200056', '2082200055', '2082200054', '2082200051', '2082200050', '2082200049', '2082200048', '2082200047', '2082200046', '2082200044', '2082200043', '2082200042', '2082200041', '2082200040', '2082200039', '2082200038', '2082200037', '2082200036', '2082200035', '2082200034', '2082200033', '2082200031', '2082200030', '2082200029', '2082200028', '2082200027', '2082200026', '2082200025', '2082200024', '2082200023', '2082200018', '2082200016', '2082200014', '2082200012', '2082200011', '2082200009', '2082200003', '2082200002', '2082100068', '2082100067', '2082100066', '2082100065', '2082100063', '2082100062', '2082100061', '2082100060', '2082100059', '2082100058', '2082100055', '2082100053', '2082100052', '2082100051', '2082100050', '2082100049', '2082100048', '2082100047', '2082100046', '2082100045', '2082100044', '2082100043', '2082100042', '2082100040', '2082100039', '2082100038', '2082100037', '2082100036', '2082100034', '2082100031', '2082100030', '2082100029', '2082100028', '2082100027', '2082100025', '2082100023', '2082100020', '2082100019', '2082100018', '2082100017', '2082100016', '2082100015', '2082100013', '2082100005', '2082100002', '2082100001', '2082000072', '2082000066', '2082000064', '2082000063', '2082000056', '2082000055', '2082000054', '2082000052', '2082000051', '2082000049', '2082000048', '2082000045', '2082000044', '2082000042', '2082000041', '2082000039', '2082000037', '2082000036', '2082000035', '2082000034', '2082000033', '2082000032', '2082000028', '2082000025', '2082000024', '2082000023', '2082000022', '2082000021', '2082000020', '2082000019', '2082000017', '2082000016', '2082000015', '2082000012', '2082000011', '2082000006', '2082000004', '2082000003', '2082000001', '2081900079', '2081900071', '2081900065', '2081900064', '2081900059', '2081900058', '2081900057', '2081900056', '2081900054', '2081900053', '2081900052', '2081900051', '2081900050', '2081900049', '2081900048', '2081900047', '2081900045', '2081900043', '2081900040', '2081900039', '2081900038', '2081900037', '2081900036', '2081900035', '2081900031', '2081900029', '2081900028', '2081900027', '2081900025', '2081900024', '2081900021', '2081900019', '2081900015', '2081900014', '2081900013', '2081900012', '2081900009', '2081900008', '2081900006', '2081900005', '2081900004', '2081900003', '2081900002', '2081800097', '2081800096', '2081800093', '2081800091', '2081800090', '2081800089', '2081800086', '2081800083', '2081800082', '2081800080', '2081800079', '2081800078', '2081800076', '2081800074', '2081800073', '2081800072', '2081800070', '2081800069', '2081800068', '2081800066', '2081800065', '2081800063', '2081800062', '2081800061', '2081800058', '2081800057', '2081800055', '2081800052', '2081800050', '2081800049', '2081800048', '2081800046', '2081800045', '2081800044', '2081800041', '2081800040', '2081800034', '2081800033', '2081800031', '2081800030', '2081800026', '2081800023', '2081800021', '2081800014', '2081800012', '2081800010', '2081800009', '2081800008', '2081800007', '2081800003', '2081800002', '2081700089', '2081700085', '2081700081', '2081700080', '2081700077', '2081700076', '2081700075', '2081700071', '2081700070', '2081700069', '2081700068', '2081700067', '2081700066', '2081700065', '2081700064', '2081700063', '2081700062', '2081700060', '2081700059', '2081700057', '2081700056', '2081700055', '2081700054', '2081700053', '2081700049', '2081700047', '2081700046', '2081700045', '2081700040', '2081700037', '2081700036', '2081700035', '2081700033', '2081700031', '2081700028', '2081700027', '2081700026', '2081700025', '2081700024', '2081700023', '2081700020', '2081700019', '2081700012', '2081700011', '2081700001', '2081600068', '2081600064', '2081600063', '2081600061', '2081600060', '2081600059', '2081600058', '2081600056', '2081600055', '2081600052', '2081600051', '2081600050', '2081600048', '2081600047', '2081600046', '2081600044', '2081600042', '2081600040', '2081600039', '2081600036', '2081600034', '2081600033', '2081600032', '2081600031', '2081600028', '2081600027', '2081600026', '2081600024', '2081600018', '2081600016', '2081600015', '2081600013', '2081600011', '2081600007', '2081600006', '2081600002', '2081500126', '2081500125', '2081500120', '2081500117', '2081500115', '2081500114', '2081500113', '2081500110', '2081500109', '2081500108', '2081500107', '2081500106', '2081500105', '2081500102', '2081500101', '2081500100', '2081500099', '2081500098', '2081500097', '2081500096', '2081500095', '2081500094', '2081500093', '2081500092', '2081500091', '2081500087', '2081500086', '2081500085', '2081500084', '2081500083', '2081500081', '2081500079', '2081500078', '2081500074', '2081500051', '2081500048', '2081500043', '2081500032', '2081500028', '2081500027', '2081500026', '2081500022', '2081500021', '2081500017', '2081500016', '2081500008', '2081500006', '2081500004', '2081500003', '2081500002', '2081500001', '2081400059', '2081400058', '2081400057', '2081400056', '2081400053', '2081400052', '2081400049', '2081400048', '2081400047', '2081400046', '2081400045', '2081400044', '2081400043', '2081400041', '2081400036', '2081400034', '2081400033', '2081400032', '2081400029', '2081400024', '2081400022', '2081400019', '2081400017', '2081400016', '2081400014', '2081400012', '2081400009', '2081400008', '2081400004', '2081300053', '2081300052', '2081300051', '2081300049', '2081300048', '2081300047', '2081300046', '2081300045', '2081300044', '2081300043', '2081300042', '2081300041', '2081300040', '2081300038', '2081300037', '2081300036', '2081300034', '2081300030', '2081300028', '2081300027', '2081300022', '2081300021', '2081300013', '2081300011', '2081300010', '2081300009', '2081300007', '2081300005', '2081300004', '2081300003', '2081200061', '2081200059', '2081200056', '2081200054', '2081200053', '2081200052', '2081200050', '2081200049', '2081200048', '2081200047', '2081200046', '2081200045', '2081200043', '2081200040', '2081200039', '2081200037', '2081200036', '2081200033', '2081200032', '2081200031', '2081200030', '2081200028', '2081200024', '2081200023', '2081200022', '2081200021', '2081200017', '2081200011', '2081200009', '2081200008', '2081200007', '2081200006', '2081100068', '2081100066', '2081100063', '2081100062', '2081100061', '2081100060', '2081100059', '2081100055', '2081100054', '2081100052', '2081100051', '2081100048', '2081100047', '2081100043', '2081100039', '2081100036', '2081100035', '2081100032', '2081100031', '2081100030', '2081100026', '2081100024', '2081100023', '2081100022', '2081100020', '2081100013', '2081100011', '2081100010', '2081100008', '2081100006', '2081100003', '2081100002', '2081100001', '2081000061', '2081000060', '2081000056', '2081000054', '2081000053', '2081000052', '2081000050', '2081000049', '2081000047', '2081000046', '2081000045', '2081000043', '2081000038', '2081000036', '2081000034', '2081000029', '2081000028', '2081000027', '2081000024', '2081000021', '2081000017', '2081000016', '2081000015', '2081000013', '2081000011', '2081000003', '208900060', '208900057', '208900056', '208900054', '208900052', '208900051', '208900050', '208900049', '208900048', '208900047', '208900046', '208900044', '208900042', '208900041', '208900040', '208900039', '208900037', '208900034', '208900033', '208900029', '208900024', '208900013', '208900009', '208900008', '208900007', '208900006', '208800068', '208800067', '208800065', '208800064', '208800063', '208800061', '208800060', '208800059', '208800057', '208800052', '208800051', '208800050', '208800047', '208800046', '208800045', '208800044', '208800040', '208800039', '208800038', '208800037', '208800033', '208800032', '208800031', '208800024', '208800022', '208800020', '208800019', '208800016', '208800006', '208800004', '208800003', '208700045', '208700043', '208700042', '208700041', '208700040', '208700038', '208700037', '208700036', '208700034', '208700031', '208700029', '208700028', '208700027', '208700026', '208700025', '208700023', '208700021', '208700020', '208700019', '208700018', '208700016', '208700013', '208700012', '208700008', '208700007', '208700006', '208700003', '208700002', '208700001', '208600048', '208600047', '208600046', '208600044', '208600043', '208600042', '208600041', '208600038', '208600036', '208600035', '208600034', '208600028', '208600026', '208600022', '208600020', '208600008', '208600006', '208600005', '208500025', '208500023', '208500022', '208500017', '208500015', '208500014', '208500013', '208500012', '208500011', '208500010', '208500009', '208500008', '208500006', '208500004', '208500003', '208500001', '208400044', '208400040', '208400034', '208400030', '208400029', '208400028', '208400024', '208400021', '208400020', '208400019', '208400008', '208400007', '208400006', '208400005', '208300035', '208300027', '208300026', '208300022', '208300021', '208300018', '208300016', '208300015', '208300012', '208300005', '208300004', '208200015', '208200012', '208200011', '208200010', '208200004', '208200002', '208100007', '208100006', '208100003', '208100002', '208100001']
len(payment_log_list)

1225

In [38]:
payment_match_list = ['208100001', '208100002', '208100003', '208100006', '208100007', '208200002', '208200004', '208200010', '208200011', '208200012', '208200015', '208300004', '208300012', '208300015', '208300016', '208300018', '208300021', '208300022', '208300026', '208300027', '208300035', '208400005', '208400006', '208400007', '208400008', '208400019', '208400020', '208400021', '208400024', '208400028', '208400029', '208400030', '208400034', '208400040', '208400044', '208500001', '208500003', '208500004', '208500006', '208500008', '208500009', '208500010', '208500011', '208500012', '208500013', '208500014', '208500015', '208500017', '208500022', '208500023', '208500025', '208600005', '208600006', '208600008', '208600020', '208600022', '208600026', '208600028', '208600034', '208600035', '208600036', '208600038', '208600041', '208600042', '208600044', '208600043', '208600046', '208600047', '208600048', '208700001', '208700002', '208700003', '208700006', '208700007', '208700008', '208700012', '208700013', '208700016', '208700018', '208700019', '208700020', '208700021', '208700023', '208700026', '208700025', '208700027', '208700028', '208700029', '208700031', '208700034', '208700036', '208700037', '208700038', '208700040', '208700041', '208700042', '208700043', '208700045', '208800003', '208800004', '208800006', '208800016', '208800019', '208800020', '208800022', '208800024', '208800031', '208800032', '208800033', '208800037', '208800038', '208800039', '208800040', '208800044', '208800045', '208800046', '208800047', '208800050', '208800051', '208800052', '208800057', '208800059', '208800060', '208800061', '208800065', '208800067', '208800068', '208900006', '208900007', '208900008', '208900009', '208900013', '208900024', '208900029', '208900033', '208900034', '208900037', '208900039', '208900040', '208900041', '208900042', '208900046', '208900047', '208900048', '208900049', '208900050', '208900051', '208900052', '208900054', '208900056', '208900057', '208900060', '2081000003', '2081000011', '2081000013', '2081000015', '2081000016', '2081000017', '2081000021', '2081000024', '2081000029', '2081000027', '2081000028', '2081000034', '2081000038', '2081000043', '2081000045', '2081000046', '2081000047', '2081000049', '2081000050', '2081000052', '2081000053', '2081000056', '2081000054', '2081000060', '2081000061', '2081100001', '2081100002', '2081100003', '2081100006', '2081100008', '2081100010', '2081100011', '2081100013', '2081100020', '2081100023', '2081100022', '2081100024', '2081100026', '2081100030', '2081100032', '2081100035', '2081100036', '2081100039', '2081100043', '2081100047', '2081100048', '2081100051', '2081100052', '2081100055', '2081100054', '2081100059', '2081100061', '2081100062', '2081100063', '2081100066', '2081100068', '2081200006', '2081200007', '2081200008', '2081200009', '2081200011', '2081200017', '2081200021', '2081200022', '2081200023', '2081200024', '2081200028', '2081200030', '2081200031', '2081200032', '2081200033', '2081200036', '2081200037', '2081200039', '2081200040', '2081200043', '2081200046', '2081200045', '2081200047', '2081200049', '2081200050', '2081200052', '2081200054', '2081200053', '2081200056', '2081200059', '2081200061', '2081300003', '2081300004', '2081300005', '2081300007', '2081300009', '2081300010', '2081300011', '2081300013', '2081300021', '2081300022', '2081300027', '2081300028', '2081300034', '2081300036', '2081300037', '2081300038', '2081300040', '2081300041', '2081300042', '2081300043', '2082000063', '2082000064', '2082000066', '2082000072', '2082100001', '2082100002', '2082100005', '2082100013', '2082100015', '2082100016', '2082100017', '2082100018', '2082100019', '2082100020', '2082100023', '2082100025', '2082100027', '2082100028', '2082100029', '2082100031', '2082100030', '2082100034', '2082100036', '2082100037', '2082100038', '2082100039', '2082100040', '2082100042', '2082100043', '2082100044', '2082100045', '2082100048', '2082100047', '2082100046', '2082100049', '2082100050', '2082100051', '2082100053', '2082100055', '2082100058', '2082100059', '2082100060', '2082100062', '2082100063', '2082100061', '2082100065', '2082100066', '2082100067', '2082100068', '2082200009', '2082200012', '2082200016', '2082200023', '2082200024', '2082200029', '2082200030', '2082200031', '2082200033', '2082200034', '2082200035', '2082200036', '2082200037', '2082200038', '2082200039', '2082200040', '2082200041', '2082200042', '2082200043', '2082200044', '2082200046', '2082200047', '2082200048', '2082200049', '2082200050', '2082200051', '2082200054', '2082200055', '2082200056', '2082200059', '2082200058', '2082200060', '2082200064', '2082200065', '2082200070', '2082200069', '2082200071', '2082200072', '2082200074', '2082200076', '2082300002', '2082300003', '2082300004', '2082300006', '2082300007', '2082300008', '2082300009', '2082300010', '2082300012', '2082300013', '2082300014', '2082300015', '2082300016', '2082300017', '2082300018', '2082300019', '2082300020', '2081300044', '2082300022', '2082300024', '2081300045', '2082300026', '2081300046', '2081300047', '2082300028', '2081300048', '2082300029', '2081300049', '2081300051', '2081300052', '2082300031', '2081300053', '2081400004', '2081400008', '2082300032', '2081400009', '2082300033', '2081400012', '2082300034', '2081400014', '2082300035', '2081400016', '2081400017', '2082300037', '2081400019', '2081400022', '2081400024', '2081400029', '2082300039', '2081400032', '2081400033', '2081400034', '2082300040', '2081400036', '2081400041', '2082300041', '2081400044', '2082300046', '2082300047', '2081400045', '2081400043', '2081400046', '2082300050', '2081400047', '2082300051', '2081400048', '2081400049', '2082300052', '2081400052', '2082300054', '2081400053', '2082300055', '2081400056', '2082400002', '2081400057', '2081400058', '2082400001', '2081400059', '2082400003', '2081500001', '2081500002', '2082400004', '2081500003', '2082400005', '2081500004', '2082400007', '2081500006', '2081500008', '2082400008', '2081500016', '2081500017', '2081500021', '2081500022', '2081500026', '2082400009', '2081500027', '2082400012', '2081500028', '2081500032', '2082400016', '2081500043', '2081500048', '2081500051', '2082400018', '2081500074', '2081500078', '2082400020', '2081500079', '2082400021', '2081500081', '2081500083', '2082400030', '2081500084', '2082400032', '2081500085', '2081500086', '2082400034', '2081500087', '2082400037', '2081500092', '2081500091', '2082400038', '2081500094', '2081500093', '2082400039', '2081500095', '2081500096', '2082400041', '2081500097', '2082400045', '2081500099', '2081500098', '2081500100', '2081500101', '2081500102', '2082400046', '2081500105', '2082400047', '2081500107', '2081500108', '2082400048', '2081500106', '2081500110', '2082400049', '2081500109', '2082400050', '2081500113', '2081500114', '2082400052', '2081500115', '2081500117', '2082400057', '2081500120', '2082400056', '2081500125', '2082400058', '2081500126', '2082400059', '2081600002', '2081600006', '2082400060', '2081600007', '2081600011', '2082400061', '2081600013', '2082400062', '2081600015', '2081600016', '2082400063', '2081600018', '2082400065', '2081600024', '2081600026', '2082400067', '2081600027', '2081600028', '2081600031', '2081600032', '2081600033', '2081600034', '2082400064', '2081600036', '2082400069', '2081600040', '2081600039', '2082400070', '2081600042', '2082400073', '2081600047', '2081600046', '2082400072', '2081600048', '2082400075', '2081600050', '2081600051', '2082400078', '2081600052', '2081600055', '2082400082', '2081600056', '2082400079', '2081600058', '2081600060', '2082400083', '2081600059', '2081600063', '2082400086', '2081600061', '2081600064', '2082400084', '2082400087', '2081600068', '2081700001', '2082400096', '2081700011', '2082400099', '2081700012', '2082400101', '2081700020', '2082400103', '2081700019', '2081700023', '2081700024', '2081700025', '2081700026', '2082500001', '2081700027', '2081700028', '2081700031', '2082500002', '2081700033', '2082500003', '2081700035', '2081700036', '2082500004', '2081700037', '2082500005', '2081700040', '2082500006', '2081700045', '2082500009', '2081700046', '2081700047', '2082500010', '2081700049', '2082500012', '2081700053', '2081700054', '2082500013', '2081700055', '2082500016', '2081700056', '2082500017', '2081700057', '2082500018', '2081700059', '2081700060', '2082500020', '2081700064', '2081700063', '2082500023', '2081700065', '2082500024', '2081700066', '2081700067', '2081700068', '2081700069', '2082500026', '2081700070', '2082500027', '2081700075', '2082500029', '2081700077', '2082500034', '2081700076', '2082500037', '2081700080', '2081700081', '2082500038', '2081700085', '2082500039', '2081700089', '2082500040', '2081800002', '2081800003', '2082500041', '2081800007', '2082500042', '2081800008', '2081800009', '2082500043', '2081800010', '2082500044', '2081800012', '2082500046', '2081800014', '2082500047', '2081800021', '2081800023', '2082500048', '2081800026', '2082500049', '2081800030', '2082500050', '2081800031', '2081800033', '2082500051', '2081800034', '2081800040', '2081800041', '2082500054', '2081800044', '2082500056', '2081800045', '2082500058', '2081800046', '2082500060', '2081800048', '2082500061', '2081800049', '2081800050', '2082500065', '2081800052', '2082500066', '2081800055', '2081800057', '2082500072', '2081800058', '2082500074', '2081800061', '2082500076', '2081800062', '2081800063', '2082500078', '2081800065', '2082500079', '2081800066', '2081800068', '2081800069', '2082500080', '2081800070', '2082500081', '2081800072', '2081800073', '2082500082', '2081800074', '2081800076', '2082500083', '2082500084', '2081800078', '2081800079', '2081800080', '2081800082', '2081800083', '2082500085', '2081800086', '2082500086', '2082500087', '2081800089', '2082500089', '2081800090', '2082500090', '2081800091', '2082500094', '2081800093', '2082500095', '2081800096', '2081800097', '2082500097', '2081900002', '2082500099', '2081900003', '2081900004', '2082500100', '2081900005', '2081900006', '2082500101', '2081900008', '2082500102', '2081900009', '2081900012', '2082500103', '2081900013', '2081900014', '2082500105', '2081900015', '2082600001', '2081900019', '2081900021', '2082600002', '2081900024', '2081900025', '2082600005', '2081900027', '2081900028', '2081900029', '2082600010', '2081900035', '2082600014', '2081900036', '2082600015', '2081900037', '2082600017', '2082600026', '2081900038', '2081900039', '2082600027', '2081900040', '2082600030', '2081900043', '2081900045', '2082600031', '2081900047', '2081900048', '2082600033', '2081900049', '2081900050', '2082600034', '2082600035', '2081900051', '2081900052', '2082600036', '2081900053', '2082600037', '2081900054', '2081900057', '2082600038', '2081900056', '2081900058', '2081900059', '2082600040', '2081900065', '2081900064', '2082600041', '2081900071', '2082600042', '2081900079', '2082000001', '2082000003', '2082000004', '2082000006', '2082600045', '2082000011', '2082600046', '2082000012', '2082600047', '2082600048', '2082000015', '2082600049', '2082000016', '2082000017', '2082600052', '2082000019', '2082600054', '2082000020', '2082000021', '2082600057', '2082000022', '2082000024', '2082600058', '2082000023', '2082600060', '2082000025', '2082000028', '2082600061', '2082000032', '2082600062', '2082000033', '2082000034', '2082600063', '2082000036', '2082000037', '2082600064', '2082000042', '2082600065', '2082000041', '2082000044', '2082600066', '2082000039', '2082000045', '2082600067', '2082000049', '2082000048', '2082000051', '2082000052', '2082600068', '2082600069', '2082000054', '2082600070', '2082000055', '2082600072', '2082000056', '2082600073', '2082600074', '2082600075', '2082600077', '2082600081', '2082600083', '2082600104', '2082600086', '2082600107', '2082600088', '2082600114', '2082600119', '2082600087', '2082600121', '2082600089', '2082600124', '2082600090', '2082700001', '2082600091', '2082700003', '2082700004', '2082600094', '2082600096', '2082700006', '2082700007', '2082600095', '2082600097', '2082700008', '2082700009', '2082600098', '2082600102', '2082700017', '2082700018', '2082700038', '2082700040', '2083100083', '2082700042', '2082700050', '2082700052', '2082700054', '2082700056', '2082700057', '2082700059', '2082700060', '2082700061', '2082700102', '2083100086', '2082700105', '2082700108', '2082700110', '2083100088', '2082700111', '2082700112', '2082700115', '2083100089', '2082700116', '2082700117', '2082700118', '2083100087', '2082700119', '2082700120', '2083100090', '2082700121', '2082700122', '2082700124', '2083100092', '2082700127', '2082700130', '2083100093', '2082700129', '2082700132', '2083100096', '2082700137', '2083100097', '2082700138', '2082700140', '2082700141', '2082700143', '2083100098', '2082700146', '2083100099', '2082700145', '2083100101', '2082700149', '2082700148', '2083100106', '2082700150', '2082700151', '2083100108', '2082700154', '2083100109', '2082700155', '2082700158', '2083100110', '2082700159', '2082700160', '2083100113', '2082700161', '208800064', '2082700163', '2082700166', '2082700168', '2082700167', '2082700169', '2082700170', '2082700171', '2082700172', '2082700173', '2082700175', '2082700176', '2082800002', '2082800001', '2082800003', '2082800004', '2082800006', '2082800021', '2082800022', '2082800025', '2082800026', '2082800028', '2082800033', '2082800042', '2082800043', '2082800045', '2082800046', '2082800047', '2082800049', '2082800050', '2082800051', '2082800052', '2082800054', '2082800058', '2082800056', '2082800059', '2082800060', '2082800061', '2082800063', '2082800062', '2082800065', '2082800066', '2082800069', '2082800070', '2082800071', '2082800073', '2082800074', '2082800078', '2082800077', '2082800079', '2082800080', '2082800081', '2082800083', '2082800082', '2082800084', '2082800086', '2082800085', '2082800088', '2082800089', '2082800087', '2082800090', '2082800093', '2082800095', '2082800098', '2082800096', '2082800097', '2082800099', '2082800100', '2082800101', '2082800102', '2082800103', '2082800104', '2082800105', '2082800106', '2082800107', '2082800110', '2082800109', '2082800111', '2082800113', '2082800114', '2082800115', '2082800116', '2082800117', '2082800118', '2082800119', '2082800120', '2082800122', '2082800123', '2082800125', '2082800126', '2082800127', '2082900001', '2082900002', '2082900003', '2082900004', '2082900005', '2082900007', '2082900008', '2082900009', '2082900015', '2082900016', '2082900017', '2082900018', '2082900019', '2082900020', '2082900023', '2082900024', '2082900026', '2082900029', '2082900032', '2082900034', '2082900036', '2082900038', '2082900040', '2082900041', '2082900043', '2082900042', '2082900044', '2082900045', '2082900046', '2082900049', '2082900048', '2082900047', '2082900050', '2082900051', '2082900052', '2082900054', '2082900056', '2082900057', '2082900060', '2082900066', '2082900067', '2082900068', '2082900069', '2082900072', '2082900074', '2082900073', '2082900076', '2082900077', '2082900078', '2082900080', '2082900081', '2082900085', '2082900086', '2082900087', '2082900089', '2083000005', '2083000006', '2083000007', '2083000011', '2083000009', '2083000014', '2083000015', '2083000016', '2083000020', '2083000022', '2083000024', '2083000026', '2083000027', '2083000032', '2083000033', '2083000034', '2083000035', '2083000036', '2083000037', '2083000039', '2083000040', '2083000038', '2083000041', '2083000043', '2083000044', '2083000045', '2083000046', '2083000047', '2083000050', '2083000049', '2083000051', '2083000054', '2083000053', '2083000057', '2083000056', '2083000058', '2083000059', '2083000060', '2083000061', '2083000062', '2083000063', '2083000067', '2083000068', '2083000069', '2083000070', '2083000073', '2083000074', '2083000075', '2083000076', '2083000077', '2083000078', '2083000079', '2083000081', '2083000080', '2083000082', '2083000083', '2083000085', '2083000084', '2083000086', '2083000087', '2083000093', '2083000092', '2083000094', '2083000095', '2083000098', '2083100002', '2083100005', '2083100007', '2083100008', '2083100009', '2083100015', '2083100014', '2083100017', '2083100021', '2083100025', '2083100027', '2083100029', '2083100030', '2083100031', '2083100033', '2083100035', '2083100038', '2083100040', '2083100043', '2083100047', '2083100046', '2083100048', '2083100049', '2083100050', '2083100051', '2083100053', '2083100056', '2083100061', '2083100063', '2083100062', '2083100064', '2083100065', '2083100066', '2083100067', '2083100069', '2083100074', '2083100073', '2083100080', '2083100081', '2083100084', '208900044', '2081000036', '2081100031', '2081100060', '2081300030', '2081700062', '2081700071', '2081900031', '2082000035', '208800063', '2081600044', '2082100052', '2082200003', '2082200002', '2082200011', '2082200018', '2082200014', '2082200026', '2082200027', '2082200025', '2082200028', '2082500091', '2082600016', '2082700055', '2082700109', '2082700142', '2082800037', '2081200048', '2082900064', '2082900091', '2083000048', '2083100068', '2083100078', '2083100085']
len(payment_match_list)

1223

In [39]:
df1 = pd.DataFrame(list(PaymentRequestResponse.objects.filter(rid__in=payment_log_list).values('rid', 'amt')))
df2 = pd.DataFrame(list(BankTransaction.objects.filter(rid__in=payment_match_list).values('rid', 'amount')))
merged_df = pd.merge(df1, df2, left_on='rid', right_on='rid', how='outer')
merged_df = merged_df.fillna(0)
merged_df['amount_miss'] = merged_df['amt'] - merged_df['amount']
merged_df[merged_df['amount_miss'] != 0]
# merged_df['amount'].sum()
# merged_df

,rid,amt,amount,amount_miss
820,2082800007,343.00,0,343.00
984,208300005,98.00,0,98.00


In [48]:
9765273.77 - 9767232.770-98+2400

343.0

In [29]:
merged_df['amt_y'].sum()

Decimal('9767330.77')

In [21]:
from_date = '2020-08-1'
to_date = '2020-08-31'
PaymentRequest.objects.get(time_created__date__gte=from_date,time_created__date__lte=to_date, rid='2082400027').amt

Decimal('2400.00')

In [26]:
merged_df[merged_df['rid']=='208300005']

,rid,amt_x,amt_y,amount_miss
984,208300005,98.00,98.00,0.00


In [95]:
def find_out_booth_or_customer_code(string):
    string = str(string)
    index = string.index('b')
    return string[index + 1:]

In [101]:
from_date = '2020-09-1'
to_date = '2020-09-30'

payment_request_obj = PaymentRequest.objects.filter(time_created__date__range=[from_date,to_date]).order_by('-time_created')
payment_request_list = list(
    payment_request_obj.values_list('id', 'rid', 'status__description', 'enquiry_response_status_latest__name',
                                    'time_modified', 'encrypted_string', 'decrypted_string',
                                    'payment_request_for__name', 'crn', 'amt', 'ppi', 'cks', 'is_wallet_selected',
                                    'is_amount_returened_to_wallet', 'wallet_balance_after_this_transaction',
                                    'time_created'))
payment_request_column = ['id', 'rid', 'request_status_name', 'enquiry_response_status_latest',
                          'latest_enquiryed_time', 'request_encrypted_string', 'request_decrypted_string',
                          'payment_request_for', 'crn', 'amt', 'ppi', 'cks', 'is_wallet_selected',
                          'is_amount_returened_to_wallet', 'wallet_balance_after_this_transaction',
                          'request_time_created']
payment_request_df = pd.DataFrame(payment_request_list, columns=payment_request_column)
# responce
payment_responce_obj = PaymentRequestResponse.objects.filter(payment_request__time_created__date__range=[from_date,to_date])
payment_responce_list = list(
    payment_responce_obj.values_list('payment_request', 'status__name', 'is_enquired', 'encrypted_string',
                                     'decrypted_string', 'brn', 'trn', 'tet', 'pmd', 'stc', 'rmk', 'time_created'))
payment_responce_column = ['payment_request', 'responce_status_name', 'is_enquired', 'responce_encrypted_string',
                           'responce_decrypted_string', 'brn', 'trn', 'tet', 'pmd', 'stc', 'rmk',
                           'responce_time_created']
payment_responce_df = pd.DataFrame(payment_responce_list, columns=payment_responce_column)
# requested user
payment_request_user = PaymentRequestUserMap.objects.filter(payment_request_id__in=list(payment_request_df['id']))
payment_request_user_list = list(
    payment_request_user.values_list('payment_request', 'payment_intitated_by', 'payment_intitated_by__first_name',
                                     'payment_intitated_by__last_name'))
payment_request_user_column = ['request_map_request_id', 'user_id', 'first_name', 'last_name']
payment_request_user_df = pd.DataFrame(payment_request_user_list, columns=payment_request_user_column)
# merge user with request table
payment_request_df = pd.merge(payment_request_df, payment_request_user_df, how='outer', left_on='id',
                              right_on='request_map_request_id')

# merge request with responce
final_df = pd.merge(payment_request_df, payment_responce_df, left_on='id', right_on='payment_request', how='outer')
if not final_df.empty:
    final_df['booth_or_customer_code'] = final_df.apply(lambda x: find_out_booth_or_customer_code(x['crn']), axis=1)

final_df = final_df.fillna(0)

# excel part
final_df = final_df.reindex(columns=[
    'first_name', 'last_name', 'booth_or_customer_code', 'amt', 'request_time_created', 
    'payment_request_for', 'responce_status_name', 'responce_time_created', 'enquiry_response_status_latest', 
    'latest_enquiryed_time', 'is_wallet_selected', 'rid', 'crn', 'trn', 'tet', 'brn', 'pmd'
])


final_df=final_df.rename(columns={
    'first_name':'First Name', 'last_name':'Last Name', 'booth_or_customer_code': 'Booth/Customer Code', 'amt': 'Amount', 'request_time_created': 'Requested Time', 
    'payment_request_for': 'Requested For', 'responce_status_name': 'Responce Status', 'responce_time_created': 'Responce Time', 'enquiry_response_status_latest': 'Enquiry Status', 
    'latest_enquiryed_time': 'Enquiry Time', 'is_wallet_selected': 'Is Wallet Used', 'rid': 'RID', 'crn': 'CRN', 'trn': 'Transaction Number', 'tet': 'Transaction Time', 'brn': 'BRN', 'pmd': 'Payment Mode'
})

final_df = final_df[(final_df['Responce Status']== 'Success') | (final_df['Enquiry Status']== 'Success')] 

final_amount_df = final_df.groupby('RID').agg({'Amount': sum}).reset_index()

final_amount_df[final_amount_df['Amount'] == 2400] 


,RID,Amount
639,2091700076,2400.00
1077,2092100231,2400.00


In [93]:
df_merg = pd.merge(df2, final_amount_df, left_on='rid', right_on='RID', how='outer')
df_merg['diff'] = df_merg['amount']-df_merg['Amount']
df_merg = df_merg.fillna(0)
df_merg_diff = df_merg[df_merg['diff'] != 0]

In [94]:
df_merg_diff

,rid,amount,RID,Amount,diff
182,2082600077,15.500,2082600077,98.00,-82.500


In [69]:
rid_list = ['2091400121', '209100007', '209100008', '209100009', '209100010', '209100011', '209100015', '209100019', '209100020', '209100023', '209100027', '209100032', '209100033', '209100034', '209100035', '209100036', '209100039', '209100040', '209100041', '209100042', '209100043', '209100051', '209100053', '209100054', '209100061', '209100059', '209100063', '209100064', '209100065', '209100068', '209100067', '209100069', '209100072', '209100074', '209100076', '209100079', '209100078', '209100081', '209100084', '209100085', '209100086', '209100087', '209100090', '209100091', '209100094', '209100096', '209100093', '209100098', '209100100', '209100102', '209100101', '209100104', '209100105', '209100106', '209100103', '209100107', '209100110', '209100109', '209100111', '209100112', '209100115', '209100118', '209100119', '209200001', '209200002', '209200007', '209200006', '209200008', '209200010', '209200011', '209200012', '209200013', '209200016', '209200017', '209200020', '209200022', '209200024', '209200026', '209200027', '209200030', '209200028', '209200031', '209200035', '209200037', '209200038', '209200039', '209200040', '209200041', '209200044', '209200045', '209200046', '209200047', '209200048', '209200050', '209200051', '209200052', '209200053', '209200054', '209200055', '209200056', '209200057', '209200060', '209200061', '209200059', '209200062', '209200063', '209200064', '209200071', '209200075', '209200076', '209200077', '209200079', '209200080', '209200081', '209200082', '209200084', '209200085', '209200088', '209200086', '209200090', '209200091', '209200096', '209200095', '209200097', '209200098', '209200099', '209200100', '209300001', '209300002', '209300003', '209300004', '209300005', '209300012', '209300015', '209300017', '209300023', '209300026', '209300031', '209300032', '209300033', '209300036', '209300037', '209300038', '209300039', '209300040', '209300041', '209500047', '209300042', '209500048', '209300043', '209500049', '209300044', '209500050', '209300045', '209500052', '209300046', '209500053', '209500054', '209300048', '209500056', '209300051', '209500058', '209300053', '209500057', '209500059', '209300054', '209500062', '209300056', '209500061', '209300058', '209500064', '209300060', '209500065', '209300061', '209500066', '209300062', '209500068', '209300064', '209500069', '209500070', '209500071', '209500074', '209500072', '209500076', '209500077', '209500078', '209300065', '209500080', '209300070', '209300073', '209500081', '209300074', '209500082', '209500083', '209300075', '209300077', '209300078', '209500084', '209300076', '209500085', '209500086', '209300080', '209500087', '209500088', '209300079', '209500089', '209300082', '209500090', '209300081', '209600002', '209300083', '209600004', '209300085', '209600006', '209300084', '209600007', '209300087', '209600008', '209300088', '209600010', '209300086', '209300090', '209600012', '209600013', '209600015', '209600018', '209300091', '209600020', '209300093', '209600022', '209600025', '209300096', '209600026', '209600028', '209300095', '209600029', '209300098', '209600035', '209300097', '209600036', '209600034', '209300100', '209600037', '209300101', '209600040', '209600039', '209300103', '209300104', '209600038', '209300107', '209600043', '209300108', '209600046', '209600047', '209300110', '209600048', '209600050', '209300112', '209600051', '209300111', '209600054', '209600057', '209300116', '209600056', '209600058', '209300118', '209600059', '209300119', '209600060', '209600061', '209600062', '209600063', '209600064', '209300120', '209600066', '209600068', '209300121', '209600067', '209400001', '209600069', '209400002', '209600073', '209600078', '209400006', '209600083', '209400007', '209600084', '209400012', '209600082', '209400011', '209600087', '209600092', '209400013', '209600090', '209400015', '209600091', '209600093', '209400017', '209600094', '209400019', '209600101', '209600098', '209400020', '209600102', '209400024', '209600103', '209600104', '209400023', '209600105', '209400026', '209600108', '209400031', '209600111', '209600110', '209600109', '209600113', '209400030', '209600114', '209600115', '209400032', '209600116', '209400033', '209600117', '209400034', '209600122', '209600123', '209400035', '209600124', '209400036', '209600136', '209400037', '209600139', '209400038', '209700001', '209400043', '209700002', '209400044', '209700003', '209700008', '209700009', '209400046', '209700010', '209400045', '209700011', '209400047', '209400049', '209700015', '209400050', '209700020', '209700024', '209400053', '209400054', '209700036', '209400055', '209700038', '209700039', '209400058', '209700040', '209700046', '209700042', '209700045', '209400057', '209700047', '209700049', '209400062', '209700050', '209700057', '209400064', '209700058', '209400069', '209700059', '209400070', '209700062', '209700064', '209400067', '209700071', '209400071', '209700072', '209700074', '209400072', '209700075', '209400073', '209700076', '209700080', '209400074', '209700083', '209400076', '209700084', '209400075', '209700085', '209400078', '209700086', '209400079', '209700087', '209400081', '209700089', '209700090', '209400080', '209700091', '209400083', '209700093', '209400082', '209700096', '209700097', '209700099', '209700100', '209700102', '209400088', '209700104', '209400090', '209700107', '209400089', '209700108', '209700109', '209400092', '209700112', '209400093', '209700114', '209400094', '209700122', '209400099', '209700127', '209400102', '209700129', '209700133', '209400105', '209700135', '209400107', '209700140', '209700141', '209400110', '209700139', '209500001', '209700143', '209700145', '209500002', '209700144', '209500003', '209700146', '209700147', '209500004', '209700148', '209700149', '209500006', '209700150', '209500009', '209700152', '209700151', '209700153', '209700154', '209700155', '209500010', '209700158', '209700159', '209500011', '209700160', '209500015', '209700161', '209700164', '209700163', '209500019', '209700162', '209500022', '209700165', '209500023', '209700166', '209700167', '209500024', '209700168', '209700169', '209500025', '209700170', '209500026', '209700171', '209700172', '209500028', '209700175', '209500029', '209700179', '209700181', '209500030', '209700182', '209700183', '209500031', '209700184', '209500032', '209700191', '209700192', '209500038', '209700195', '209800001', '209800002', '209800003', '209800004', '209500039', '209800005', '209500040', '209800006', '209800009', '209500041', '209800012', '209500042', '209800013', '209500043', '209800014', '209500045', '209800021', '209800024', '209800025', '209900055', '209800026', '209800030', '209800031', '209800036', '209800037', '209800039', '209800040', '209800042', '209800043', '209800044', '209800046', '209800047', '209800081', '209900056', '209800082', '209900057', '209800087', '209800089', '209800093', '209900060', '209800092', '209800094', '209900059', '209800098', '209800097', '209900061', '209800099', '209800104', '209800105', '209800107', '209800106', '209800108', '209900064', '209800110', '209900063', '209800112', '209900066', '209800113', '209800114', '209800115', '209900070', '209800116', '209900072', '209800117', '209800118', '209900074', '209800119', '209800124', '209900073', '209800122', '209900075', '209800129', '209900077', '209800137', '209800135', '209900076', '209800131', '209800136', '209900079', '209800142', '209900083', '209800143', '209900084', '209800145', '209900085', '209800146', '209900090', '209800148', '209800147', '209900087', '209800149', '209800154', '209800152', '209800155', '209900089', '209800156', '209900093', '209800157', '209800158', '209900092', '209800164', '209900094', '209800173', '209900095', '209800175', '209900096', '209800176', '209800177', '209900099', '209800180', '209900097', '209800188', '209800190', '209900100', '209900001', '209900101', '209900004', '209900005', '209900102', '209900009', '209900010', '209900105', '209900011', '209900012', '209900103', '209900013', '209900016', '209900106', '209900017', '209900018', '209900107', '209900019', '209900020', '209900108', '209900024', '209900027', '209900028', '209900029', '209900110', '209900034', '209900035', '209900111', '209900037', '209900038', '209900112', '209900039', '209900040', '209900119', '209900041', '209900044', '209900046', '209900120', '209900047', '209900121', '209900048', '209900049', '209900050', '209900123', '209900051', '209900129', '209900053', '2091000001', '2091000002', '2091000005', '2091000006', '2091000007', '2091000008', '2091000015', '2091300088', '2091300089', '2091000018', '2091300091', '2091300092', '2091000019', '2091300093', '2091300098', '2091000022', '2091300099', '2091000024', '2091300103', '2091300102', '2091300104', '2091300106', '2091300105', '2091000025', '2091300107', '2091300108', '2091000026', '2091300109', '2091300111', '2091000029', '2091300110', '2091300112', '2091000027', '2091300113', '2091300114', '2091000036', '2091300115', '2091300116', '2091000038', '2091300118', '2091000039', '2091300119', '2091300120', '2091000044', '2091300121', '2091300123', '2091000045', '2091300125', '2091000046', '2091300126', '2091300127', '2091000047', '2091300128', '2091300130', '2091000049', '2091400001', '2091400002', '2091000050', '2091400006', '2091400007', '2091000054', '2091400008', '2091000052', '2091400010', '2091400011', '2091400013', '2091400018', '2091400019', '2091000055', '2091400021', '2091400024', '2091000058', '2091400027', '2091000059', '2091400028', '2091400029', '2091000060', '2091400032', '2091400039', '2091000061', '2091400040', '2091000065', '2091400041', '2091400044', '2091000068', '2091400045', '2091400047', '2091000069', '2091400048', '2091400049', '2091000070', '2091400055', '2091000071', '2091400058', '2091400052', '2091000074', '2091400060', '2091000073', '2091400059', '2091000076', '2091400062', '2091400063', '2091000078', '2091400064', '2091400066', '2091000079', '2091400068', '2091400067', '2091400069', '2091400070', '2091000080', '2091400071', '2091400072', '2091000083', '2091400073', '2091400074', '2091000085', '2091400075', '2091000088', '2091400076', '2091400077', '2091000087', '2091400078', '2091000089', '2091400079', '2091400080', '2091000090', '2091400081', '2091400083', '2091000092', '2091400085', '2091000096', '2091400084', '2091400089', '2091400088', '2091000104', '2091400091', '2091400092', '2091400095', '2091000106', '2091400098', '2091000105', '2091400096', '2091000107', '2091400099', '2091400100', '2091400101', '2091000108', '2091400104', '2091400106', '2091000111', '2091400107', '2091400111', '2091400115', '2091400118', '2091000110', '2091400122', '2091400123', '2091000113', '2091500002', '2091500006', '2091000115', '2091500008', '2091500009', '2091000117', '2091500011', '2091500012', '2091000118', '2091500013', '2091000119', '2091500017', '2091000124', '2091500019', '2091500020', '2091000125', '2091500027', '2091000127', '2091500025', '2091500028', '2091000130', '2091500030', '2091000139', '2091500031', '2091500035', '2091500036', '2091000143', '2091500037', '2091500038', '2091000142', '2091500040', '2091500041', '2091000144', '2091500043', '2091000145', '2091500048', '2091500052', '2091500053', '2091500056', '2091000150', '2091500059', '2091500062', '2091000151', '2091500061', '2091500063', '2091000153', '2091500064', '2091500067', '2091000155', '2091500068', '2091500069', '2091100001', '2091500071', '2091100002', '2091500073', '2091500075', '2091100003', '2091500076', '2091500078', '2091100004', '2091500085', '2091100005', '2091500082', '2091500086', '2091500087', '2091100007', '2091500089', '2091100010', '2091500090', '2091500091', '2091100011', '2091500096', '2091100014', '2091500097', '2091500095', '2091100015', '2091500100', '2091500102', '2091100018', '2091500101', '2091500105', '2091500106', '2091500112', '2091500110', '2091100019', '2091500114', '2091500113', '2091100020', '2091500115', '2091100023', '2091500118', '2091500121', '2091100027', '2091500125', '2091500124', '2091100028', '2091500126', '2091100029', '2091500127', '2091500131', '2091100034', '2091500129', '2091500132', '2091100033', '2091500134', '2091500135', '2091100036', '2091500137', '2091100037', '2091500138', '2091500139', '2091100039', '2091500140', '2091500143', '2091100045', '2091500144', '2091500146', '2091100046', '2091500147', '2091500149', '2091100048', '2091500152', '2091600002', '2091600004', '2091600006', '2091600009', '2091100049', '2091600010', '2091600011', '2091600014', '2091100053', '2091600015', '2091600020', '2091100055', '2091600021', '2091600024', '2091100062', '2091600026', '2091600037', '2091600038', '2091600043', '2091600046', '2091100063', '2091600050', '2091100065', '2091600053', '2091600054', '2091100066', '2091600058', '2091100067', '2091600060', '2091600064', '2091600073', '2091100069', '2091600074', '2091100071', '2091600080', '2091600079', '2091100075', '2091600081', '2091600083', '2091100074', '2091600086', '2091600087', '2091100077', '2091600088', '2091600089', '2091600092', '2091600094', '2091600095', '2091100078', '2091600100', '2091600098', '2091100079', '2091600101', '2091100081', '2091600102', '2091600103', '2091100083', '2091600108', '2091600106', '2091100085', '2091600104', '2091100084', '2091600105', '2091600112', '2091100087', '2091600115', '2091600117', '2091100086', '2091600119', '2091600120', '2091100089', '2091600121', '2091100090', '2091600123', '2091600125', '2091100091', '2091600127', '2091600128', '2091100093', '2091600129', '2091600130', '2091100092', '2091600131', '2091600133', '2091100095', '2091600134', '2091100096', '2091600135', '2091600136', '2091600137', '2091600138', '2091600139', '2091100099', '2091600140', '2091600143', '2091100097', '2091600142', '2091100102', '2091600144', '2091100106', '2091600146', '2091600147', '2091100112', '2091600149', '2091600150', '2091100114', '2091600151', '2091600153', '2091100115', '2091600155', '2091100117', '2091600163', '2091600166', '2091100116', '2091600174', '2091600176', '2091100121', '2091600179', '2091600182', '2091100119', '2091700001', '2091100123', '2091700006', '2091100122', '2091700009', '2091700010', '2091100124', '2091700012', '2091700013', '2091100125', '2091700016', '2091700015', '2091100127', '2091700017', '2091700019', '2091700020', '2091700022', '2091700023', '2091100128', '2091700024', '2091700027', '2091100126', '2091700033', '2091100129', '2091700034', '2091700038', '2091100131', '2091700039', '2091100132', '2091700040', '2091700044', '2091700045', '2091100133', '2091700046', '2091100138', '2091700047', '2091200001', '2091700049', '2091700050', '2091200003', '2091700051', '2091700052', '2091200006', '2091700055', '2091700056', '2091200008', '2091700057', '2091200009', '2091700058', '2091200010', '2091700060', '2091700061', '2091200011', '2091700062', '2091700063', '2091200012', '2091700064', '2091700066', '2091700067', '2091700065', '2091700068', '2091200018', '2091700069', '2091200019', '2091700070', '2091700071', '2091200023', '2091700073', '2091700074', '2091200024', '2091700076', '2091200025', '2091700078', '2091700080', '2091200027', '2091700081', '2091700086', '2091700087', '2091200030', '2091700089', '2091200029', '2091700090', '2091700092', '2091200034', '2091700097', '2091700095', '2091200033', '2091700099', '2091200036', '2091700107', '2091700110', '2091200037', '2091700111', '2091200038', '2091700114', '2091200039', '2091700116', '2091700117', '2091200046', '2091700120', '2091700123', '2091700125', '2091700124', '2091700122', '2091200048', '2091700126', '2091200050', '2091200053', '2091700127', '2091200058', '2091700128', '2091700130', '2091200061', '2091700133', '2091700134', '2091200066', '2091700135', '2091200064', '2091700138', '2091700141', '2091200069', '2091700142', '2091200074', '2091700144', '2091700145', '2091200076', '2091700146', '2091700147', '2091200077', '2091700149', '2091700151', '2091200080', '2091700155', '2091800001', '2091200083', '2091800005', '2091200084', '2091800006', '2091800007', '2091200085', '2091800009', '2091200086', '2091800014', '2091800015', '2091200087', '2091800017', '2091800021', '2091800025', '2091800028', '2091200088', '2091800032', '2091800034', '2091200089', '2091800036', '2091200090', '2091800037', '2091800038', '2091200091', '2091800039', '2091800041', '2091200092', '2091800043', '2091200093', '2091800047', '2091800048', '2091200094', '2091800051', '2091200095', '2091800054', '2091200101', '2091800055', '2091800056', '2091200102', '2091800057', '2091200103', '2091800061', '2091800067', '2091200107', '2091800068', '2091800070', '2091200105', '2091800069', '2091800073', '2091200106', '2091800075', '2091200104', '2091800076', '2091800079', '2091200112', '2091800084', '2091800086', '2091800091', '2091800094', '2091200113', '2091800096', '2091800098', '2091200117', '2091800099', '2091800101', '2091200118', '2091800104', '2091200119', '2091800106', '2091800109', '2091200120', '2091800111', '2091800112', '2091200121', '2091800115', '2091200122', '2091800113', '2091800116', '2091200124', '2091800117', '2091800118', '2091200125', '2091800119', '2091200128', '2091800114', '2091800120', '2091200133', '2091800121', '2091800126', '2091300001', '2091800129', '2091800130', '2091300004', '2091800131', '2091300007', '2091800132', '2091800134', '2091800135', '2091800139', '2091800138', '2091800140', '2091300008', '2091800142', '2091800144', '2091300009', '2091800148', '2091800149', '2091300011', '2091800151', '2091800156', '2091300013', '2091800159', '2091300014', '2091800158', '2091800161', '2091300017', '2091800163', '2091300018', '2091800164', '2091300022', '2091800166', '2091800167', '2091300023', '2091800173', '2091800174', '2091300025', '2091300026', '2091800177', '2091800179', '2091300027', '2091800182', '2091800184', '2091300029', '2091900004', '2091900006', '2091300030', '2091900011', '2091300031', '2091900018', '2091300032', '2091900024', '2091900026', '2091900025', '2091900033', '2091300033', '2091900034', '2091900035', '2091300034', '2091900036', '2091900037', '2091300036', '2091900038', '2091300037', '2091900045', '2091900060', '2091300038', '2091900062', '2091300040', '2091900061', '2091900069', '2091300044', '2091900082', '2091900083', '2091300042', '2091900095', '2091300051', '2091900094', '2091900096', '2091300055', '2091900111', '2091300058', '2091900112', '2091900115', '2091300059', '2091900118', '2091300060', '2091900127', '2091300061', '2091900135', '2091900134', '2091300062', '2091900138', '2091300063', '2091900142', '2091900150', '2091300067', '2091900149', '2091900151', '2091900153', '2091900156', '2091900157', '2091300066', '2091900160', '2091900161', '2091300065', '2091900162', '2091300068', '2091900163', '2091900165', '2091300069', '2091900166', '2091300070', '2091900167', '2091900168', '2091300071', '2091900170', '2091300073', '2091900171', '2091900172', '2091300078', '2091900174', '2091300080', '2091900175', '2091900176', '2092100078', '2091900177', '2091900178', '2091900180', '2091900179', '2091900181', '2091900182', '2091900183', '2091900184', '2091900187', '2091900186', '2091900188', '2091900192', '2091900199', '2091900201', '2091900198', '2091900203', '2091900205', '2091900207', '2091900208', '2091900210', '2091900212', '2091900218', '2091900220', '2092100130', '2091900224', '2091900226', '2091900233', '2091900234', '2092100136', '2091900236', '2091900237', '2092100144', '2091900241', '2091900240', '2091900245', '2092100145', '2091900244', '2091900247', '2092100148', '2091900248', '2091900250', '2092100149', '2091900257', '2092100150', '2091900260', '2092100151', '2091900264', '2092100153', '2091900265', '2092100152', '2091900267', '2091900269', '2092100161', '2092000001', '2092100163', '2092000002', '2092100164', '2092000004', '2092000007', '2092000008', '2092000009', '2092000010', '2092100166', '2092000015', '2092000018', '2092100168', '2092000029', '2092000030', '2092100172', '2092000031', '2092000032', '2092000033', '2092100173', '2092000037', '2092100175', '2092000039', '2092100177', '2092000041', '2092100183', '2092000042', '2092000043', '2092100185', '2092000045', '2092100189', '2092000051', '2092100196', '2092000053', '2092100197', '2092000055', '2092000056', '2092100199', '2092000057', '2092100203', '2092000059', '2092000072', '2092100201', '2092000062', '2092000079', '2092100205', '2092100213', '2092000085', '2092000086', '2092100219', '2092000088', '2092000089', '2092000090', '2092000092', '2092000093', '2092100216', '2092000094', '2092000096', '2092100217', '2092000097', '2092100218', '2092000098', '2092000100', '2092100220', '2092000101', '2092100223', '2092000102', '2092000103', '2092000104', '2092100231', '2092000105', '2092000108', '2092100230', '2092000106', '2092000109', '2092100237', '2092000112', '2092100239', '2092000113', '2092000116', '2092100241', '2092000117', '2092100243', '2092000118', '2092100244', '2092000121', '2092100246', '2092000122', '2092100247', '2092000123', '2092100251', '2092000127', '2092100253', '2092000131', '2092000133', '2092000135', '2092000138', '2092100258', '2092000140', '2092100259', '2092000141', '2092000143', '2092100260', '2092000144', '2092100261', '2092000145', '2092000146', '2092100267', '2092000147', '2092000149', '2092100263', '2092000150', '2092100265', '2092000151', '2092000153', '2092100271', '2092100272', '2092000154', '2092000155', '2092100273', '2092000156', '2092100274', '2092000157', '2092100276', '2092000158', '2092000159', '2092100279', '2092000160', '2092100278', '2092000162', '2092100277', '2092000165', '2092000166', '2092100286', '2092000167', '2092100289', '2092000169', '2092100293', '2092000171', '2092000173', '2092000176', '2092000177', '2092000178', '2092100001', '2092100297', '2092100002', '2092100004', '2092100303', '2092100005', '2092100304', '2092100006', '2092100305', '2092100023', '2092100308', '2092100039', '2092100309', '2092100045', '2092100046', '2092100311', '2092100047', '2092100048', '2092200001', '2092100049', '2092200002', '2092100050', '2092200003', '2092100051', '2092200004', '2092100053', '2092200005', '2092100054', '2092100055', '2092200006', '2092100059', '2092100062', '2092200007', '2092100064', '2092200008', '2092100065', '2092200010', '2092100070', '2092200009', '2092100068', '2092200011', '2092200013', '2092200019', '2092200020', '2092400059', '2092200025', '2092400061', '2092400063', '2092200026', '2092400064', '2092200028', '2092400065', '2092200033', '2092400066', '2092400067', '2092200035', '2092400068', '2092400069', '2092200040', '2092400072', '2092400075', '2092200039', '2092400076', '2092200043', '2092400077', '2092400079', '2092200044', '2092400081', '2092200045', '2092400082', '2092200050', '2092400084', '2092400085', '2092400086', '2092200052', '2092400088', '2092200053', '2092400090', '2092200054', '2092400091', '2092200055', '2092400093', '2092400095', '2092400097', '2092400100', '2092400101', '2092400105', '2092400103', '2092200061', '2092400107', '2092400110', '2092400106', '2092200063', '2092400109', '2092400111', '2092200069', '2092400112', '2092200070', '2092400113', '2092400114', '2092200071', '2092400115', '2092400116', '2092200072', '2092400117', '2092200073', '2092400119', '2092200074', '2092400120', '2092400121', '2092200075', '2092400122', '2092200076', '2092400123', '2092400124', '2092200078', '2092400126', '2092200080', '2092400127', '2092400128', '2092200083', '2092400129', '2092200086', '2092400131', '2092400130', '2092200087', '2092400132', '2092400133', '2092400134', '2092400136', '2092400138', '2092200090', '2092400140', '2092200089', '2092400141', '2092200092', '2092400143', '2092400144', '2092200093', '2092400146', '2092400145', '2092200097', '2092400148', '2092200099', '2092200098', '2092400149', '2092400150', '2092200100', '2092400151', '2092400153', '2092200102', '2092400152', '2092400155', '2092200103', '2092400156', '2092200104', '2092400157', '2092400158', '2092200105', '2092400159', '2092200106', '2092400161', '2092400160', '2092200107', '2092400162', '2092400164', '2092200108', '2092400166', '2092400167', '2092200110', '2092400168', '2092400170', '2092400172', '2092400173', '2092400184', '2092200112', '2092400188', '2092400189', '2092200113', '2092400192', '2092200114', '2092400191', '2092400194', '2092200115', '2092400195', '2092400196', '2092200116', '2092200118', '2092400197', '2092200120', '2092400199', '2092400200', '2092200119', '2092400201', '2092400202', '2092200122', '2092400203', '2092400204', '2092200121', '2092400205', '2092200123', '2092400206', '2092500001', '2092200124', '2092500002', '2092200126', '2092500003', '2092500004', '2092200127', '2092500005', '2092200130', '2092500006', '2092200129', '2092500010', '2092500012', '2092500013', '2092500016', '2092500017', '2092200133', '2092500018', '2092500020', '2092200134', '2092500024', '2092200135', '2092500026', '2092500027', '2092200136', '2092500031', '2092500028', '2092200137', '2092200138', '2092500032', '2092500033', '2092200140', '2092500034', '2092500037', '2092200143', '2092500038', '2092200145', '2092500039', '2092500043', '2092200147', '2092500044', '2092200148', '2092500046', '2092500045', '2092200149', '2092500047', '2092500048', '2092200152', '2092500052', '2092300001', '2092500054', '2092500055', '2092300002', '2092500057', '2092300005', '2092500058', '2092500064', '2092500063', '2092500067', '2092500065', '2092300007', '2092500068', '2092300008', '2092500069', '2092500070', '2092300009', '2092500071', '2092500072', '2092300010', '2092500073', '2092300011', '2092500074', '2092500076', '2092500078', '2092300012', '2092500077', '2092300013', '2092500080', '2092500081', '2092300014', '2092500083', '2092300035', '2092500087', '2092500089', '2092500090', '2092300037', '2092500091', '2092300040', '2092500092', '2092500093', '2092300041', '2092500095', '2092500097', '2092300043', '2092500099', '2092300050', '2092500100', '2092500101', '2092300051', '2092500103', '2092500105', '2092500108', '2092500109', '2092300054', '2092500112', '2092500113', '2092300057', '2092500114', '2092500116', '2092300056', '2092500117', '2092300061', '2092500118', '2092500119', '2092300066', '2092500120', '2092500121', '2092300065', '2092500122', '2092500124', '2092300072', '2092500126', '2092300075', '2092500125', '2092500127', '2092300076', '2092500128', '2092500130', '2092300079', '2092500131', '2092300083', '2092500132', '2092300081', '2092500133', '2092500134', '2092300084', '2092500136', '2092500137', '2092300086', '2092500135', '2092300092', '2092500141', '2092500138', '2092500144', '2092500143', '2092500145', '2092500146', '2092300094', '2092500147', '2092500148', '2092300095', '2092500152', '2092500153', '2092300096', '2092500154', '2092300097', '2092500158', '2092500161', '2092300098', '2092500162', '2092300099', '2092500165', '2092500167', '2092300100', '2092500168', '2092500170', '2092300101', '2092600004', '2092600005', '2092300102', '2092600006', '2092600007', '2092300103', '2092600008', '2092300104', '2092600009', '2092600010', '2092300105', '2092600011', '2092600014', '2092300106', '2092600017', '2092300108', '2092600019', '2092300109', '2092600020', '2092300110', '2092600023', '2092600024', '2092600025', '2092600026', '2092600028', '2092300112', '2092600029', '2092600030', '2092300113', '2092600032', '2092300114', '2092600033', '2092600034', '2092300115', '2092600035', '2092300116', '2092600041', '2092600044', '2092300117', '2092600049', '2092300118', '2092600050', '2092600056', '2092300120', '2092600059', '2092600067', '2092300121', '2092600069', '2092300122', '2092600073', '2092300123', '2092600077', '2092600079', '2092300124', '2092600081', '2092300126', '2092600085', '2092600086', '2092300127', '2092600088', '2092600087', '2092300128', '2092600089', '2092600090', '2092600091', '2092600093', '2092300129', '2092600098', '2092600107', '2092300130', '2092600106', '2092600110', '2092300131', '2092600112', '2092300132', '2092600113', '2092600114', '2092300134', '2092600116', '2092300133', '2092600117', '2092600118', '2092300135', '2092600119', '2092300136', '2092600120', '2092600124', '2092300137', '2092600122', '2092600123', '2092300138', '2092600126', '2092300140', '2092600131', '2092600132', '2092300139', '2092600134', '2092600137', '2092300141', '2092600139', '2092300142', '2092600138', '2092600140', '2092300143', '2092600144', '2092300144', '2092600143', '2092600145', '2092600146', '2092600147', '2092600142', '2092300145', '2092600148', '2092600149', '2092300146', '2092600150', '2092300149', '2092600152', '2092600153', '2092300150', '2092600154', '2092600155', '2092300152', '2092600156', '2092300151', '2092600159', '2092300153', '2092600158', '2092300154', '2092600160', '2092300155', '2092600161', '2092600162', '2092300156', '2092600163', '2092300157', '2092600164', '2092600165', '2092300158', '2092600167', '2092300159', '2092600166', '2092600168', '2092300160', '2092600170', '2092300161', '2092600171', '2092300162', '2092600173', '2092600172', '2092300163', '2092600174', '2092600175', '2092600177', '2092300168', '2092600178', '2092600179', '2092600180', '2092300170', '2092600181', '2092600183', '2092300175', '2092600184', '2092300176', '2092600186', '2092600189', '2092300177', '2092600193', '2092300179', '2092700001', '2092300180', '2092700003', '2092700004', '2092400001', '2092700010', '2092400004', '2092700011', '2092400005', '2092700012', '2092400009', '2092700013', '2092400013', '2092700014', '2092700016', '2092400014', '2092700018', '2092700019', '2092400017', '2092700020', '2092700021', '2092400018', '2092700022', '2092400020', '2092700023', '2092400023', '2092700024', '2092700025', '2092700026', '2092700027', '2092400032', '2092700033', '2092700034', '2092400040', '2092700035', '2092700038', '2092400041', '2092700040', '2092400044', '2092700041', '2092700042', '2092400046', '2092700043', '2092700045', '2092400051', '2092700048', '2092400052', '2092700051', '2092700050', '2092400054', '2092700053', '2092400055', '2092700055', '2092400057', '2092700057', '2092700058', '2092400060', '2092700060', '2092700062', '2092700063', '2092700064', '2092700066', '2092700067', '2092700065', '2092700068', '2092700073', '2092700074', '2092700080', '2092700083', '2092700085', '2092700088', '2092700086', '2092700089', '2092700091', '2092700092', '2092700093', '2092700094', '2092700095', '2092700096', '2092700098', '2092700097', '2092700100', '2092700102', '2092700103', '2092700105', '2092700110', '2092700108', '2092700113', '2092700112', '2092700116', '2092700117', '2092700118', '2092700120', '2092700121', '2092700123', '2092700125', '2092700127', '2092700130', '2092700133', '2092700132', '2092700131', '2092700134', '2092700136', '2092700137', '2092700138', '2092700139', '2092700141', '2092700140', '2092700142', '2092700145', '2092700146', '2092700147', '2092700148', '2092700150', '2092700151', '2092700154', '2092700153', '2092700156', '2092700155', '2092700157', '2092700160', '2092700161', '2092700162', '2092700164', '2092700165', '2092700167', '2092700168', '2092700170', '2092700173', '2092700174', '2092700176', '2092700177', '2092700178', '2092700179', '2092700180', '2092700182', '2092700184', '2092700188', '2092700187', '2092700191', '2092700190', '2092800001', '2092800002', '2092800003', '2092800004', '2092800005', '2092800006', '2092800007', '2092800011', '2092800012', '2092800016', '2092800017', '2092800019', '2092800018', '2092800020', '2092800021', '2092800025', '2092800024', '2092800027', '2092800028', '2092800031', '2092800032', '2092800033', '2092800034', '2092800036', '2092800037', '2092800040', '2092800039', '2092800041', '2092800042', '2092800043', '2092800044', '2092800045', '2092800046', '2092800048', '2092800052', '2092800056', '2092800058', '2092800057', '2092800059', '2092800063', '2092800066', '2092800068', '2092800071', '2092800072', '2092800073', '2092800074', '2092800075', '2092800076', '2092800080', '2092800083', '2092800084', '2092800086', '2092800085', '2092800087', '2092800088', '2092800089', '2092800091', '2092800092', '2092800093', '2092800095', '2092800096', '2092800098', '2092800100', '2092800101', '2092800102', '2092800105', '2092800106', '2092800107', '2092800109', '2092800110', '2092800111', '2092800112', '2092800114', '2092800115', '2092800116', '2092800118', '2092800121', '2092800123', '2092800126', '2092800127', '2092800128', '2092800129', '2092800131', '2092800132', '2092800134', '2092800137', '2092800136', '2092800138', '2092800139', '2092800140', '2092800141', '2092800142', '2092800144', '2092800145', '2092800146', '2092800147', '2092800154', '2092800155', '2092800157', '2092800158', '2092800161', '2092800162', '2092800163', '2092800166', '2092800170', '2092800171', '2092800172', '2092800173', '2092800174', '2092800175', '2092800176', '2092800177', '2092800178', '2092800179', '2092800182', '2092800183', '2092800184', '2092800186', '2092800188', '2092800189', '2092800192', '2092900012', '2092900043', '2092900063', '2092900072', '2092900075', '209300069', '209100006', '209400087', '209700173', '209700176', '209700178', '209800038', '209800048', '209800111', '209800168', '2091000072', '2091200035', '2091200110', '2091400016', '2091400094', '2091800088', '2091800090', '2091800105', '2092000048', '2092200139', '2092100255', '2092400015', '2092400186', '2092500140', '2092600021', '2092600083', '2092700007', '2092700152', '209900015']

In [167]:
df = pd.read_excel('online_excel/online-Sep 2020.xlsx')
df1 = pd.read_excel('online_excel/test_sep.xlsx')


# df1=pd.DataFrame(list(PaymentRequest.objects.filter(rid__in=list(df['Enquiry Id'])).values('rid','amt')))
df1['rid'] = df1['rid'].astype(str)
df1['amt'] = df1['amt'].astype(float)

df2=df.groupby('Enquiry Id').agg({'Total Amount': sum}).reset_index()
df2 = df2.rename(columns={'Enquiry Id': 'rid', 'Total Amount': 'amt1'})
df2['rid'] = df2['rid'].astype(str)
df2['amt1'] = df2['amt1'].astype(float)

final_df = pd.merge(df2, df1, left_on='rid', right_on='rid', how='right')
final_df = final_df.fillna(0)
final_df['diff'] = final_df['amt1'] - final_df['amt']
final_df[final_df['diff'] != 0]

,rid,amt1,amt,diff
2579,2082800007,0.0,343.0,-343.0


In [168]:
final_df = pd.merge(df2, df1, left_on='rid', right_on='rid', how='left')
final_df = final_df.fillna(0)
final_df['diff'] = final_df['amt1'] - final_df['amt']
final_df[final_df['diff'] != 0]

,rid,amt1,amt,diff
661,2082400027,2400.0,0.0,2400.0


In [72]:
9764832.77

In [59]:
df = pd.read_excel('online_excel/online-Aug 2020.xlsx')

df['Total Amount'].sum()

8604264.27

In [17]:
8604264.27 - 26521.5 - 9764832.77 

-1187090.0

In [34]:
 8604264.27 - 8602209.27 - 2025

30.0

In [41]:
# month 8  --- > amount match 30 
amount = 9764862.77
amount_miss = 2025
amount_carry_over = 1189145
past_month_amount = 26521.5

bank_excel = 8604264.27

9764862.77 - 1189145 + 26521.5 + 2025


8604264.27

In [48]:
# month 9  --- > amount not carry over 343
amount = 18697246.68
amount_miss = 7524
amount_carry_over = 1307468
past_month_amount = 1189488

bank_excel = 18586447.68

18697246.68 - 1307468 + 1189488 + 7524

18586790.68

In [49]:
18586790.68 - 18586447.68

343.0

In [163]:
df1 = pd.read_excel('online_excel/test_sep.xlsx')
df1['amt'].sum()

18584390.68

In [56]:
8179722.27+1185815

9365537.27

In [35]:
9764832.77 - 9765273.77 

-441.0

In [169]:
rid_list = ['2012900003','2012300208','2111300193','211700051','2111100202','2122300054','2122200065','2122600116','2122600201','2122600437','2132600368','2132300152']

In [3]:
PaymentRequestResponse.objects.filter(rid__in= rid_list)

NameError: name 'rid_list' is not defined

In [2]:
icustomer_list = list(ICustomerSaleGroup.objects.filter(time_created__date__range=['2021-07-01', '2021-07-06']).values_list('icustomer__customer_code', 'business__code', 'business__zone__name', 'total_cost_for_month', 'icustomer__icustomerwallet__current_balance'))                               
icustomer_column = ['Customer Code', 'Booth Code', 'Zone', 'Total Amount', 'Wallet Balace']
icustomer_df = pd.DataFrame(icustomer_list, columns=icustomer_column)
icustomer_df = icustomer_df.groupby('Customer Code').agg({'Booth Code': 'first', 'Zone': 'first', 'Total Amount': sum, 'Wallet Balace': 'first'}).reset_index()
icustomer_df[['Total Amount', 'Wallet Balace']] = icustomer_df[['Total Amount', 'Wallet Balace']].astype(float)
icustomer_df.index += 1
icustomer_df.to_excel('customer_order_from_Jul01_2021_to_Jul06_2021.xlsx')